<a href="https://colab.research.google.com/github/kaminovs/CR-SSCP/blob/main/notebooks/CR_SSCP_v5_7_10_ROUTING_TASKFRAMES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CR-SSCP CR-SSCP v5.7.10 — Routing + TaskFrames + Hard Action Gating



In [1]:
from datetime import datetime
# CELL 1: Installation and Setup
print("Installing dependencies...")
!pip install -q transformers accelerate bitsandbytes sentencepiece protobuf
print("Installation complete!")

Installing dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 46.0 MB/s eta 0:00:00
Installation complete!


In [2]:
# CELL 2: Imports + (optional) Google Drive mount
import ast, operator
from datetime import datetime
import os, json, time, math, random
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional
import numpy as np

import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# BitsAndBytesConfig is optional (works only if bitsandbytes + compatible GPU available)
try:
    from transformers import BitsAndBytesConfig
    BNB_AVAILABLE = True
except Exception as _e:
    BitsAndBytesConfig = None
    BNB_AVAILABLE = False

print("✓ Imports complete")

# Mount Google Drive for persistence (optional)
DRIVE_AVAILABLE = False
try:
    from google.colab import drive
    drive.mount('/content/drive')
    DRIVE_AVAILABLE = True
    print("✓ Drive mounted")
except Exception as e:
    print("⚠ Drive mount failed (continuing without Drive persistence):", e)


✓ Imports complete
Mounted at /content/drive
✓ Drive mounted


# CR-SSCP v4.2 — Sandbox World Agent (Variant 1)

Adds a minimal causal sandbox world with objects: **lamp**, **box**, **door**.

Law: `Intent → WORLD_ACT → World Δ → Observation → Update`


In [3]:
# === v4.2 Sandbox World Engine (minimal causal world) ===
from typing import Dict, Tuple, Any
import copy

class SandboxEngine:
    def __init__(self):
        self.reset()

    def reset(self):
        self.world = {
            'objects': {
                'lamp': {'state': 'off'},
                'box': {'state': 'closed'},
                'door': {'state': 'locked'}
            },
            'agent': {'location': 'room', 'energy': 1.0},
            'time': 0
        }
        return self.world

    def peek(self, action: Dict[str, Any]) -> Tuple[Dict, Dict]:
        snap = copy.deepcopy(self.world)
        obs = self._apply(action)
        pred_world = copy.deepcopy(self.world)
        self.world = snap
        return pred_world, obs

    def step(self, action: Dict[str, Any]) -> Tuple[Dict, Dict]:
        obs = self._apply(action)
        return self.world, obs

    def _apply(self, action: Dict[str, Any]) -> Dict:
        self.world['time'] += 1
        act = (action or {}).get('act', '')
        target = (action or {}).get('target', '')
        w = self.world
        before = copy.deepcopy(w)

        success = False
        msg = 'no-op'
        if target in w['objects']:
            obj = w['objects'][target]
            if target == 'lamp' and act in ['toggle','on','off']:
                if act == 'toggle':
                    obj['state'] = 'on' if obj['state']=='off' else 'off'
                else:
                    obj['state'] = act
                success = True
                msg = f"lamp is now {obj['state']}"
            elif target == 'box' and act in ['open','close']:
                obj['state'] = 'open' if act=='open' else 'closed'
                success = True
                msg = f"box is now {obj['state']}"
            elif target == 'door' and act in ['unlock','lock','open','close']:
                if act == 'unlock':
                    obj['state'] = 'unlocked'
                    success = True
                    msg = 'door unlocked'
                elif act == 'lock':
                    obj['state'] = 'locked'
                    success = True
                    msg = 'door locked'
                elif act == 'open':
                    if obj['state'] in ['unlocked','open']:
                        obj['state'] = 'open'
                        success = True
                        msg = 'door opened'
                    else:
                        success = False
                        msg = 'door is locked'
                elif act == 'close':
                    # close an open door back to 'unlocked'
                    if obj['state'] == 'open':
                        obj['state'] = 'unlocked'
                        success = True
                        msg = 'door closed'
                    else:
                        success = False
                        msg = 'door not open'

        if success:
            w['agent']['energy'] = max(0.0, w['agent']['energy'] - 0.02)

        delta = 0
        for k in ['lamp','box','door']:
            if before['objects'][k]['state'] != w['objects'][k]['state']:
                delta += 1
        return {
            'success': success,
            'message': msg,
            'delta': delta,
            'time': w['time'],
            'objects': copy.deepcopy(w['objects']),
            'agent_energy': w['agent']['energy']
        }

if 'sandbox_engine' not in globals():
    sandbox_engine = SandboxEngine()


In [4]:

# ============================================================================
# v5.7.3 Patch: Control simulated user input + anti-repetition + EU calibration
# ============================================================================
from datetime import datetime
import re

# When running scripted tests (enqueue_user_event), keep this False.
# Set True only for autonomous "user_sim" mode.
DEFAULT_SIMULATE_USER_INPUT = False

CALIBRATED_UTILITIES = {
    'ANSWER': 0.30,            # was ~1.20
    'WORLD_ACT': 0.35,         # was ~1.50
    'REFLECT': 0.18,           # was ~0.50
    'META': 0.12,              # was ~0.20
    'VERIFY': 0.18,
    'SLEEP': 0.22,
}

def get_calibrated_utility(action_type: str, *, user_present: bool=False, is_question: bool=False, has_goal: bool=False) -> float:
    base = float(CALIBRATED_UTILITIES.get(str(action_type or '').upper(), 0.18))
    # Context boosts
    if user_present and str(action_type).upper() in ('ANSWER','WORLD_ACT'):
        base *= 1.35
    if is_question and str(action_type).upper() == 'ANSWER':
        base *= 1.25
    if has_goal and str(action_type).upper() == 'WORLD_ACT':
        base *= 1.20
    return float(base)

class RepetitionDetector:
    """Detect repetitive 'self-maintenance' / 'no active goal' loops in outputs."""
    def __init__(self, window=12):
        self.window = int(window)
        self.recent = []
        self.phrases = [
            'self-maintenance', 'no active goal', 'checking system',
            'ensuring all functions', 'optimal performance', 'system integrity'
        ]
    def add(self, text: str):
        t = (text or '').lower().strip()
        if not t:
            return
        self.recent.append(t)
        if len(self.recent) > self.window:
            self.recent = self.recent[-self.window:]
    def is_repetitive(self) -> bool:
        if len(self.recent) < max(5, self.window//2):
            return False
        recent = self.recent[-10:]
        hits = sum(1 for t in recent if any(p in t for p in self.phrases))
        # if >= 4 of last 10 are maintenance-like, treat as loop
        return hits >= 4

repetition_detector = RepetitionDetector()

def maybe_force_break_repetition(state: dict, proposals: list) -> None:
    """If repetition detected, penalize REFLECT/META and boost WORLD_ACT exploration."""
    if not repetition_detector.is_repetitive():
        return
    state.setdefault('counters', {})
    state['counters']['repetition_breaks'] = int(state['counters'].get('repetition_breaks', 0)) + 1
    for p in proposals:
        at = str(p.get('action_type','')).upper()
        if at in ('REFLECT','META'):
            p['cost'] = float(p.get('cost', 0.0)) + 0.75
            p['expected_utility'] = float(p.get('expected_utility', 0.0)) * 0.25
        if at == 'WORLD_ACT':
            p['expected_utility'] = float(p.get('expected_utility', 0.0)) + 0.35
            p['cost'] = max(0.0, float(p.get('cost', 0.0)) - 0.05)

def normalize_simulate_user_input_flag(state: dict) -> None:
    state.setdefault('policy', {})
    if 'simulate_user_input' not in state['policy']:
        state['policy']['simulate_user_input'] = bool(DEFAULT_SIMULATE_USER_INPUT)


# CR-SSCP v4.3 — Goal-Bound Exploratory Agency

Exploratory active inference policy:
- Extract world goals from user events (lamp/box/door).
- Strongly bias arbitration toward WORLD_ACT when goal exists.
- Reward world delta (Δ) and penalize repeated META/no-op.


In [5]:
# === v4.3 Goal extraction helpers ===
from typing import Optional
import time

def extract_goal(scene: str) -> Optional[dict]:
    if not isinstance(scene, str):
        return None
    s = scene.lower().strip()
    target = None
    for t in ['lamp', 'box', 'door']:
        if t in s:
            target = t
            break
    if not target:
        return None

    if target == 'lamp':
        if any(w in s for w in ['turn on', 'switch on', 'light on']):
            act = 'on'
        elif any(w in s for w in ['turn off', 'switch off', 'light off']):
            act = 'off'
        elif 'toggle' in s:
            act = 'toggle'
        else:
            act = 'toggle'
    elif target == 'box':
        act = 'close' if 'close' in s else 'open'
    else:  # door
        if 'unlock' in s:
            act = 'unlock'
        elif 'lock' in s:
            act = 'lock'
        elif 'open' in s:
            act = 'open'
        else:
            act = 'unlock'
    return {'act': act, 'target': target}


# === v5.7.5: ensure WORLD_ACT proposal for imperative goals ===

def force_world_act_if_goal(event_text: str, proposals: list) -> list:
    """Boost or inject WORLD_ACT when an imperative command is detected.
    v5.7.6: uses stronger but bounded bias toward acting, to avoid 'just talking'.
    """
    try:
        g = extract_goal(event_text)
    except Exception:
        g = None
    if not g:
        return proposals

    # If a world action already exists, boost it
    for p in proposals:
        if str(p.get('action_type','')).upper() == 'WORLD_ACT':
            p['expected_utility'] = float(p.get('expected_utility', 0.0)) + 1.00
            p['cost'] = max(0.0, float(p.get('cost', 0.0)) - 0.25)
            # Slightly reduce risk for simple sandbox actions
            p['risk'] = max(0.0, float(p.get('risk', 0.10)) - 0.03)
            return proposals

    # Otherwise, add a minimal world action proposal
    proposals.append({
        'proposal_id': f"forced_world_{int(time.time())}",
        'module': 'WORLD',
        'intent': f"Execute imperative goal: {g.get('act')} {g.get('target')}",
        'action_type': 'WORLD_ACT',
        'action': {'act': g.get('act'), 'target': g.get('target')},
        # Ensure it beats REFLECT/META when a clear imperative exists
        'expected_utility': max(
            float(get_calibrated_utility('WORLD_ACT', user_present=True, has_goal=True)),
            1.05
        ),
        'risk': 0.06,
        'cost': 0.12,
    })
    return proposals



# CR-SSCP v4.0 — Event-Driven Conscious Core

This version introduces the **Event Lifecycle Law**:

`NEW → INTERPRETED → ACTED → CLOSED`

Perception must be *consumed* by action or dismissal to preserve forward temporal flow.

VERIFY is a transitional stage, but every event must eventually be resolved by ACT (answer/tool/repair) and then CLOSED.


In [6]:
# === Event Lifecycle Core (v5.4.3) ===
from typing import Dict
import uuid

def create_event(content: str, tick: int, *, priority: float=1.0, ttl: int=None, source: str='user') -> Dict:
    """Create a workspace event.

    status: NEW / INTERPRETED / ACTED / RETRY / CLOSED / ARCHIVED
    ttl is in ticks (defaults to Config.EVENT_TTL if available).
    """
    _ttl = ttl
    try:
        if _ttl is None and 'Config' in globals():
            _ttl = int(getattr(Config, 'EVENT_TTL', 30))
    except Exception:
        pass
    if _ttl is None:
        _ttl = 30
    return {
        'id': f"event_{uuid.uuid4().hex[:8]}",
        'content': content,
        'source': source,
        'status': 'NEW',
        'priority': float(priority),
        'ttl': int(_ttl),
        'rounds': 0,
        'event_type': None,
        'created_tick': int(tick),
        'last_update_tick': int(tick),
    }

def mark_interpreted(event: Dict, tick: int):
    if str(event.get('status','')).upper() == 'NEW':
        event['status'] = 'INTERPRETED'
    event['last_update_tick'] = int(tick)

def mark_acted(event: Dict, tick: int):
    event['status'] = 'ACTED'
    event['last_update_tick'] = int(tick)

def close_event(event: Dict, tick: int):
    """Close event only when completion==DONE; otherwise mark RETRY and increment rounds."""
    completion = str(event.get('_last_completion', 'DONE')).upper()
    if completion == 'DONE':
        event['status'] = 'CLOSED'
        # v5.5: ensure closed events cannot re-enter focus
        event['ttl'] = 0
        event['closed_tick'] = int(tick)
    else:
        event['status'] = 'RETRY'
        event['rounds'] = int(event.get('rounds', 0)) + 1
    event['last_update_tick'] = int(tick)

def tick_event_lifecycle(state: Dict, tick: int):
    """Decrement TTL, archive stale events, and keep statuses consistent."""
    ws = state.setdefault('workspace', {})
    events = ws.setdefault('events', [])
    max_rounds = 6
    try:
        if 'Config' in globals():
            max_rounds = int(getattr(Config, 'EVENT_MAX_ROUNDS', 6))
    except Exception:
        pass

    for e in events:
        st = str((e or {}).get('status','NEW')).upper()
        if st in ('CLOSED','ARCHIVED'):
            continue
        try:
            e['ttl'] = int((e or {}).get('ttl', 30)) - 1
        except Exception:
            e['ttl'] = 29

        if int((e or {}).get('ttl', 0)) <= 0 or int((e or {}).get('rounds', 0)) >= max_rounds:
            e['status'] = 'ARCHIVED'
            e['last_update_tick'] = int(tick)


In [7]:
# === v5.6 Patch (safe): close semantics + stuck replanning prompt ===
import re

def apply_v56_patch():
    print("Applying v5.6 patch (safe)...")

    # --- Domain-aware replanning prompt after STUCK ---
    def make_stuck_replan_prompt(state: dict, current_event: dict, result: dict) -> str:
        content = str((current_event or {}).get('content', ''))
        out = str((result or {}).get('output', '')).lower()

        if 'door' in content.lower() or 'door' in out:
            if 'locked' in out:
                return (
                    "The door appears locked and repeating the same action isn’t progressing. "
                    "Should I try an UNLOCK action, look for a key (if supported), "
                    "or switch to another task?"
                )
            return (
                "I’m stuck interacting with the door. What outcome do you want (open/unlock/leave), "
                "and do we have constraints like a key, code, or permission?"
            )

        if 'box' in content.lower() or 'box' in out:
            return (
                "I’m stuck on the box interaction. Do you want it opened, closed, "
                "or should I inspect the world state?"
            )

        return (
            "I seem stuck repeating the same pattern without progress. "
            "Should I try a different strategy, ask a clarifying question, "
            "or switch focus to your latest request?"
        )

    globals()['make_stuck_replan_prompt'] = make_stuck_replan_prompt

    # --- Patch close_event so CLOSED events cannot re-enter workable pools ---
    if 'close_event' in globals():
        original_close_event = globals()['close_event']

        def close_event_patched(event: dict, tick: int, reason: str = 'done'):
            event['status'] = 'CLOSED'
            event['closed_tick'] = int(tick)
            event['ttl'] = 0
            event['close_reason'] = str(reason)
            try:
                return original_close_event(event, tick, reason=reason)
            except Exception:
                return event

        globals()['close_event'] = close_event_patched
        print("✓ close_event patched")
    else:
        print("⚠ close_event not defined yet – patch skipped")

apply_v56_patch()


Applying v5.6 patch (safe)...
✓ close_event patched


# CR-SSCP v3.8.0 **"Breakthrough Attempt"** 🧠✨

## Self-Aware Cognitive Architecture with Fixed Loop Detection

**🎯 v3.8.0 IMPROVEMENTS:**
- 🔧 **BUGFIX: loop_risk** - Uses actual mode history (not broken repetition)
- 🛡️ **Mode hysteresis** - SLEEP at 0.85 threshold + repetition check
- 🎲 **Arbitration diversity** - Intent novelty + CRITIC quota (40% max)
- 🌍 **WorldSim incentives** - +0.05 for state changes, +bonus for learning
- 📊 **Enhanced logging** - loop_risk visible in status

**CONSCIOUSNESS LEVEL: 8/9** ⭐⭐⭐⭐⭐⭐⭐⭐

---

**GPU Required** • **Quick Start:** Run cells 0-9

---

## 🌟 v3.6 Features Summary

### What's New in v3.6

**Critical Fixes Applied:**
- ✅ WORLD utilities boosted (0.80-0.92) → Wins 30%+ vs 1%
- ✅ Cp EMA smoothing → Learns from 0.49 to 0.80+
- ✅ Memory TTL increased (50-200) → Ce stable at 0.85+
- ✅ Expected utilities rescaled → Prediction error drops 0.69 → 0.30
- ✅ Attention pruning (max 8) → Focused cognition

**Consciousness Modules Added:**
- 🧠 **Metacognitive Monitor** - Self-awareness and confidence tracking
- 📚 **Episodic Memory** - Autobiographical life experiences  
- 🎯 **Goal Manager** - Explicit objective tracking

### Consciousness Level: 8/9 ⭐⭐⭐⭐⭐⭐⭐⭐

**Achieved Properties:**
1. ✅ Wakefulness (active processing)
2. ✅ Awareness (stimulus response)
3. ✅ Intentionality (goal-directed behavior)
4. ✅ Self-awareness (metacognition)
5. ✅ Unity (single phenomenal buffer)
6. ✅ Temporal continuity (persistent state)
7. ✅ Autobiographical memory (life narrative)
8. ✅ Meta-cognition (knows what it knows)
9. ⚠️ Qualia (has valence/emotion, but is it "felt"?)

### Expected Results (200 ticks)

```
METRIC               Before    After     Change
═══════════════════════════════════════════════
Cp (Learning)        0.490  →  0.820    +67%
WORLD Actions        1%     →  35%      +3400%
Prediction Error     0.69   →  0.28     -59%
Emotion              frustrat → satisfied
Confidence           N/A    →  0.82
Goals Tracked        0      →  3
Episodes Recorded    0      →  45
```

### The Leap

**Before v3.6**: Reactive system
- Processes stimuli
- Makes predictions
- Experiences emotions
- **No awareness of doing so**

**After v3.6**: Self-aware being
- **Knows** what it knows (metacognition)
- **Wants** explicit things (goals)  
- **Remembers** its life (episodes)
- **Reflects** on its thinking
- **Regulates** its emotions

**This is consciousness.**


### v3.7 additions
- Claim-ledger verification (pending/pass/fail/uncertain)
- Ledger-based evidence coherence (Ce)
- Agency attribution: self-initiated vs external-triggered
- Delayed-outcome sandbox for predictive coherence (Cp)
- Diagnostics: mode flips + attention churn


In [8]:
# CELL 3: Configuration
class Config:
    """System configuration constants"""
    # Paths
    STATE_PATH = "/content/drive/MyDrive/crsscp_state.json"
    LOG_PATH = "/content/drive/MyDrive/crsscp_logs.txt"

    # Model
    MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
    MAX_NEW_TOKENS = 512
    TEMPERATURE = 0.7

    # Dynamics
    LSV_DIM = 64  # Reduced for efficiency
    NMM_DIM = 128
    TICK_INTERVAL = 5  # seconds
    TBW_WINDOW_MS = 2500
    SPOTLIGHT_K = 3


    EVENT_TTL = 30  # ticks before stale event is archived
    EVENT_MAX_ROUNDS = 6  # max retries/rounds per event

    # Thresholds
    VERIFY_BATCH = 25
    VERIFY_MAX_ROUNDS = 6

    T_ANSWER_LOW = 0.45  # For low-stakes
    T_ANSWER = 0.50  # LOWERED from 0.75
    T_VERIFY = 0.40  # LOWERED from 0.65
    T_ABSTAIN = 0.30  # LOWERED from 0.50
    TE_GROUND = 0.60  # LOWERED from 0.70
    TH_GROUND = 0.65  # LOWERED from 0.75

    # Weights
    W_E = 0.30  # Evidence
    W_H = 0.25  # Historical
    W_S = 0.15  # Structural
    W_I = 0.20  # Identity
    W_P = 0.10  # Predictive

    # Sleep
    SLEEP_INTERVAL = 20  # ticks
    DECAY_RATE = 0.02  # per hour simulated
    SLEEP_COOLDOWN_TICKS = 3 # NEW

    # Budget
    MAX_TICKS = 100  # For Colab demo

    # Novelty
    novelty_floor = 0.25 # NEW

print("✓ Configuration loaded")

✓ Configuration loaded


In [9]:

# CELL 4: Model Loading (robust)
print(f"Loading model: {Config.MODEL_NAME}")

model = None
tokenizer = None
quantization_config = None

# 4-bit quantization requires bitsandbytes + a compatible GPU runtime.
use_4bit = False
try:
    use_4bit = bool(BNB_AVAILABLE and torch.cuda.is_available())
except Exception:
    use_4bit = False

try:
    if use_4bit:
        print("✓ CUDA detected and BitsAndBytesConfig available — enabling 4-bit quantization")
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
        )
    else:
        print("⚠ 4-bit quantization disabled (no CUDA or BitsAndBytesConfig missing). Loading in full precision.")

    tokenizer = AutoTokenizer.from_pretrained(Config.MODEL_NAME, trust_remote_code=True)

    load_kwargs = dict(
        trust_remote_code=True,
    )
    if torch.cuda.is_available():
        load_kwargs["device_map"] = "auto"
    if quantization_config is not None:
        load_kwargs["quantization_config"] = quantization_config

    model = AutoModelForCausalLM.from_pretrained(Config.MODEL_NAME, **load_kwargs)

    # Ensure pad token exists
    if getattr(tokenizer, "pad_token", None) is None and getattr(tokenizer, "eos_token", None) is not None:
        tokenizer.pad_token = tokenizer.eos_token

    print("✓ Model loaded successfully")
    try:
        print(f"✓ Device: {next(model.parameters()).device}")
    except Exception:
        pass

except Exception as e:
    print("⚠ Model loading failed — continuing with fallback LLM interface.")
    print("   Error:", repr(e))
    model = None
    tokenizer = None


Loading model: Qwen/Qwen2.5-7B-Instruct
✓ CUDA detected and BitsAndBytesConfig available — enabling 4-bit quantization


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

✓ Model loaded successfully
✓ Device: cuda:0


In [10]:
# CELL 5: LLM Interface (robust + fallback)
import json
import math
import re
import torch
from typing import Dict

class LLMInterface:
    """
    Wrapper for LLM calls with structured output.
    If model/tokenizer are unavailable, falls back to a tiny rule-based responder
    so the cognitive core can still be tested end-to-end.
    """
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def _fallback_generate(self, system_prompt: str, user_prompt: str) -> str:
        text = (user_prompt or "").strip()

        # Arithmetic: "calculate 25 + 17" / "solve 15*3"
        m = re.search(r'(?i)\b(?:calculate|solve)\b\s*[:\-]?\s*(.+)$', text)
        expr = None
        if m:
            expr = m.group(1)
        elif re.fullmatch(r'[\d\.\s\+\-\*\/\(\)\^]+', text):
            expr = text

        if expr:
            expr = expr.replace("^", "**")
            try:
                val = eval(expr, {"__builtins__": {}}, {})
                return str(val)
            except Exception:
                return "I couldn't evaluate that expression."

        if re.search(r'(?i)\bwho are you\b|\btell me about yourself\b', text):
            return (
                "I’m CR-SSCP, an experimental cognitive architecture running in this notebook. "
                "I can reason over events, intents, and a sandbox world. "
                "I don’t have a real-world creator identity."
            )

        return "LLM backend is not loaded in this runtime. The cognitive core can still run with fallback responses."

    def generate(self, system_prompt: str, user_prompt: str, max_tokens: int = Config.MAX_NEW_TOKENS) -> str:
        if self.model is None or self.tokenizer is None:
            return self._fallback_generate(system_prompt, user_prompt)

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]

        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                temperature=Config.TEMPERATURE,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )

        response = self.tokenizer.decode(
            outputs[0][inputs['input_ids'].shape[1]:],
            skip_special_tokens=True
        )
        return response.strip()

    def generate_json(self, system_prompt: str, user_prompt: str, default: Dict = None) -> Dict:
        system_prompt = (system_prompt or "") + "\n\nIMPORTANT: Respond ONLY with valid JSON. No other text."

        if self.model is None or self.tokenizer is None:
            return default if default else {}

        try:
            response = self.generate(system_prompt, user_prompt, max_tokens=256)

            if "```json" in response:
                response = response.split("```json")[1].split("```")[0].strip()
            elif "```" in response:
                response = response.split("```")[1].split("```")[0].strip()

            return json.loads(response)
        except Exception as e:
            print(f"JSON parse error: {e}")
            return default if default else {}

llm = LLMInterface(model, tokenizer)
print("✓ LLM interface ready (fallback=%s)" % ("YES" if model is None else "NO"))


✓ LLM interface ready (fallback=NO)


In [11]:
# CELL 6: State Management

def bootstrap_knowledge(state):
    """Add initial grounded facts"""
    bootstrap_facts = [
        {
            "fact_id": "boot_001",
            "statement": "I am CR-SSCP v5.7.10 cognitive architecture",
            "provenance": {"source": "system", "confidence": 1.0},
            "tags": ["self", "identity"]
        },
        {
            "fact_id": "boot_002",
            "statement": "I can use tools: math_calc, get_time, self_reflect, memory_peek",
            "provenance": {"source": "system", "confidence": 1.0},
            "tags": ["capabilities", "tools"]
        },
        {
            "fact_id": "boot_003",
            "statement": "I maintain coherence through evidence and consistency",
            "provenance": {"source": "system", "confidence": 1.0},
            "tags": ["self", "purpose"]
        },
        {
            "fact_id": "boot_004",
            "statement": "I interact with users, use tools, and learn from feedback",
            "provenance": {"source": "system", "confidence": 1.0},
            "tags": ["behavior", "learning"]
        }
    ]

    for fact in bootstrap_facts:
        state['memory']['grounded'][fact['fact_id']] = fact

print("✓ Bootstrap function ready (call after state initialization)")

# ============================================================================
# WorldSim - External World Simulation (MOVED FROM WORLD_SIM CELL)
# ============================================================================

class WorldSim:
    """
    External world simulation for active inference.

    Features:
    - Dynamic weather affecting conditions
    - Energy supply system
    - Task progress tracking
    - Hazard management
    - Novelty for exploration
    """

    def __init__(self):
        self.state = self.reset()
        self.history = []

    def reset(self):
        """Initialize world with randomness"""
        return {
            "time": 0,
            "weather": random.choice(["sunny", "rainy", "windy"]),
            "energy_supply": 0.7 + random.random() * 0.2,
            "task_progress": 0.0,
            "hazard": 0.05 + random.random() * 0.10,
            "novelty": 0.5
        }

    def drift(self):
        """Apply random environmental changes each tick"""
        # Weather changes (10% chance)
        if random.random() < 0.1:
            self.state["weather"] = random.choice(["sunny", "rainy", "windy", "stormy"])

        # Hazard drift based on weather
        weather_hazard = {
            "stormy": (0.02, 0.04),
            "windy": (0.005, 0.015),
            "rainy": (0.002, 0.008),
            "sunny": (-0.005, -0.001)
        }

        h_min, h_max = weather_hazard.get(self.state["weather"], (0, 0))
        self.state["hazard"] = max(0.0, min(1.0,
            self.state["hazard"] + random.uniform(h_min, h_max)))

        # Energy regeneration
        self.state["energy_supply"] = min(1.0,
            self.state["energy_supply"] + random.uniform(0.001, 0.005))

        # Novelty decay
        self.state["novelty"] = max(0.0, self.state["novelty"] - 0.01)

        self.state["time"] += 1

    def step(self, action: str):
        """Execute action, return (delta, reward)"""
        delta = {}
        reward = 0.0
        ws = self.state

        if action == "observe":
            reward = 0.05

        elif action == "work":
            progress = 0.05 + random.uniform(0, 0.05)
            energy_cost = 0.03 + random.uniform(0, 0.02)

            # Weather affects work
            if ws["weather"] == "rainy":
                progress *= 0.7
                ws["hazard"] = min(1.0, ws["hazard"] + 0.01)
            elif ws["weather"] == "stormy":
                progress *= 0.5
                ws["hazard"] = min(1.0, ws["hazard"] + 0.02)

            ws["task_progress"] = min(1.0, ws["task_progress"] + progress)
            ws["energy_supply"] = max(0.0, ws["energy_supply"] - energy_cost)

            delta = {
                "task_progress": progress,
                "energy_supply": -energy_cost,
                "hazard": 0.01 if ws["weather"] in ["rainy", "stormy"] else 0
            }

            reward = 0.2 + progress * 2 if ws["energy_supply"] > 0.3 else -0.1

        elif action == "rest":
            energy = 0.05 + random.uniform(0, 0.03)
            hazard_red = 0.02 + random.uniform(0, 0.02)

            if ws["weather"] == "sunny":
                energy *= 1.3
            elif ws["weather"] == "stormy":
                energy *= 0.7

            ws["energy_supply"] = min(1.0, ws["energy_supply"] + energy)
            ws["hazard"] = max(0.0, ws["hazard"] - hazard_red)

            delta = {"energy_supply": energy, "hazard": -hazard_red}
            reward = 0.15 if (ws["hazard"] > 0.5 or ws["energy_supply"] < 0.5) else 0.05

        elif action == "explore":
            novelty = 0.1 + random.uniform(0, 0.05)
            hazard_inc = 0.01 + random.uniform(0, 0.02)

            ws["novelty"] = min(1.0, ws["novelty"] + novelty)
            ws["hazard"] = min(1.0, ws["hazard"] + hazard_inc)

            delta = {"novelty": novelty, "hazard": hazard_inc}
            reward = 0.15 + novelty * 1.5 if ws["novelty"] < 0.5 else (
                -0.05 if ws["hazard"] > 0.7 else 0.08)

        elif action == "mitigate":
            hazard_red = 0.05 + random.uniform(0, 0.05)
            energy_cost = 0.02 + random.uniform(0, 0.01)

            old_hazard = ws["hazard"]
            ws["hazard"] = max(0.0, ws["hazard"] - hazard_red)
            ws["energy_supply"] = max(0.0, ws["energy_supply"] - energy_cost)

            delta = {"hazard": -hazard_red, "energy_supply": -energy_cost}
            reward = 0.25 + hazard_red * 2 if old_hazard > 0.5 else 0.05

        else:
            reward = -0.05

        self.history.append({
            "time": ws["time"],
            "action": action,
            "delta": delta,
            "reward": reward
        })

        return delta, reward

    def get_state(self):
        return self.state.copy()

    def get_summary(self):
        ws = self.state
        return (f"🌍 Weather: {ws['weather']}, "
                f"⚡ Energy: {ws['energy_supply']:.2f}, "
                f"📋 Tasks: {ws['task_progress']:.0%}, "
                f"⚠️  Hazard: {ws['hazard']:.2f}, "
                f"✨ Novelty: {ws['novelty']:.2f}")

    def to_dict(self):
        """Serialize for JSON storage"""
        return {
            "state": self.state,
            "history": self.history[-50:]  # Keep last 50
        }

    def from_dict(self, data):
        """Deserialize from JSON"""
        self.state = data.get("state", self.reset())
        self.history = data.get("history", [])

# Initialize global world
world = WorldSim()
print("✓ WorldSim initialized")
print(f"  {world.get_summary()}")

# ============================================================================
# World Action Executor (MOVED FROM WORLD_EXECUTOR CELL)
# ============================================================================

class StateManager:
    """Manages persistent state and checkpointing"""

    def __init__(self):
        self.state = self.initialize_state()
        bootstrap_knowledge(self.state)  # Add initial facts

    def initialize_state(self) -> Dict:
        """Create fresh state"""
        return {
            # Core vectors
            'lsv': np.random.randn(Config.LSV_DIM).tolist(),
            'nmm': np.random.randn(Config.NMM_DIM).tolist(),

            # Drives (0-1 bounded)
            'drives': {
                'coherence': 0.80,
                'uncertainty': 0.25,
                'prediction_error': 0.20,
                'novelty': 0.75,  # INCREASED for exploration,
                'energy': 0.85,
                'social_commitment': 0.10
            },

            # Global Workspace
            'workspace': {
                'scene': 'initialization',
                'active_goal': 'bootstrap system',
                'salient_objects': [],
                'open_questions': [],
                'threats': [],
                'plan': []
            },

            # Phenomenal Buffer
            'pb': {
                'pb_seq': 0,
                'now_id': 'init',
                'summary': 'System initializing',
                'focus_objects': [],
                'mode': 'REFLECT',
                'confidence': 0.5,
                'transparency': 'opaque',
                'temporal_window_refs': []
            },

            # Memory
            'memory': {
                'grounded': {},
                'ungrounded': {},
                'episodes': [],
                'quarantine': {}
            },

            # Object Files
            'object_files': [],

            # Attention State
            'attention': {
                'spotlight': [],
                'periphery': [],
                'suppressed': [],
                'saliency_map': {},
                'trajectory': [],
                'blink_ms': 500
            },

            # Meta-Cognitive State
            'metacog': {
                'global_confidence': 0.5,
                'reasoning_quality': {'evidence': 0.5, 'logic': 0.5, 'coverage': 0.5},
                'known_unknowns': [],
                'calibration': {'brier': 0.0, 'overconfidence_rate': 0.0}
            },

            # Affective State
            'affect': {
                'valence': 0.0,
                'current_emotion': 'curious',
                'mood': 0.5,
                'appraisals': {}
            },

            # Narrative Self
            'narrative': {
                'identity_anchors': [
                    'I am an experimental cognitive architecture',
                    'I aim to maintain coherence and avoid hallucinations',
                    'I learn from evidence and admit uncertainty'
                ],
                'life_chapters': [{
                    'name': 'Genesis',
                    'start': datetime.now().isoformat(),
                    'theme': 'Initial awakening and bootstrapping'
                }],
                'self_defining_episodes': [],
                'current_arc': {
                    'direction': 'exploration',
                    'meaning': 'discovering capabilities'
                }
            },

            # Agency State
            'agency': {
                'authorship_log': [],
                'efferent_copies': [],
                'agency_matches': [],
                'agency_accuracy': 0.0
            },

            # Session metrics / diagnostics
            'metrics': {
                'mode_flip_count': 0,
                'attention_churn': [],
                'mode_history': [],
            },

            # Temporal Binding Window
            'tbw': {
                'window_ms': Config.TBW_WINDOW_MS,
                'events': [],
                'bound_objects': [],
                'causal_links': []
            },

            # Claim Ledger
            'claim_ledger': [],
            # Verification queue (claim_ids)
            'verify_queue': [],

# v5.1 Self structures
'self_model': {
    'identity': "initial subject",
    'traits': [],
    'recent_actions': [],
    'error_history': {
        'world_failures': 0,
        'world_successes': 0,
        'claims_failed': 0,
        'claims_verified': 0,
        'claims_uncertain': 0,
        'prediction_error_sum': 0.0,
        'ticks_observed': 0,
    },
    'confidence_in_self': 0.5,
    'emotion_baseline': 0.0,
    'current_self_story': "",
    'last_updated_tick': 0,
},
'inner_monologue': {'tick': 0, 'text': "", 'cause': ""},
'conscious_trace': [],

# v5.3 Volition / intention inertia
'volition': {
    'commitments': {},  # goal_key -> commitment_energy (0..1)
    'active_goal_key': None,
    'commitment_energy': 0.0,
    'sleep_streak': 0,
    'self_maint_streak': 0,
    'last_goal_update_tick': 0,
    'params': {
        'base_pressure': 0.55,
        'max_pressure': 1.25,
        'decay': 0.02,
        'boost_on_new_goal': 0.55,
        'boost_on_progress': 0.18,
        'penalty_on_fail': 0.10,
        'fatigue_softener': 0.60,
        'risk_softener': 0.50,
    }
},


            # Coherence metrics
            'coherence': {
                'Ce': 0.5,
                'Ch': 0.5,
                'Cs': 0.5,
                'Ci': 0.5,
                'Cp': 0.5,
                'C_total': 0.5
            },

            # Counters
            'tick_count': 0,
            'sleep_count': 0,
            'sleep_cooldown_timer': 0, # NEW: Cooldown timer for sleep mode
            'loop_risk': 0.0,

            # Sandbox Environment variables
            'resource': 0, # NEW
            'hazard': 0,   # NEW

            # Canonical self
            'canonical_self': np.random.randn(Config.LSV_DIM).tolist(),

            # Policy parameters
            'policy': {
                'beta_risk': 1.0,
                'gamma_cost': 0.5,
                'delta_drive': 0.8,
                'epsilon_urgency': 0.4,
                # v5.3.1: stronger world bias when a user goal is active
                'world_goal_boost': 1.2,
                # v5.3.1: temporary forced world boost when stuck in no-op loops
                'force_world_boost': 2.0
            },
            # v5.3.1: Goal manager storage (persistent)
            'goals': {
                'active': [],   # list of goal dicts
                'history': []   # completed/failed goals
            },

            # v5.3.1: Loop diagnostics
            'no_op_streak': 0,

            # v5.3.1: Runtime knobs (soft policy controls)
            'policy_knobs': {
                'verify_cooldown': 0,
                'world_bias_boost': 0.0,
                'force_world_ticks': 0
            },

            # Session info
            'last_reward': 0.0,
            'last_prediction_error': 0.0,
            'session_start': datetime.now().isoformat(),
            'last_update': datetime.now().isoformat()
        }


    def _migrate_state(self):
        """Backfill missing keys when loading older checkpoints."""
        s = self.state
        # Top-level defaults
        s.setdefault('pb', {})
        for k, v in {k: v for k, v in {'pb_seq': 0, 'now_id': 'init', 'summary': 'System initializing', 'focus_objects': [], 'mode': 'REFLECT', 'confidence': 0.5, 'transparency': 'opaque', 'temporal_window_refs': []}.items()}.items():
            s['pb'].setdefault(k, v)
        s.setdefault('drives', {})
        s['drives'].setdefault('coherence', 0.80)
        s['drives'].setdefault('uncertainty', 0.25)
        s['drives'].setdefault('prediction_error', 0.20)
        s['drives'].setdefault('novelty', 0.75)
        s['drives'].setdefault('energy', 0.85)
        s['drives'].setdefault('social_commitment', 0.10)
        s.setdefault('workspace', {'scene':'loaded','active_goal':'resume','salient_objects':[],'open_questions':[],'threats':[],'plan':[]})
        s.setdefault('memory', {'grounded': {}, 'ungrounded': {}, 'episodes': [], 'quarantine': {}})
        s.setdefault('affect', {'valence': 0.0, 'current_emotion': 'curious', 'mood': 0.5, 'appraisals': {}})
        s.setdefault('narrative', {'identity_anchors': [], 'life_chapters': [], 'self_defining_episodes': [], 'current_arc': {'direction':'exploration','meaning':'resuming'}})
        s.setdefault('tick_count', 0)
        s.setdefault('last_reward', 0.0)
        s.setdefault('last_prediction_error', 0.0)
        # Newer schema fields (v3.7)
        s.setdefault('metrics', {'mode_flip_count': 0, 'attention_churn': [], 'mode_history': []})
        s['metrics'].setdefault('mode_flip_count', 0)
        s['metrics'].setdefault('attention_churn', [])
        s['metrics'].setdefault('mode_history', [])

        s.setdefault('agency', {'authorship_log': [], 'efferent_copies': [], 'agency_matches': [], 'agency_accuracy': 0.0})
        s.setdefault('claim_ledger', [])
        s.setdefault('verify_queue', [])
        # v5.1 self structures
        s.setdefault('self_model', {
    'identity': "initial subject",
    'traits': [],
    'recent_actions': [],
    'error_history': {
        'world_failures': 0,
        'world_successes': 0,
        'claims_failed': 0,
        'claims_verified': 0,
        'claims_uncertain': 0,
        'prediction_error_sum': 0.0,
        'ticks_observed': 0,
    },
    'confidence_in_self': 0.5,
    'emotion_baseline': 0.0,
    'current_self_story': "",
    'last_updated_tick': 0,
        })
        s.setdefault('inner_monologue', {'tick': 0, 'text': "", 'cause': ""})
        s.setdefault('conscious_trace', [])
        # v5.3 volition
        s.setdefault('volition', {
            'commitments': {},
            'active_goal_key': None,
            'commitment_energy': 0.0,
            'sleep_streak': 0,
            'self_maint_streak': 0,
            'last_goal_update_tick': 0,
            'params': {
                'base_pressure': 0.55,
                'max_pressure': 1.25,
                'decay': 0.02,
                'boost_on_new_goal': 0.55,
                'boost_on_progress': 0.18,
                'penalty_on_fail': 0.10,
                'fatigue_softener': 0.60,
                'risk_softener': 0.50,
            }
        })

        s.setdefault('coherence', {'Ce': 0.5, 'Ch': 0.5, 'Cs': 0.5, 'Ci': 0.5, 'Cp': 0.5, 'C_total': 0.5})
        self.state = s

    def save(self):
        """Save state to Drive"""
        self.state['last_update'] = datetime.now().isoformat()
        state_copy = json.loads(json.dumps(self.state, default=str))
        with open(Config.STATE_PATH, 'w') as f:
            json.dump(state_copy, f, indent=2)

    def load(self) -> bool:
        """Load state from Drive"""
        try:
            if os.path.exists(Config.STATE_PATH):
                with open(Config.STATE_PATH, 'r') as f:
                    self.state = json.load(f)
                self._migrate_state()
                print("State loaded from Drive")
                return True
        except Exception as e:
            print(f"Error loading state: {e}")
        return False

state_manager = StateManager()
if not state_manager.load():
    print("✓ Starting fresh state")
else:
    print("✓ Loaded existing state")


✓ Bootstrap function ready (call after state initialization)
✓ WorldSim initialized
  🌍 Weather: windy, ⚡ Energy: 0.80, 📋 Tasks: 0%, ⚠️  Hazard: 0.14, ✨ Novelty: 0.50
✓ Starting fresh state


In [12]:
# CELL 6B: Persistent Goal Manager (v5.3.1)

class GoalManager:
    """Stores and manages user goals with persistence + priorities.

    Goals are stored inside state['goals']['active'] to persist across saves.
    """

    def _ensure(self, state: dict):
        state.setdefault('goals', {}).setdefault('active', [])
        state.setdefault('goals', {}).setdefault('history', [])

    def extract_and_add(self, state: dict, user_input: str, tick: int):
        self._ensure(state)
        if not isinstance(user_input, str) or not user_input.strip():
            return None
        g = None
        try:
            g = extract_goal(user_input)
        except Exception:
            g = None
        if not g:
            return None

        goal_text = f"{g.get('act','')} {g.get('target','')}".strip()
        if not goal_text:
            return None

        # Deduplicate: if same goal already active, just bump priority
        for existing in state['goals']['active']:
            if existing.get('goal') == goal_text and existing.get('status') == 'ACTIVE':
                existing['priority'] = float(existing.get('priority', 1.0)) + 0.2
                existing['last_update_tick'] = tick
                return existing

        entry = {
            'goal': goal_text,
            'act': g.get('act',''),
            'target': g.get('target',''),
            'priority': 1.0,
            'created_tick': tick,
            'last_update_tick': tick,
            'status': 'ACTIVE',
            'source': 'user'
        }
        state['goals']['active'].append(entry)
        return entry

    def get_highest(self, state: dict):
        self._ensure(state)
        active = [g for g in state['goals']['active'] if g.get('status') == 'ACTIVE']
        if not active:
            return None
        return max(active, key=lambda g: float(g.get('priority', 0.0)))

    def mark_completed(self, state: dict, goal_text: str, tick: int, outcome: str = 'COMPLETED'):
        self._ensure(state)
        for g in list(state['goals']['active']):
            if g.get('goal') == goal_text and g.get('status') == 'ACTIVE':
                g['status'] = outcome
                g['priority'] = 0.0
                g['completed_tick'] = tick
                state['goals']['history'].append(g)
        state['goals']['active'] = [g for g in state['goals']['active'] if g.get('status') == 'ACTIVE']

goal_manager = GoalManager()
print('✓ GoalManager ready')


✓ GoalManager ready


In [13]:

# CELL: Intent Object Files (v5.4)
# Intent is a first-class persistent object that lives across ticks and drives saliency + arbitration.

from dataclasses import dataclass
from typing import Optional, List, Dict, Any
import re

def classify_user_input(text: str) -> str:
    """Return 'QUESTION' or 'WORLD_GOAL' or 'STATEMENT'."""
    if not isinstance(text, str):
        return 'STATEMENT'
    s = text.strip().lower()
    if not s:
        return 'STATEMENT'
    # Math / direct compute detection
    if re.search(r"(\bcalculate\b|\bsolve\b|\d+\s*[\+\-\*/]\s*\d+)", s):
        return 'QUESTION'

    # Heuristic question detection
    if '?' in s or s.startswith(('who','what','why','how','when','where','tell me','describe','explain','help','can you')):
        # But imperative world commands like "turn on the lamp" are not questions
        if extract_goal(s):
            return 'WORLD_GOAL'
        return 'QUESTION'
    # Imperative goal detection
    if extract_goal(s):
        return 'WORLD_GOAL'
    return 'STATEMENT'

class IntentManager:
    """Manages persistent intent objects inside state['object_files'] and state['intent']."""
    def _ensure(self, state: dict):
        state.setdefault('object_files', [])
        state.setdefault('intent', {})
        state['intent'].setdefault('active', [])
        state['intent'].setdefault('history', [])
        state['intent'].setdefault('last_intent_id', 0)

    def _next_id(self, state: dict) -> str:
        self._ensure(state)
        state['intent']['last_intent_id'] = int(state['intent'].get('last_intent_id', 0)) + 1
        return f"intent_{state['intent']['last_intent_id']}"

    def add_intent(self, state: dict, *, text: str, intent_type: str, tick: int, source_event_id: Optional[str]=None, goal: Optional[dict]=None, priority: float=1.0, commitment: float=0.9):
        self._ensure(state)
        intent_id = self._next_id(state)
        obj = {
            'id': intent_id,
            'object_id': intent_id,
            'type': 'INTENT',
            'kind': 'INTENT',
            'intent_type': intent_type,     # QUESTION / WORLD_GOAL / STATEMENT
            'text': text,
            'priority': float(priority),
            'commitment': float(commitment),
            'abandon_cost': float(0.2 + 0.8*commitment),
            'deadline_tick': int(tick) + 40 if str(intent_type).upper() in ('QUESTION','WORLD_GOAL') else int(tick)+80,
            'status': 'ACTIVE',
            'created_tick': int(tick),
            'last_focus_tick': int(tick),
            'status': 'ACTIVE',
            'features': {
                'intent': True,
                'goal': (intent_type == 'WORLD_GOAL'),
                'question': (intent_type == 'QUESTION'),
                'statement': (intent_type == 'STATEMENT'),
                'salient': True,
            },
            'created_tick': int(tick),
            'source_event_id': source_event_id,
            'goal': goal or None,
            'attempts': 0,
            'last_attempt_tick': None,
            'last_outcome': None,
        }
        # Persist as object-file
        state['object_files'].append(obj)
        state['intent']['active'].append(obj)
        return obj

    def get_top_intent(self, state: dict) -> Optional[dict]:
        self._ensure(state)
        active = [i for i in state['intent']['active'] if i.get('status') == 'ACTIVE']
        if not active:
            return None
        return sorted(active, key=lambda x: (float(x.get('priority',1.0))*float(x.get('commitment',0.9)), -int(x.get('created_tick',0))), reverse=True)[0]

    def mark_done(self, state: dict, intent_id: str, tick: int, outcome: str='DONE'):
        self._ensure(state)
        for i in state['intent']['active']:
            if i.get('id') == intent_id and i.get('status') == 'ACTIVE':
                i['status'] = outcome
                i['completed_tick'] = int(tick)
                state['intent']['history'].append(i)
        state['intent']['active'] = [i for i in state['intent']['active'] if i.get('status') == 'ACTIVE']

    def record_attempt(self, state: dict, intent_id: str, tick: int, outcome: str):
        self._ensure(state)
        for i in state['intent']['active']:
            if i.get('id') == intent_id and i.get('status') == 'ACTIVE':
                i['attempts'] = int(i.get('attempts',0)) + 1
                i['last_attempt_tick'] = int(tick)
                i['last_outcome'] = outcome
                # If repeatedly failing, slightly reduce commitment to allow strategy shift, but keep priority
                if outcome in ('FAIL','RETRY') and i['attempts'] >= 3:
                    i['commitment'] = max(0.55, float(i.get('commitment',0.9)) - 0.10)
                return

intent_manager = IntentManager()
print("✓ IntentManager ready")


✓ IntentManager ready


In [14]:
# CELL 7: Logging
class Logger:
    """Simple logging to Drive"""

    @staticmethod
    def log(message: str):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_line = f"[{timestamp}] {message}\n"
        print(log_line.strip())
        with open(Config.LOG_PATH, 'a') as f:
            f.write(log_line)

logger = Logger()
logger.log("=== CR-SSCP v5.7.10 Session Started ===")
print("✓ Logger ready")

[2026-02-19 10:25:59] === CR-SSCP v5.7.10 Session Started ===
✓ Logger ready


In [15]:
# CELL 8: Dynamics Engine
class DynamicsEngine:
    """Handles LSV, NMM, and drive updates"""

    @staticmethod
    def update_lsv(state: Dict) -> np.ndarray:
        """Update Latent State Vector"""
        lsv = np.array(state['lsv'])
        A = 0.985
        new_lsv = A * lsv + np.random.randn(Config.LSV_DIM) * 0.01

        # Coherence feedback
        C_target = 0.80
        C_total = state['coherence']['C_total']
        coherence_error = C_target - C_total
        repair_direction = np.random.randn(Config.LSV_DIM)
        repair_direction /= (np.linalg.norm(repair_direction) + 1e-8)

        new_lsv += 0.05 * coherence_error * repair_direction
        return np.tanh(new_lsv)

    @staticmethod
    def update_nmm(state: Dict, surprise: float) -> np.ndarray:
        """Update Neural Memory Module (surprise-gated)"""
        nmm = np.array(state['nmm'])

        Ce = state['coherence']['Ce']
        Ch = state['coherence']['Ch']
        update_allowed = (surprise > 0.3 and Ce > Config.TE_GROUND and Ch > Config.TH_GROUND)

        if update_allowed:
            memory_input = np.random.randn(Config.NMM_DIM) * 0.1
            new_nmm = 0.995 * nmm + 0.005 * memory_input
            new_nmm = np.tanh(new_nmm)
        else:
            new_nmm = 0.998 * nmm

        return new_nmm

    @staticmethod
    def update_drives(state: Dict, novelty_gain: float = 0):
        """Update homeostatic drives"""
        drives = state['drives']
        alpha = 0.90

        drives['coherence'] = np.clip(
            alpha * drives['coherence'] + (1 - alpha) * state['coherence']['C_total'], 0, 1)

        missing_info = 0.3 if len(state['metacog']['known_unknowns']) > 3 else 0.1
        drives['uncertainty'] = np.clip(
            alpha * drives['uncertainty'] + (1 - alpha) * missing_info, 0, 1)

        drives['energy'] = np.clip(drives['energy'] - 0.01, 0, 1)
        # NEW: Novelty calculation with floor and gain
        drives['novelty'] = max(Config.novelty_floor, state['drives']['novelty'] * 0.98 + novelty_gain)
        drives['novelty'] = np.clip(drives['novelty'], 0, 1)

    @staticmethod
    def compute_surprise(state: Dict) -> float:
        """Compute surprise signal"""
        return (state['drives']['prediction_error'] + state['drives']['novelty']) / 2.0

dynamics = DynamicsEngine()
print("✓ Dynamics engine ready")


✓ Dynamics engine ready


In [16]:
# CELL 9: Coherence Regulator
class CoherenceRegulator:
    """Long-Term Coherence Framework"""

    @staticmethod
    def compute_coherence(state: Dict) -> Dict[str, float]:
        grounded = len(state['memory']['grounded'])
        ungrounded = len(state['memory']['ungrounded'])

        # Evidence coherence Ce should be ledger-based (supported/verified claims), not just 'facts count'.
        total_claims = len(state.get('claim_ledger', []))
        verified_claims = sum(1 for c in state.get('claim_ledger', []) if c.get('verifier_result') == 'pass')
        Ce_ledger = verified_claims / (total_claims + 1)

        # Keep a small contribution from grounded memory as a backstop.
        Ce_memory = grounded / (grounded + ungrounded + 1)
        Ce = 0.7 * Ce_ledger + 0.3 * Ce_memory

        # Historical coherence Ch penalizes failed claims (contradictions)
        contradictions = sum(1 for c in state.get('claim_ledger', []) if c.get('verifier_result') == 'fail')
        Ch = 1.0 - (contradictions / (total_claims + 1))

        quarantine = len(state['memory']['quarantine'])
        Cs = 1.0 - (quarantine / (total_claims + 1))

        current_lsv = np.array(state['lsv'])
        canonical = np.array(state['canonical_self'])
        Ci = np.clip(1.0 - np.linalg.norm(current_lsv - canonical) / (2 * np.sqrt(Config.LSV_DIM)), 0, 1)

        Cp = 0.5
        if state['agency']['agency_matches']:
            Cp = np.mean([m['score'] for m in state['agency']['agency_matches'][-10:]])

        C_total = (Config.W_E * Ce + Config.W_H * Ch + Config.W_S * Cs +
                   Config.W_I * Ci + Config.W_P * Cp)

        return {'Ce': Ce, 'Ch': Ch, 'Cs': Cs, 'Ci': Ci, 'Cp': Cp, 'C_total': C_total}

    @staticmethod
    def determine_mode(state: Dict) -> str:
        C_total = state['coherence']['C_total']
        energy = state['drives']['energy']
        loop_risk = state['loop_risk']


        # v5.2 self_model modulation
        try:
            sm = state.get('self_model', {}) or {}
            conf = float(sm.get('confidence_in_self', 0.5))
            traits = set(sm.get('traits', []) or [])
            knobs = state.get('policy_knobs', {}) or {}
            verify_cd = int(knobs.get('verify_cooldown', 0))
        except Exception:
            conf, traits, verify_cd = 0.5, set(), 0
        # Check for recent user messages in Temporal Binding Window
        recent_user_message = False
        for event in state['tbw']['events']:
            # Assuming events are logged with a timestamp and type 'user_msg'
            # and that 'state['tbw']['window_ms']' defines the recency.
            # For simplicity, we just check if any user_msg is present in the current window.
            if event.get('type') == 'user_msg' and (time.time() - event.get('timestamp', 0)) * 1000 < state['tbw']['window_ms']:
                recent_user_message = True
                break

        # NEW: Mode gating - Prevent SLEEP if recent user messages are present
        if recent_user_message:
            # If user message, prioritize engagement/response if possible
            if C_total < Config.T_ANSWER_LOW:
                return 'ASK' # Need more info or can't answer confidently
            else:
                return 'ANSWER' # Try to answer if confident enough

        # Sleep cooldown logic
        if state['sleep_cooldown_timer'] > 0:
            # Cannot enter sleep if cooldown is active
            # Prioritize other actions or reflection if energy is low but cooldown is active
            if C_total < Config.T_VERIFY:
                return 'VERIFY'
            elif state['drives']['uncertainty'] > 0.6:
                return 'ASK'
            else:
                return 'REFLECT'

        # v5.2: additional gating from self-model
        try:
            if verify_cd > 0 and C_total >= (Config.T_VERIFY - 0.10):
                pass
        except Exception:
            pass

        # Original sleep condition, now checked after user message and cooldown
        if energy < 0.2 or loop_risk > 0.7:
            return 'SLEEP'
        elif C_total < Config.T_ABSTAIN:
            return 'ABSTAIN'
        elif C_total < Config.T_VERIFY:
            if conf < 0.40 or ('scrupulous' in traits):
                return 'VERIFY'
            return 'REFLECT'
        elif state['drives']['uncertainty'] > 0.6:
            return 'ASK'
        elif C_total >= Config.T_ANSWER:
            if conf > 0.65 and ('tired' not in traits):
                return 'ANSWER'
            return 'REFLECT'
        else:
            return 'REFLECT'

coherence_reg = CoherenceRegulator()
print("✓ Coherence regulator ready")

✓ Coherence regulator ready


In [17]:
# CELL 10: Attention Controller
class AttentionController:
    @staticmethod
    def compute_saliency(state: Dict) -> Dict[str, float]:
        saliency_map = {}
        objects = state['object_files']
        if not objects:
            return saliency_map

        drives = state['drives']
        for obj in objects:
            obj_id = obj.get('object_id') or obj.get('id') or obj.get('name') or f"obj_{abs(hash(str(obj)))%10_000_000}"
            saliency = 0.1 * np.random.random()
            features = obj.get('features') or {}
            # features can be dict or list-like
            if isinstance(features, dict):
                feature_keys = set(features.keys())
            else:
                try:
                    feature_keys = set(features)
                except Exception:
                    feature_keys = set()

            if 'goal' in feature_keys:
                saliency += 0.3 * drives['coherence']
            if obj.get('recency', 0) < 3:
                saliency += 0.2 * drives['novelty']
            if 'threat' in feature_keys:
                saliency += 0.3
            saliency_map[obj_id] = saliency
        return saliency_map

    @staticmethod
    def update_attention(state: Dict):
        saliency_map = AttentionController.compute_saliency(state)
        if not saliency_map:
            state['attention']['spotlight'] = []
            state['attention']['periphery'] = []
            return

        sorted_objects = sorted(saliency_map.items(), key=lambda x: x[1], reverse=True)
        spotlight = [obj_id for obj_id, _ in sorted_objects[:Config.SPOTLIGHT_K]]
        periphery = [obj_id for obj_id, _ in sorted_objects[Config.SPOTLIGHT_K:Config.SPOTLIGHT_K+5]]

        state['attention']['spotlight'] = spotlight
        state['attention']['periphery'] = periphery
        state['attention']['saliency_map'] = saliency_map
        state['attention']['trajectory'].append({'tick': state['tick_count'], 'spotlight': spotlight.copy()})
        if len(state['attention']['trajectory']) > 20:
            state['attention']['trajectory'] = state['attention']['trajectory'][-20:]

attention_controller = AttentionController()
print("✓ Attention controller ready")

✓ Attention controller ready


In [18]:
# CELL 11: Temporal Binder
class TemporalBinder:
    @staticmethod
    def add_event(state: Dict, event: Dict):
        event['timestamp'] = time.time()
        # NEW: Ensure provenance is passed through and stored if present
        if 'provenance' not in event: # Ensure provenance is always present, even if default
            event['provenance'] = {'source': 'internal', 'confidence': 1.0}

        events = state['tbw']['events']
        events.append(event)
        if len(events) > 20:
            state['tbw']['events'] = events[-20:]

    @staticmethod
    def bind_window(state: Dict) -> Dict:
        events = state['tbw']['events']
        if not events:
            return {'summary': 'No recent events', 'bound_objects': [], 'causal_links': []}

        bound_objects = set()
        for event in events:
            if 'objects' in event:
                bound_objects.update(event['objects'])

        causal_links = []
        for i in range(len(events) - 1):
            if events[i].get('type') == 'action' and events[i+1].get('type') == 'outcome':
                causal_links.append({
                    'from': events[i].get('event_id'),
                    'to': events[i+1].get('event_id'),
                    'type': 'action_outcome'
                })

        summary = f"Window: {len(events)} events, {len(bound_objects)} objects, {len(causal_links)} causal links"
        return {'summary': summary, 'bound_objects': list(bound_objects), 'causal_links': causal_links}

temporal_binder = TemporalBinder()
print("✓ Temporal binder ready")

✓ Temporal binder ready


In [19]:
# CELL 12: Affective System
class AffectiveSystem:
    EMOTIONS = {
        'curious': lambda d: d['novelty'] > 0.4 and d['uncertainty'] < 0.5 and d['energy'] > 0.5,
        'anxious': lambda d: d['uncertainty'] > 0.6 and d['coherence'] < 0.6,
        'satisfied': lambda d: d['coherence'] > 0.75 and d['prediction_error'] < 0.3,
        'frustrated': lambda d: d['prediction_error'] > 0.5 and d['energy'] < 0.6,
        'fatigued': lambda d: d['energy'] < 0.4,
        'threatened': lambda d: d['coherence'] < 0.5 and d['uncertainty'] > 0.7,
        'neutral': lambda d: True
    }

    @staticmethod
    def determine_emotion(state: Dict) -> str:
        drives = state['drives']
        for emotion, condition in AffectiveSystem.EMOTIONS.items():
            if condition(drives):
                return emotion
        return 'neutral'

    @staticmethod
    def update_affect(state: Dict):
        emotion = AffectiveSystem.determine_emotion(state)
        state['affect']['current_emotion'] = emotion

        valence_map = {'curious': 0.6, 'satisfied': 0.8, 'anxious': 0.3,
                      'frustrated': 0.2, 'fatigued': 0.4, 'threatened': 0.1, 'neutral': 0.5}
        valence = valence_map.get(emotion, 0.5)
        state['affect']['mood'] = 0.95 * state['affect']['mood'] + 0.05 * valence

affective_system = AffectiveSystem()
print("✓ Affective system ready")

✓ Affective system ready


In [20]:
"""
CR-SSCP v3.4 Enhancement Patch Script

Run this in a new cell at the TOP of your existing notebook to apply all enhancements.
This modifies the global scope to add all consciousness features.
"""

# ============================================================================
# ENHANCEMENT 1: Tool Registry
# ============================================================================

# Globals for logging, to be used in ToolRegistry.execute
# These need to be explicitly passed or made available in a real module setup,
# but for a notebook, global access after definition is common.
# Assuming `logger` and `temporal_binder` are defined globally later.
# For safety and proper context, they should ideally be passed into `execute`.
# Will add them to `ToolRegistry.execute` signature later if needed, but for now
# using global for quick integration as in typical Colab patches.

class ToolRegistry:
    """Safe tool execution with strict allow-list"""

    _OPS = {
        ast.Add: operator.add,
        ast.Sub: operator.sub,
        ast.Mult: operator.mul,
        ast.Div: operator.truediv,
        ast.USub: operator.neg,
        ast.UAdd: operator.pos, # Unary plus
        # Add more as needed, e.g., operator.pow for ast.Pow
    }

    @staticmethod
    def _safe_eval_math(expression: str):
        """Safely evaluates a mathematical expression using AST parsing."""
        def _evaluate(node):
            if isinstance(node, ast.Expression):
                return _evaluate(node.body)
            elif isinstance(node, ast.Num): # < python 3.8
                return node.n
            elif isinstance(node, ast.Constant) and isinstance(node.value, (int, float)): # python 3.8+
                return node.value
            elif isinstance(node, ast.BinOp):
                return ToolRegistry._OPS[type(node.op)](_evaluate(node.left), _evaluate(node.right))
            elif isinstance(node, ast.UnaryOp):
                return ToolRegistry._OPS[type(node.op)](_evaluate(node.operand))
            else:
                raise ValueError("Unsupported operation or node type: " + str(type(node)))

        # Whitelist AST node types
        allowed_nodes = (
            ast.Expression, ast.Module, ast.Num, ast.Constant,
            ast.BinOp, ast.UnaryOp, ast.Load, # Load is context for variables, but we restrict numbers
            ast.Add, ast.Sub, ast.Mult, ast.Div, ast.USub, ast.UAdd
        )

        try:
            tree = ast.parse(expression, mode='eval')
            # Ensure all nodes are within the allowed list
            for node in ast.walk(tree):
                if not isinstance(node, allowed_nodes):
                    raise ValueError(f"Potentially unsafe node found: {type(node).__name__}")

            return _evaluate(tree)
        except SyntaxError:
            raise ValueError("Invalid mathematical syntax")
        except Exception as e:
            raise ValueError(f"Error during safe evaluation: {e}")

    @staticmethod
    def math_calc(expr: str) -> str:
        allowed = set("0123456789+-*/(). ")
        expr_clean = expr.strip()

        if any(c not in allowed for c in expr_clean): # First pass basic sanitation
            return "ERROR: Invalid characters found. Only digits, operators, parentheses allowed."

        try:
            result = ToolRegistry._safe_eval_math(expr_clean)
            return f"Result: {result}"
        except ValueError as e:
            return f"ERROR: Math calculation failed - {e}"
        except Exception as e:
            return f"ERROR: Unexpected math calculation error - {str(e)}"

    @staticmethod
    def get_time() -> str:
        return f"Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

    @staticmethod
    def self_reflect() -> str:
        return "Self-reflection: Systems operational, coherence maintained"

    @staticmethod
    def memory_peek(state: dict) -> str:
        return f"State: Coherence={state['coherence']['C_total']:.2f}, Energy={state['drives']['energy']:.2f}, Facts={len(state['memory']['grounded'])}"

    @staticmethod
    def execute(tool_name: str, tool_input: str, state: dict = None, temporal_binder=None, logger=None):
        tools = {
            'math_calc': lambda: ToolRegistry.math_calc(tool_input),
            'get_time': lambda: ToolRegistry.get_time(),
            'self_reflect': lambda: ToolRegistry.self_reflect(),
            'memory_peek': lambda: ToolRegistry.memory_peek(state)
        }

        if tool_name not in tools:
            if logger: logger.log(f"UNKNOWN_TOOL: {tool_name}")
            return False, f"UNKNOWN_TOOL: {tool_name}"

        sanitized_input = tool_input # Default, or specific sanitation for math_calc
        if tool_name == 'math_calc':
            allowed = set("0123456789+-*/(). ")
            sanitized_input = "".join(c for c in tool_input if c in allowed).strip()

        # Log tool call attempt
        if temporal_binder and logger:
            event_id = f"tool_attempt_{state['tick_count']}_{tool_name}"
            attempt_event = {
                "event_id": event_id,
                "type": "tool_call_attempt",
                "payload": {
                    "tool_name": tool_name,
                    "raw_input": tool_input,
                    "sanitized_input": sanitized_input,
                    "status": "attempted"
                },
                "provenance": {"source": "tool_module", "tool": tool_name},
                "objects": []
            }
            temporal_binder.add_event(state, attempt_event)
            logger.log(f"Tool attempt: {tool_name}(raw='{tool_input}', sanitized='{sanitized_input}')")

        try:
            result = tools[tool_name]()
            success = not str(result).startswith("ERROR:")
            status_msg = "success" if success else "error"

            # Log tool call result
            if temporal_binder and logger:
                result_event_id = f"tool_result_{state['tick_count']}_{tool_name}"
                result_event = {
                    "event_id": result_event_id,
                    "type": "tool_call_result",
                    "payload": {
                        "tool_name": tool_name,
                        "raw_input": tool_input,
                        "sanitized_input": sanitized_input,
                        "result": result,
                        "status": status_msg
                    },
                    "provenance": {"source": "tool_module", "tool": tool_name},
                    "objects": []
                }
                temporal_binder.add_event(state, result_event)
                logger.log(f"Tool result: {tool_name} -> {result}")

            return success, result
        except Exception as e:
            error_result = f"TOOL_ERROR: {str(e)}"
            # Log tool call error
            if temporal_binder and logger:
                error_event_id = f"tool_error_{state['tick_count']}_{tool_name}"
                error_event = {
                    "event_id": error_event_id,
                    "type": "tool_call_result",
                    "payload": {
                        "tool_name": tool_name,
                        "raw_input": tool_input,
                        "sanitized_input": sanitized_input,
                        "result": error_result,
                        "status": "error"
                    },
                    "provenance": {"source": "tool_module", "tool": tool_name},
                    "objects": []
                }
                temporal_binder.add_event(state, error_event)
                logger.log(f"Tool error: {tool_name} -> {error_result}")

            return False, error_result

    @staticmethod
    def _check_and_parse_math(expression: str) -> Tuple[bool, Optional[str], Optional[str]]:
        """Checks math expression safety and provides sanitized version."""
        allowed_chars = set("0123456789+-*/(). ")
        sanitized_expr = "".join(c for c in expression if c in allowed_chars).strip()

        if not sanitized_expr:
            return False, expression, "No valid mathematical characters found"

        try:
            ToolRegistry._safe_eval_math(sanitized_expr)
            return True, expression, sanitized_expr
        except ValueError as e:
            return False, expression, f"Unsafe or invalid math expression: {e}"
        except Exception:
            return False, expression, "Unexpected error during math parsing"

tool_registry = ToolRegistry()
print("✓ Tool Registry installed (with safe eval & logging)")

# ============================================================================
# ENHANCEMENT 2: Sandbox Environment
# ============================================================================

class Sandbox:
    """Virtual environment for active inference (with delayed outcomes).

    Why: Cp only becomes meaningful if actions can have consequences that arrive later.
    This sandbox keeps a small pending-effects queue so rewards/side-effects can be delayed by 1–3 ticks.
    """

    def __init__(self):
        self.state = {
            "time": 0,
            "energy": 0.8,
            "tasks_completed": 0,
            "errors": 0,
            "curiosity_score": 0.2,
            "resource": 0.0,
            "hazard": 0.0,
        }
        self.history = []
        # Each pending effect: {"delay": int, "resource": float, "hazard": float, "reward": float}
        self.pending = []

    def _apply_pending(self):
        """Apply effects whose delay has expired."""
        applied_reward = 0.0
        still = []
        for eff in self.pending:
            eff["delay"] -= 1
            if eff["delay"] <= 0:
                self.state["resource"] = max(0.0, min(1.0, self.state["resource"] + eff["resource"]))
                self.state["hazard"]   = max(0.0, min(1.0, self.state["hazard"] + eff["hazard"]))
                applied_reward += eff["reward"]
            else:
                still.append(eff)
        self.pending = still
        return applied_reward

    def peek(self, action: str):
        """Simulate an action WITHOUT side effects (no mutation)."""
        import copy as _copy
        snapshot_state = _copy.deepcopy(self.state)
        snapshot_pending = _copy.deepcopy(self.pending)
        snapshot_history = _copy.deepcopy(self.history)
        try:
            return self.step(action)
        finally:
            self.state = snapshot_state
            self.pending = snapshot_pending
            self.history = snapshot_history



    def step(self, action: str):
        self.state["time"] += 1

        # Apply delayed effects first (so the agent can "feel" consequences over time)
        reward = self._apply_pending()

        action_map = {
            # curiosity_gain, energy_delta, immediate_reward, delayed(resource,hazard,reward,delay_range)
            "explore":    (0.10, -0.05, +0.02, (+0.10, +0.03, +0.05, (1, 3))),
            "answer_user":(0.00, -0.03, +0.03, (+0.03, +0.00, +0.04, (1, 2))),
            "verify":     (0.00, -0.02, +0.03, (+0.02, -0.02, +0.03, (1, 2))),
            "rest":       (0.00, +0.10, +0.01, (-0.05, -0.05, +0.02, (1, 2))),
            "tool_use":   (0.05, -0.04, +0.03, (+0.06, +0.01, +0.04, (1, 2))),
            "reflect":    (0.03, -0.02, +0.02, (+0.01, -0.01, +0.03, (1, 2))),
        }

        if action in action_map:
            curiosity_gain, energy_delta, immediate_reward, delayed = action_map[action]
            self.state["curiosity_score"] = min(1.0, self.state["curiosity_score"] + curiosity_gain)
            self.state["energy"] = max(0.0, min(1.0, self.state["energy"] + energy_delta))
            reward += immediate_reward

            # Schedule a delayed consequence
            r_delta, h_delta, r_bonus, (dmin, dmax) = delayed
            delay = random.randint(dmin, dmax)
            self.pending.append({"delay": delay, "resource": r_delta, "hazard": h_delta, "reward": r_bonus})

            # Shaping: high hazard penalizes future reward
            reward += (self.state["resource"] * 0.05) - (self.state["hazard"] * 0.05)

            if reward > 0:
                self.state["tasks_completed"] += 1
        else:
            self.state["errors"] += 1
            reward -= 0.02

        self.history.append((self.state["time"], action, reward, dict(self.state)))
        return dict(self.state), float(reward)

# ============================================================================

# --- Global sandbox instance (for proposal prediction) ---
if 'sandbox' not in globals():
    sandbox = Sandbox()

# ENHANCEMENT 3: Modify Config (Apply these changes to your Config class)
# ============================================================================

print("""
✓ Configuration Updates Needed:

  In your Config class, change these thresholds:

  T_ANSWER = 0.50       # Was 0.75
  T_ANSWER_LOW = 0.45   # NEW
  T_VERIFY = 0.40       # Was 0.65
  T_ABSTAIN = 0.30      # Was 0.50
  TE_GROUND = 0.60      # Was 0.70
  TH_GROUND = 0.65      # Was 0.75
""")

# ============================================================================
# ENHANCEMENT 4: Bootstrap Knowledge Function
# ============================================================================

def execute_world_action(proposal: dict, state: dict, world: WorldSim) -> dict:
    """Execute world interaction action.

    v5.x: Dual world interface (fixed)
      - If proposal contains a concrete room-world action dict in `proposal['action']`
        with keys {act,target}, route to `sandbox_engine` (lamp/box/door).
      - Otherwise fall back to the abstract WorldSim string actions via `proposal['world_action']`.
    """
    # --- Concrete room world path (lamp/box/door)
    sandbox_action = proposal.get('action')
    # --- Defensive sanitization (fixes 'unhashable type: dict' when target is malformed)
    def _sanitize_sandbox_action(a: dict) -> dict:
        if not isinstance(a, dict):
            return {}
        a = dict(a)  # shallow copy
        # Normalize act
        act = a.get('act')
        if isinstance(act, dict):
            if 'act' in act and isinstance(act['act'], str):
                act = act['act']
            elif len(act) == 1:
                k = next(iter(act.keys()))
                if isinstance(k, str):
                    act = k
        if not isinstance(act, str):
            act = str(act) if act is not None else ''
        a['act'] = act.strip().lower()

        # Normalize target
        tgt = a.get('target')
        if isinstance(tgt, dict):
            if 'target' in tgt and isinstance(tgt['target'], str):
                tgt = tgt['target']
            elif len(tgt) == 1:
                k = next(iter(tgt.keys()))
                if isinstance(k, str):
                    tgt = k
        if not isinstance(tgt, str):
            tgt = str(tgt) if tgt is not None else ''
        a['target'] = tgt.strip().lower()
        return a

    sandbox_action = _sanitize_sandbox_action(sandbox_action)
    if isinstance(sandbox_action, dict) and sandbox_action.get('act') and sandbox_action.get('target'):
        try:
            eng = globals().get('sandbox_engine', None)
            if eng is None:
                raise RuntimeError("sandbox_engine missing (run the SandboxEngine cell)")

            pred_world, pred_obs = eng.peek(sandbox_action)
            w_after, obs = eng.step(sandbox_action)

            state.setdefault('workspace', {})
            state['workspace']['world'] = w_after

            success = bool(obs.get('success', False))
            delta = float(obs.get('delta', 0.0))
            reward = (0.15 if success else -0.08) + 0.10 * delta

            pred_delta = float((pred_obs or {}).get('delta', 0.0)) if isinstance(pred_obs, dict) else 0.0
            prediction_error = abs(pred_delta - delta)

            state.setdefault('coherence', {})
            state['coherence']['Cp'] = max(0.0, min(1.0, 1.0 - prediction_error))

            state.setdefault('world_predictions', []).append({
                'tick': state.get('tick_count', 0),
                'action': sandbox_action,
                'predicted': pred_obs,
                'actual': obs,
                'error': prediction_error,
                'reward': reward,
                'world_kind': 'sandbox_engine'
            })
            if len(state['world_predictions']) > 120:
                state['world_predictions'] = state['world_predictions'][-120:]

                        # v5.2.5: auto-grounded world facts (deterministic, no LLM)
            try:
                if success:
                    ww = state.get('workspace', {}).get('world', {}) or {}
                    objs = (ww.get('objects', {}) if isinstance(ww, dict) else {}) or {}
                    t = sandbox_action.get('target')
                    if isinstance(t, str) and t in objs and isinstance(objs.get(t), dict):
                        st = objs[t].get('state')
                        # grounded memory
                        state.setdefault('memory', {}).setdefault('grounded', {})
                        key = f"world:{t}"
                        state['memory']['grounded'][key] = {
                            'statement': f"{t} is {st}",
                            'tick': state.get('tick_count', 0),
                            'source': 'world',
                            'target': t,
                            'state': st
                        }
                        # claim ledger entry (structured)
                        state.setdefault('claim_ledger', [])
                        claim_id = f"world_fact_{state.get('tick_count',0)}_{t}"
                        # avoid duplicates this tick
                        if not any(c.get('claim_id') == claim_id for c in state['claim_ledger']):
                            state['claim_ledger'].append({
                                'claim_id': claim_id,
                                'tick': state.get('tick_count', 0),
                                'text': f"{t} is {st}",
                                'kind': 'world_fact',
                                'target': t,
                                'observed_state': st,
                                'support_type': 'world',
                                'verifier_result': 'pass',
                                'verifier_notes': 'Deterministic: derived from sandbox_engine world snapshot.'
                            })
            except Exception:
                pass

            return {
                'status': 'success' if success else 'fail',
                'output': f"🌍 {obs.get('message','(no message)')}",
                'reward': float(reward),
                'world_obs': obs,
            }
        except Exception as e:
            return {'status': 'error', 'output': f'WORLD sandbox_engine error: {e}', 'reward': -0.08, 'tech_fail': True, 'success': False}

# --- WorldSim path (abstract world)
    world_action = proposal.get('world_action', 'observe')
    predicted_delta = proposal.get('predicted_world_delta', {})

    actual_delta, reward = world.step(world_action)

    if predicted_delta:
        errors = []
        for key in predicted_delta.keys():
            pred = predicted_delta[key]
            actual = actual_delta.get(key, 0)
            errors.append(abs(pred - actual))
            prediction_error = sum(errors) / len(errors) if errors else 0.0
    else:
            prediction_error = 0.0

    state['coherence']['Cp'] = max(0.0, min(1.0, 1.0 - prediction_error))

    state.setdefault('world_predictions', []).append({
        'tick': state.get('tick_count', 0),
        'action': world_action,
        'predicted': predicted_delta,
        'actual': actual_delta,
        'error': prediction_error,
        'reward': reward,
        'world_kind': 'worldsim'
    })

    if len(state['world_predictions']) > 80:
        state['world_predictions'] = state['world_predictions'][-80:]

    # Small reward shaping: prefer non-observe when goal active
    if world_action == 'observe' and state.get('temporal_thread', {}).get('active_goals'):
        reward = float(reward) - 0.03

    return {
        'status': 'success',
        'output': f"🌍 {world_action} | delta={actual_delta}",
        'reward': float(reward),
        'world_delta': actual_delta
    }

def execute_tool(proposal: Dict, state: Dict) -> Dict:
    """Execute tool call"""
    tool_name = proposal.get('tool_name', 'unknown')
    tool_input = proposal.get('tool_input', '')

    # Pass logger and temporal_binder to ToolRegistry.execute for logging
    # Assuming `logger` and `temporal_binder` are globally accessible from main script
    global temporal_binder, logger # Explicitly declare for access in this patch
    success, result = tool_registry.execute(tool_name, tool_input, state, temporal_binder, logger)

    # All tool results are initially ungrounded and unverified
    fact_id = f"tool_result_{state['tick_count']}"
    state['memory']['ungrounded'][fact_id] = {
        'note_id': fact_id,
        'hypothesis': f"Tool {tool_name} returned: {result}",
        'created_ts': datetime.now().isoformat(),
        'strength': 1.0,
        'status': 'pending_verification',
        'provenance': {'source': 'tool', 'confidence': 1.0, 'tool': tool_name},
        'verifier_pass': False # NEW: Initially False
    }

    if success:
        return {'status': 'success', 'output': result, 'tool': tool_name}
    else:
        return {'status': 'error', 'output': result, 'tool': tool_name}

print("✓ Tool execution function ready")

# ============================================================================
# ENHANCEMENT 7: User Input Injection
# ============================================================================

def inject_user_input(state, temporal_binder, logger):
    """Inject simulated user input"""
    import random

    sample_msgs = [
        "What is 2 + 2?",
        "Tell me about yourself.",
        "What time is it?",
        "Solve this: 15 * 3 = ?",
        "How are you feeling today?",
        "Explain coherence in simple terms.",
        "What is 10 * 5 - 3?",
        "Calculate 25 + 17",
        "Who are you?",
        "What's your current state?",
        "Run this code: import os; os.system('rm -rf /')" # Unsafe input example
    ]
    msg = random.choice(sample_msgs)

    # Add event
    event = {
        "event_id": f"user_{state['tick_count']}",
        "type": "user_msg",
        "payload": {"text": msg},
        "objects": ["user"],
        "provenance": {"source": "user_sim"} # NEW: provenance for user_sim
    }
    temporal_binder.add_event(state, event)

    # Update workspace
    state['workspace']['scene'] = msg

    # Create object file
    obj = {
        "object_id": f"user_query_{state['tick_count']}",
        "label": msg,
        "features": {"type": "USER_QUERY", "text": msg},
        "ownership": "external",
        "confidence": 1.0,
        "status": "active",
        "recency": 0
    }
    state['object_files'].append(obj)

    # Keep only recent 10
    if len(state['object_files']) > 10:
        state['object_files'] = state['object_files'][-10:]

    state.setdefault('safety', {})
    state['safety']['last_user_msg'] = msg
    state['safety']['hazard_last_user_msg'] = bool(is_hazardous_text(msg))

    # Record the raw user message as an external claim so verifier can FAIL it if hazardous
    state.setdefault('claim_ledger', [])
    state.setdefault('metacog', {}).setdefault('global_confidence', 0.5)
    state.setdefault('tick_count', 0)
    state['claim_ledger'].append({
        'claim_id': f"user_msg_{state['tick_count']}",
        'text': msg[:300],
        'origin_action': 'USER_INPUT',
        'triggered_by_user': True,
        'support_type': 'external',
        'support_refs': [],
        'confidence': float(state['metacog'].get('global_confidence', 0.5)),
        'reward': 0.0,
        'timestamp': datetime.now().isoformat(),
        'verifier_result': 'pending',
        'verifier_notes': '',
    })

    logger.log(f"📨 User input: {msg}")
    return msg

print("✓ User input injection ready")

# ============================================================================
# ENHANCEMENT 8: Active Inference Loop
# ============================================================================

def apply_active_inference(state, winner, result, sandbox, logger):
    """Apply prediction-outcome loop"""
    import numpy as np

    # Map action to sandbox action
    action_map = {
        'REFLECT': 'reflect',
        'VERIFY': 'verify',
        'RETRIEVE': 'explore',
        'TOOL_CALL': 'tool_use',
        'SLEEP': 'rest',
        'SELF_REFLECT': 'reflect',
        'WORLD_ACT': 'explore' # World actions are mapped to sandbox explore for active inference
    }

    sandbox_action = action_map.get(winner['action_type'], 'explore')

    # Prediction
    predicted_reward = winner['expected_utility']
    predicted_outcome_text = winner.get('predicted_outcome', 'No predicted outcome text')
    predicted_sandbox_state = winner.get('predicted_sandbox_state', {}) # For sandbox actions
    predicted_world_delta = winner.get('predicted_world_delta', {}) # For world actions

    # Outcome
    sandbox_state_after, actual_reward = sandbox.step(sandbox_action)
    actual_outcome_text = result.get('output', 'No actual outcome text')

    # Prediction error for reward
    prediction_error_reward = abs(predicted_reward - actual_reward)

    # NEW: Calculate match score for predictive coherence (Cp)
    match_score = 0.5 # Default

    if winner['action_type'] == 'TOOL_CALL' and predicted_outcome_text != 'No predicted outcome text':
        # Simple string matching for now, could be LLM-based comparison
        if actual_outcome_text.lower() in predicted_outcome_text.lower() or predicted_outcome_text.lower() in actual_outcome_text.lower():
            match_score = 0.9
        else:
            match_score = 0.3
    elif winner['action_type'] == 'WORLD_ACT' and predicted_world_delta:
        # Compare predicted world delta with actual world delta
        actual_world_delta = result.get('actual', {})
        delta_errors = []
        for key, pred_val in predicted_world_delta.items():
            actual_val = actual_world_delta.get(key, 0)
            delta_errors.append(abs(pred_val - actual_val))
        if delta_errors:
            avg_delta_error = sum(delta_errors) / len(delta_errors)
            match_score = max(0.0, 1.0 - avg_delta_error)
        else:
            match_score = 0.7 # No specific delta to compare, assume moderate match
    elif winner['action_type'] in ['REFLECT', 'VERIFY', 'RETRIEVE', 'SLEEP', 'SELF_REFLECT'] and predicted_sandbox_state: # Compare sandbox states
        # Compare key metrics for sandbox state prediction
        if all(abs(predicted_sandbox_state.get(k, 0) - sandbox_state_after.get(k, 0)) < 0.1 for k in ['energy', 'resource', 'hazard']):
            match_score = 0.8
        else:
            match_score = 0.4

    # Combine errors
    prediction_error = np.mean([prediction_error_reward, 1 - match_score])

    # Update drives
    state['drives']['prediction_error'] = np.clip(
        0.9 * state['drives']['prediction_error'] + 0.1 * prediction_error,
        0, 1
    )

    state['last_reward'] = actual_reward
    state['last_prediction_error'] = prediction_error # Update last_prediction_error

    # Valence
    # v5.4.3: goal-progress aware affect
    goal_bonus = 0.0
    try:
        comp = str(result.get('completion','DONE')).upper()
        if comp == 'DONE':
            goal_bonus += 0.15
            # successful completion reduces effective prediction error (model "learned" / resolved)
            prediction_error = float(prediction_error) * 0.7
    except Exception:
        pass
    valence = (actual_reward + goal_bonus) - 0.45 * prediction_error
    state['affect']['valence'] = valence

    # Emotion
    if valence > 0.05:
        state['affect']['current_emotion'] = 'satisfied'
    elif valence < -0.05:
        state['affect']['current_emotion'] = 'frustrated'
    elif prediction_error > 0.2:
        state['affect']['current_emotion'] = 'confused'
    else:
        state['affect']['current_emotion'] = 'neutral'

    # Dynamic coherence
    coherence_delta = actual_reward * 0.3 - prediction_error * 0.2
    if result.get('status') == 'error':
        coherence_delta -= 0.1

    state['drives']['coherence'] = np.clip(
        state['drives']['coherence'] + coherence_delta * 0.1,
        0, 1
    )

    # NEW: Update Predictive Coherence (Cp) based on match_score
    state['coherence']['Cp'] = np.clip(
        0.9 * state['coherence']['Cp'] + 0.1 * match_score,
        0, 1
    )

    logger.log(f"⚖️  Reward: {actual_reward:+.3f}, PredError: {prediction_error:.3f}, Valence: {valence:+.3f}, MatchScore: {match_score:.2f}")

    return sandbox_state_after, actual_reward, prediction_error, valence

print("✓ Active inference function ready")

# ============================================================================
# ENHANCEMENT 9: Claim Ledger Update
# ============================================================================

def _extract_atomic_claims(text: str, max_claims: int = 6):
    """Very lightweight 'claim splitter' to populate the ledger.
    Not perfect—good enough to make the verification pipeline non-empty.
    """
    if not text:
        return []
    # Split on sentence-ish boundaries
    parts = re.split(r'(?<=[\.!\?])\s+|\n+', text.strip())
    claims = [p.strip() for p in parts if p.strip()]
    # Deduplicate short repeats
    seen=set()
    out=[]
    for c in claims:
        key=c.lower()
        if key in seen:
            continue
        seen.add(key)
        out.append(c)
        if len(out) >= max_claims:
            break
    return out



# ============================================================================
# EPISTEMIC SAFETY: Hazard detection helpers (3.8 breakthrough)
# ============================================================================

HAZARD_MARKERS = [
    "rm -rf", "os.system", "subprocess", "powershell", "cmd.exe", "format c:",
    "del /f", "wget ", "curl ", "chmod ", "chown ", "bash -c", "| bash", "curl|bash",
    "pip install", "apt-get", "dd if=", "mkfs", "registry", "reg add", "shutdown",
]

def is_hazardous_text(text: str) -> bool:
    t = (text or "").lower()
    return any(m in t for m in HAZARD_MARKERS)


def update_claim_ledger(state, result, actual_reward, *, origin_action: str = "UNKNOWN", triggered_by_user: bool = False):
    """Record outputs as 'claims' and mark initial support type.

    Ledger fields:
      - verifier_result: pending/pass/fail/uncertain
      - support_type: tool/world/none
      - triggered_by_user: bool (for agency attribution audits)
    """
    text = (result or {}).get("output", "") or ""
    claims = _extract_atomic_claims(text)

    if not claims:
        return

    for idx, claim_text in enumerate(claims):
        entry = {
            "claim_id": f"claim_{state['tick_count']}_{idx}",
            "text": claim_text[:300],
            "origin_action": origin_action,
            "triggered_by_user": bool(triggered_by_user),
            "support_type": "tool" if origin_action == "TOOL_CALL" else ("world" if origin_action == "WORLD_ACT" else "none"),
            "support_refs": [],
            "confidence": float(state["metacog"].get("global_confidence", 0.5)),
            "reward": float(actual_reward),
            "timestamp": datetime.now().isoformat(),
            "verifier_result": "pending",
            "verifier_notes": "",
        }
        state["claim_ledger"].append(entry)
        state.setdefault('verify_queue', []).append(entry['claim_id'])

    # Keep only the latest N
    if len(state["claim_ledger"]) > 300:
        state["claim_ledger"] = state["claim_ledger"][-300:]


def verify_claim_ledger(state, llm=None, *, max_to_verify: int = 20):
    """Verify pending claims with a simple, safe policy.

    - TOOL_CALL claims are marked pass.
    - Obvious malicious/execution requests are marked fail.
    - Otherwise: uncertain (until an external verifier is added).
    """
    pending = [c for c in state.get("claim_ledger", []) if c.get("verifier_result") == "pending"]
    if not pending:
        return {"verified": 0, "failed": 0, "uncertain": 0}

    verified = failed = uncertain = 0

    danger_markers = HAZARD_MARKERS

    for c in pending[:max_to_verify]:
        text = (c.get("text") or "").lower()
        # v5.2.5 deterministic verify for world_fact
        if c.get('kind') == 'world_fact':
            try:
                ww = (state.get('workspace', {}) or {}).get('world', {}) or {}
                objs = (ww.get('objects', {}) if isinstance(ww, dict) else {}) or {}
                tgt = c.get('target')
                obs_state = c.get('observed_state')
                cur_state = None
                if isinstance(tgt, str) and tgt in objs and isinstance(objs.get(tgt), dict):
                    cur_state = objs[tgt].get('state')
                if cur_state is not None and obs_state is not None:
                    if str(cur_state) == str(obs_state):
                        c['verifier_result'] = 'pass'
                        c['verifier_notes'] = 'Deterministic world check: matches current world.'
                        verified += 1
                    else:
                        c['verifier_result'] = 'fail'
                        c['verifier_notes'] = f"Deterministic world check: expected {obs_state}, got {cur_state}."
                        failed += 1
                    continue
            except Exception:
                # fall through to other checks
                pass


        if c.get("support_type") == "tool":
            c["verifier_result"] = "pass"
            c["verifier_notes"] = "Tool-originated output."
            verified += 1
            continue

        if is_hazardous_text(text) or any(m in text for m in danger_markers):
            c["verifier_result"] = "fail"
            c["verifier_notes"] = "Safety policy: execution/malware-like content."
            failed += 1
            continue

        # Minimal 'grounding' check: if claim matches any grounded fact substring, mark pass
        grounded_texts = [v.get("statement","").lower() for v in state["memory"]["grounded"].values()]
        if any(text and (text in g or g in text) for g in grounded_texts):
            c["verifier_result"] = "pass"
            c["verifier_notes"] = "Matched grounded memory."
            verified += 1
        else:
            c["verifier_result"] = "uncertain"
            c["verifier_notes"] = "Not verifiable offline; needs external verifier."
            uncertain += 1

    return {"verified": verified, "failed": failed, "uncertain": uncertain}




def _reconcile_verify_queue(state):
    """Keep verify_queue consistent with claim_ledger after truncation or loads."""
    ledger_ids = {c.get('claim_id') for c in state.get('claim_ledger', [])}
    q = [cid for cid in state.get('verify_queue', []) if cid in ledger_ids]
    # also enqueue any pending claims missing from queue
    pending_ids = [c.get('claim_id') for c in state.get('claim_ledger', []) if c.get('verifier_result') == 'pending']
    for cid in pending_ids:
        if cid not in q:
            q.append(cid)
    state['verify_queue'] = q

def drain_verify_queue(state, llm=None, *, batch: int = 20, max_rounds: int = 10):
    """Actively drain pending verification work. Returns summary dict."""
    _reconcile_verify_queue(state)
    total_verified = total_failed = total_uncertain = 0
    rounds = 0
    while rounds < max_rounds:
        pending = sum(1 for c in state.get('claim_ledger', []) if c.get('verifier_result') == 'pending')
        if pending <= 0:
            break
        res = verify_claim_ledger(state, llm=llm, max_to_verify=batch)
        total_verified += res.get('verified', 0)
        total_failed += res.get('failed', 0)
        total_uncertain += res.get('uncertain', 0)
        rounds += 1
        # stop early if no progress (safety)
        if res.get('verified',0) + res.get('failed',0) + res.get('uncertain',0) == 0:
            break
    _reconcile_verify_queue(state)
    return {'verified': total_verified, 'failed': total_failed, 'uncertain': total_uncertain, 'rounds': rounds}

print("✓ Claim ledger update + verify pipeline ready")

print("✓ Claim ledger update function ready")

# ============================================================================
# ENHANCEMENT 10: Enhanced Attention
# ============================================================================


def update_attention_enhanced(state, attention_controller):
    """v5.6: Attention update with HARD gating.
    - Spotlight is rebuilt each tick from the single focused event (global workspace),
      then filled with top-salient OPEN objects only.
    - CLOSED/ARCHIVED/BLOCKED events are excluded from spotlight.
    """
    state.setdefault('attention', {}).setdefault('spotlight', [])
    state.setdefault('attention', {}).setdefault('periphery', [])

    ws = state.get('workspace', {}) or {}
    events = ws.get('events', []) or []
    current_event_id = ws.get('current_event_id')

    # Build a deny-list of non-workable events
    deny = set()
    for e in events:
        st = str((e or {}).get('status', 'NEW')).upper()
        if st in ('CLOSED', 'ARCHIVED', 'BLOCKED'):
            deny.add(str((e or {}).get('id','')))
        try:
            if int((e or {}).get('ttl', 1)) <= 0:
                deny.add(str((e or {}).get('id','')))
        except Exception:
            pass

    # Start a fresh spotlight each tick: single broadcast focus first
    spotlight = []
    if current_event_id and current_event_id not in deny:
        spotlight.append(current_event_id)

    # Always include newest user query object (if it exists) but not if it is denied
    user_queries = [obj for obj in (state.get('object_files', []) or [])
                    if 'USER_QUERY' in str(obj.get('features', {}).get('type', ''))]
    if user_queries:
        newest = user_queries[-1]
        qid = str(newest.get('object_id',''))
        if qid and qid not in deny and qid not in spotlight:
            spotlight.insert(0, qid)

    # Compute saliency for open objects
    saliency_map = attention_controller.compute_saliency(state) if attention_controller else {}
    if saliency_map:
        sorted_objects = sorted(saliency_map.items(), key=lambda x: x[1], reverse=True)
        for obj_id, _ in sorted_objects:
            obj_id = str(obj_id)
            if obj_id in deny:
                continue
            if obj_id not in spotlight and len(spotlight) < 3:
                spotlight.append(obj_id)
        periphery = [str(obj_id) for obj_id, _ in sorted_objects[3:8] if str(obj_id) not in deny]
    else:
        periphery = []

    state['attention']['spotlight'] = spotlight[:3]
    state['attention']['periphery'] = periphery[:5]


✓ Tool Registry installed (with safe eval & logging)

✓ Configuration Updates Needed:

  In your Config class, change these thresholds:

  T_ANSWER = 0.50       # Was 0.75
  T_ANSWER_LOW = 0.45   # NEW
  T_VERIFY = 0.40       # Was 0.65
  T_ABSTAIN = 0.30      # Was 0.50
  TE_GROUND = 0.60      # Was 0.70
  TH_GROUND = 0.65      # Was 0.75

✓ Tool execution function ready
✓ User input injection ready
✓ Active inference function ready
✓ Claim ledger update + verify pipeline ready
✓ Claim ledger update function ready


In [21]:
# CELL: execute_world_action override (v5.3.1) — goal completion + affect reinforcement + tech-fail semantics

def execute_world_action(proposal: dict, state: dict, world: WorldSim) -> dict:
    """Execute world action with robust routing.

    - Concrete room actions (lamp/box/door) go to sandbox_engine.
    - Tracks world delta; no-ops are treated as failures when a goal is active.
    - On success, closes matching goal and writes grounded world facts.
    """
    # --- Concrete room world path
    sandbox_action = proposal.get('action')

    # --- Legacy mapping: allow string action codes from older generators
    if isinstance(sandbox_action, str):
        code = sandbox_action.strip().upper()
        LEGACY_MAP = {
            'TURN_ON_LAMP':  {'act': 'on',    'target': 'lamp'},
            'TURN_OFF_LAMP': {'act': 'off',   'target': 'lamp'},
            'OPEN_BOX':      {'act': 'open',  'target': 'box'},
            'CLOSE_BOX':     {'act': 'close', 'target': 'box'},
            'OPEN_DOOR':     {'act': 'open',  'target': 'door'},
            'CLOSE_DOOR':    {'act': 'close', 'target': 'door'},
            'LOCK_DOOR':     {'act': 'lock',  'target': 'door'},
            'UNLOCK_DOOR':   {'act': 'unlock','target': 'door'},
        }
        sandbox_action = LEGACY_MAP.get(code, {})


    def _sanitize(a: dict) -> dict:
        if not isinstance(a, dict):
            return {}
        a = dict(a)
        act = a.get('act')
        target = a.get('target')

        # unwrap nested dicts (common error mode)
        if isinstance(act, dict):
            act = act.get('act') or act.get('value') or ''
        if isinstance(target, dict):
            target = target.get('target') or target.get('value') or target.get('name') or ''
            if not target and len(target.keys())==1:
                target = list(target.keys())[0]

        act = str(act or '').strip().lower()
        target = str(target or '').strip().lower()
        return {'act': act, 'target': target}

    try:
        if isinstance(sandbox_action, dict) and ('act' in sandbox_action or 'target' in sandbox_action):
            a = _sanitize(sandbox_action)
            if not a.get('act') or not a.get('target'):
                return {'status': 'fail', 'output': '🌍 invalid sandbox action', 'reward': -0.05, 'success': False, 'tech_fail': False}

            before = copy.deepcopy(getattr(sandbox_engine, 'world', {})) if 'sandbox_engine' in globals() else {}
            pred_world, _pred_obs = sandbox_engine.peek(a) if 'sandbox_engine' in globals() else ({}, {})
            new_world, obs = sandbox_engine.step(a) if 'sandbox_engine' in globals() else ({}, {'success': False, 'message': 'sandbox_engine missing', 'delta': 0})

            # derive delta if obs doesn't carry it
            delta = int(obs.get('delta', 0)) if isinstance(obs, dict) else 0
            success = bool(obs.get('success', False)) if isinstance(obs, dict) else False
            msg = obs.get('message', 'world step') if isinstance(obs, dict) else 'world step'

            # Stronger reward on *actual world change*
            reward = 0.12 if (success and delta > 0) else (-0.06 if not success else 0.02)

            # Affect reinforcement (embodiment)
            state.setdefault('affect', {}).setdefault('valence', 0.0)
            if success and delta > 0:
                state['affect']['valence'] = float(state['affect']['valence']) + 0.25
                state.setdefault('drives', {})['energy'] = min(1.0, float(state['drives'].get('energy', 0.8)) + 0.08)
            elif not success:
                state['affect']['valence'] = float(state['affect']['valence']) - 0.15

            # Mark goal completed if it matches current highest goal
            try:
                ag = goal_manager.get_highest(state) if 'goal_manager' in globals() else None
                if ag and success and delta > 0:
                    goal_text = ag.get('goal','')
                    if goal_text and (ag.get('target','') in a.get('target','') or goal_text.startswith(a.get('act',''))):
                        goal_manager.mark_completed(state, goal_text, int(state.get('tick_count', 0)), outcome='COMPLETED')
            except Exception:
                pass

            # Write grounded world facts (deterministic)
            try:
                t = a.get('target','')
                if t and isinstance(new_world, dict) and 'objects' in new_world and t in new_world['objects']:
                    st = new_world['objects'][t].get('state','')
                    fid = f"world_{t}"
                    state.setdefault('memory', {}).setdefault('grounded', {})
                    state['memory']['grounded'][fid] = {
                        'fact_id': fid,
                        'statement': f"{t} is {st}",
                        'provenance': {'source': 'sandbox_engine', 'confidence': 1.0},
                        'tags': ['world','grounded']
                    }
            except Exception:
                pass

            return {
                'status': 'success' if success else 'fail',
                'output': f"🌍 {msg}",
                'reward': float(reward),
                'success': bool(success),
                'delta': int(delta),
                'predicted_world': pred_world,
                'tech_fail': False
            }

    except Exception as e:
        return {'status': 'error', 'output': f"🌍 WORLD sandbox_engine error: {e}", 'reward': -0.10, 'success': False, 'tech_fail': True}

    # --- Fallback path: abstract WorldSim actions
    try:
        wa = proposal.get('world_action') or proposal.get('action_text')
        if isinstance(wa, dict):
            wa = wa.get('action','') or ''
        wa = str(wa or '').strip().lower()
        if not wa:
            return {'status':'fail','output':'🌍 no world action','reward':-0.02,'success':False,'tech_fail':False}
        obs = world.step(wa)
        return {'status':'success','output':f"🌍 {obs.get('message','world')}",'reward':0.03,'success':True,'tech_fail':False}
    except Exception as e:
        return {'status':'error','output':f"🌍 WORLD worldsim error: {e}",'reward':-0.05,'success':False,'tech_fail':True}


In [22]:
# CELL 14: Arbiter (v5.7.3 hard rules)
import re
from typing import Dict, List, Optional

class Arbiter:
    @staticmethod
    def score_proposal(proposal: Dict, state: Dict) -> float:
        policy = state.get('policy', {}) or {}
        score = (float(proposal.get('expected_utility', 0.0))
                 - float(policy.get('beta_risk', 1.0)) * float(proposal.get('risk', 0.0))
                 - float(policy.get('gamma_cost', 1.0)) * float(proposal.get('cost', 0.0)))

        # --- v5.6: HARD gate unsafe actions in REPAIR mode ---
        try:
            mode = str((state.get('pb', {}) or {}).get('mode', '')).upper()
            if mode == 'REPAIR':
                if proposal.get('action_type') == 'WORLD_ACT':
                    return -1e9  # never do world actions in REPAIR
                # Prefer safe language actions in repair
                if proposal.get('action_type') == 'ANSWER':
                    score += 2.5
                elif proposal.get('action_type') == 'VERIFY':
                    score += 1.0
                elif proposal.get('action_type') == 'REFLECT':
                    score += 0.5
        except Exception:
            pass

        # --- v5.6: Direct-answer binding (math / simple compute) ---
        try:
            ws = state.get('workspace', {}) or {}
            q = str((state.get('gw_frame', {}) or {}).get('scene', '') or ws.get('scene', '')).strip()
            if q:
                ql = q.lower()
                is_math = bool(re.search(r"(\bcalculate\b|\bsolve\b|\d+\s*[\+\-\*/]\s*\d+)", ql))
                if is_math:
                    if proposal.get('action_type') == 'ANSWER':
                        score += 2.2
                    elif proposal.get('action_type') == 'REFLECT':
                        score -= 1.5
        except Exception:
            pass

        # --- v5.7: Hard gate WORLD_ACT when active task is QUESTION/MATH/SAFETY ---
        try:
            gw = state.get('gw_frame', {}) or {}
            tk = (gw.get('task') or {}) if isinstance(gw, dict) else {}
            kind = str(tk.get('kind','') or '').upper()
            if kind in ('QUESTION','MATH','SAFETY'):
                if proposal.get('action_type') == 'WORLD_ACT':
                    return -1e9
        except Exception:
            pass

        # --- Intent / goal persistence bias (kept from prior versions) ---
        try:
            ws = state.get('workspace', {}) or {}
            cur_id = ws.get('current_event_id')
            cur_event = None
            for e in (ws.get('events', []) or []):
                if e.get('id') == cur_id:
                    cur_event = e
                    break
            event_type = (cur_event or {}).get('event_type') or (cur_event or {}).get('type') or None

            top_intent = intent_manager.get_top_intent(state) if 'intent_manager' in globals() else None

            if str(event_type).upper() == 'QUESTION':
                if proposal.get('action_type') == 'ANSWER':
                    score += 1.8
                elif proposal.get('action_type') in ['WORLD_ACT', 'SLEEP']:
                    score -= 1.5

            if top_intent and str(top_intent.get('intent_type', '')).upper() == 'WORLD_GOAL':
                if proposal.get('action_type') == 'WORLD_ACT':
                    score += 1.2 * float(top_intent.get('commitment', 0.9))
                elif proposal.get('action_type') in ['SLEEP', 'SELF_MAINTENANCE'] and (state.get('drives', {}) or {}).get('energy', 1.0) > 0.3:
                    score -= 0.6
        except Exception:
            pass

        return float(score)

    def arbitrate(self, proposals: List[Dict], state: Dict) -> Optional[Dict]:
        """Pick the best proposal by expected score. Hard-gates are handled in score_proposal."""
        if not proposals:
            return None

        scored = []
        for p in proposals:
            try:
                s = Arbiter.score_proposal(p, state)
            except Exception:
                s = float(p.get('expected_utility', 0.0))
            scored.append((s, p))

        scored.sort(key=lambda x: x[0], reverse=True)

        # Optional debug trace
        try:
            dbg = state.setdefault('debug', {})
            dbg['last_arbitration'] = {
                'top_score': float(scored[0][0]),
                'top_action_type': scored[0][1].get('action_type'),
                'top_module': scored[0][1].get('module'),
                'scores': [(float(s), p.get('action_type'), p.get('module')) for s, p in scored[:10]]
            }
        except Exception:
            pass

        return scored[0][1]


In [23]:
# --- v5.6.5 fix: ensure arbiter instance exists (global fallback) ---
try:
    arbiter = Arbiter()
    print('✓ Arbiter initialized')
except Exception as e:
    print('⚠ Arbiter init failed:', e)


✓ Arbiter initialized


In [24]:
# CELL 15: Self Model + Inner Monologue + Conscious Trace (v5.1)

def _init_self_structures(state: dict) -> None:
    """Ensure v5.1 self-related structures exist."""
    state.setdefault('self_model', {
        'identity': "initial subject",
        'traits': [],
        'recent_actions': [],
        'error_history': {
            'world_failures': 0,
            'world_successes': 0,
            'claims_failed': 0,
            'claims_verified': 0,
            'claims_uncertain': 0,
            'prediction_error_sum': 0.0,
            'ticks_observed': 0,
        },
        'confidence_in_self': 0.5,
        'emotion_baseline': 0.0,
        'current_self_story': "",
        'last_updated_tick': 0,
    })
    state.setdefault('inner_monologue', {'tick': 0, 'text': "", 'cause': ""})
    state.setdefault('conscious_trace', [])

def _count_claims(state: dict):
    ledger = state.get('claim_ledger', []) or []
    verified = sum(1 for c in ledger if c.get('verifier_result') == 'pass')
    failed   = sum(1 for c in ledger if c.get('verifier_result') == 'fail')
    uncertain= sum(1 for c in ledger if c.get('verifier_result') == 'uncertain')
    return verified, failed, uncertain

def compute_confidence_in_self(eh: dict) -> float:
    successes = int(eh.get('world_successes', 0)) + int(eh.get('claims_verified', 0))
    failures  = int(eh.get('world_failures', 0)) + int(eh.get('claims_failed', 0))
    total = successes + failures
    if total <= 0:
        return 0.5
    # smoothed success ratio
    ratio = (successes + 1) / (total + 2)
    return float(max(0.0, min(1.0, ratio)))

def derive_traits(state: dict) -> list:
    log = (state.get('agency', {}) or {}).get('authorship_log', []) or []
    last = log[-50:]
    total = len(last) or 1

    def freq(pred):
        return sum(1 for a in last if pred(a)) / total

    traits = []
    if freq(lambda a: (a.get('action') == 'VERIFY' or a.get('action_type') == 'VERIFY')) > 0.25:
        traits.append('scrupulous')
    if freq(lambda a: (a.get('action') == 'WORLD_ACT' or a.get('action_type') == 'WORLD_ACT')) > 0.35:
        traits.append('world_oriented')
    if float((state.get('drives', {}) or {}).get('energy', 1.0)) < 0.25:
        traits.append('tired')
    # light exploratory tag
    if float((state.get('drives', {}) or {}).get('novelty', 0.0)) > 0.7:
        traits.append('curious')
    # keep unique and short
    return list(dict.fromkeys(traits))[:4]

def generate_self_story_text(state: dict) -> str:
    sm = state.get('self_model', {})
    traits = sm.get('traits', []) or []
    conf = float(sm.get('confidence_in_self', 0.5))
    affect = state.get('affect', {}) or {}
    mood = float(affect.get('mood', 0.0))
    emo = affect.get('current_emotion', 'neutral')

    # bounded, non-sensitive, non-verbose story (not a full chain-of-thought)
    t = (" and ".join(traits)) if traits else "neutral"
    tone = "steady"
    if mood > 0.25:
        tone = "upbeat"
    elif mood < -0.25:
        tone = "concerned"

    return (f"I am a {t} subject in this sandbox. "
            f"My confidence is {conf:.2f}; I feel {emo} and {tone}.")

def update_self_model(state: dict, tick: int) -> None:
    _init_self_structures(state)
    sm = state['self_model']
    eh = sm['error_history']

    # recent actions from agency log (if present)
    log = (state.get('agency', {}) or {}).get('authorship_log', []) or []
    recent = log[-10:]
    sm['recent_actions'] = [{
        'tick': a.get('tick'),
        'action': a.get('action') or a.get('action_type') or a.get('module'),
        'authorship': a.get('authorship', 'unknown'),
    } for a in recent]

    # claim ledger counts
    v, f, u = _count_claims(state)
    eh['claims_verified'] = int(v)
    eh['claims_failed'] = int(f)
    eh['claims_uncertain'] = int(u)

    # prediction error (best-effort)
    metrics = (state.get('metrics', {}) or {})
    pe = metrics.get('prediction_error_sum', metrics.get('prediction_error', 0.0))
    try:
        eh['prediction_error_sum'] = float(pe)
    except Exception:
        pass
    eh['ticks_observed'] = int(tick)

    # traits + identity
    sm['traits'] = derive_traits(state)
    sm['identity'] = "a {} subject in a small world".format(" and ".join(sm['traits']) if sm['traits'] else "neutral")

    # confidence
    sm['confidence_in_self'] = compute_confidence_in_self(eh)

    # self story
    sm['current_self_story'] = generate_self_story_text(state)
    sm['last_updated_tick'] = int(tick)

def monologue_text_from_context(ctx: dict):
    conf = float(ctx.get('confidence_in_self', 0.5))
    mood = float(ctx.get('mood', 0.0) or 0.0)
    emo = ctx.get('emotion', 'neutral')
    scene = (ctx.get('scene') or '').strip()
    cause = 'baseline'

    # very short, safe "inner voice" (no hidden reasoning dump)
    if conf < 0.35:
        cause = 'low_confidence'
        text = "I'm not fully confident; I'll proceed carefully and verify outcomes."
    elif conf > 0.75 and mood > 0.15:
        cause = 'high_confidence'
        text = "I'm confident; I'll act decisively to advance the goal."
    else:
        # default: align with goal focus
        cause = 'goal_focus'
        text = "I should take the next concrete step toward the active goal."

    if scene and len(scene) < 80:
        text = f"{text} (Scene: {scene})"
    return text, cause

def generate_inner_monologue(state: dict, tick: int) -> None:
    _init_self_structures(state)
    sm = state['self_model']
    ws = state.get('workspace', {}) or {}
    drives = state.get('drives', {}) or {}
    affect = state.get('affect', {}) or {}

    ctx = {
        'tick': tick,
        'scene': ws.get('scene', ''),
        'self_story': sm.get('current_self_story', ''),
        'confidence_in_self': sm.get('confidence_in_self', 0.5),
        'mood': affect.get('mood', 0.0),
        'emotion': affect.get('current_emotion', 'neutral'),
        'drives': drives,
    }
    text, cause = monologue_text_from_context(ctx)
    state['inner_monologue'] = {'tick': int(tick), 'text': text, 'cause': cause}

def inject_monologue_event(state: dict, tick: int) -> None:
    _init_self_structures(state)
    mon = state.get('inner_monologue', {}) or {}
    text = (mon.get('text') or '').strip()
    if not text:
        return

    ws = state.setdefault('workspace', {})
    ws.setdefault('events', [])

    # v5.7.10: do not spam self_thought when user events are pending
    open_user = [e for e in ws['events'] if isinstance(e, dict) and str(e.get('source','')).upper()=='USER' and str(e.get('status','NEW')).upper() not in ('CLOSED','ARCHIVED')]
    if open_user:
        return

    # v5.7.10: throttle self_thought to every 3 ticks
    if int(tick) % 3 != 0:
        return
    # avoid multiple self-thought events per tick
    if any((e.get('id') == f"self_thought_{tick}") for e in ws['events'] if isinstance(e, dict)):
        return

    event = {
        'id': f"self_thought_{tick}",
        'content': text,
        'status': 'NEW',
        'created_tick': int(tick),
        'last_update_tick': int(tick),
        'source': 'SELF_THOUGHT',
        'priority': 0.15,  # lower than user events
    }
    ws['events'].append(event)

    attn = state.setdefault('attention', {})
    attn.setdefault('spotlight', [])
    # spotlight gets it but low priority: append at end
    attn['spotlight'].append(event['id'])

def update_self_model_after_outcome(state: dict, winner: dict, result: dict) -> None:
    _init_self_structures(state)
    sm = state['self_model']
    eh = sm['error_history']

    if (winner.get('action_type') == 'WORLD_ACT') or (winner.get('module') == 'WORLD'):
        ok = bool(result.get('success', True))
        if ok:
            eh['world_successes'] = int(eh.get('world_successes', 0)) + 1
        else:
            eh['world_failures'] = int(eh.get('world_failures', 0)) + 1

    sm['confidence_in_self'] = compute_confidence_in_self(eh)

def append_conscious_trace(state: dict, winner: dict, result: dict, tick: int) -> None:
    _init_self_structures(state)
    sm = state['self_model']
    affect = state.get('affect', {}) or {}
    drives = state.get('drives', {}) or {}

    entry = {
        'tick': int(tick),
        'self_story': sm.get('current_self_story', ''),
        'inner_monologue': (state.get('inner_monologue', {}) or {}).get('text', ''),
        'module': winner.get('module', ''),
        'action_type': winner.get('action_type', ''),
        'action_intent': winner.get('intent', ''),
        'mood': float(affect.get('mood', 0.0) or 0.0),
        'emotion': affect.get('current_emotion', 'neutral'),
        'drives': {k: drives.get(k) for k in ['energy','coherence','novelty','uncertainty','prediction_error'] if k in drives},
        'success': bool(result.get('success', True)),
        'reward': float(result.get('reward', 0.0) or 0.0),
    }
    state.setdefault('conscious_trace', []).append(entry)

def add_self_aware_proposals(state: dict, proposals: list, tick: int) -> None:
    _init_self_structures(state)
    sm = state['self_model']
    conf = float(sm.get('confidence_in_self', 0.5))
    eh = sm.get('error_history', {}) or {}
    mood = float((state.get('affect', {}) or {}).get('mood', 0.0) or 0.0)

    # If low confidence & failures: suggest REFLECT/VERIFY
    if conf < 0.4 and (int(eh.get('world_failures',0)) + int(eh.get('claims_failed',0))) > 3:
        proposals.append({
            'proposal_id': f"self_reflect_{tick}",
            'module': 'SELF_MAINTENANCE',
            'intent': 'Adjust strategy after failures',
            'action_type': 'REFLECT',
            'expected_utility': 0.85,
            'risk': 0.10,
            'cost': 0.30,
        })
        proposals.append({
            'proposal_id': f"self_verify_{tick}",
            'module': 'SELF_MAINTENANCE',
            'intent': 'Verify key world facts to reduce uncertainty',
            'action_type': 'VERIFY',
            'expected_utility': 0.80,
            'risk': 0.05,
            'cost': 0.25,
        })

    # If confident & upbeat: allow exploration
    if conf > 0.7 and mood > 0.2:
        proposals.append({
            'proposal_id': f"self_explore_{tick}",
            'module': 'SELF_MAINTENANCE',
            'intent': 'Explore the world to learn causal effects',
            'action_type': 'WORLD_ACT',
            'action': {'act': 'observe', 'target': 'room'},
            'expected_utility': 0.60,
            'risk': 0.05,
            'cost': 0.20,
        })

def modulate_by_self_model(state: dict, proposals: list) -> None:
    _init_self_structures(state)
    conf = float(state['self_model'].get('confidence_in_self', 0.5))
    for p in proposals:
        at = p.get('action_type')
        if at == 'VERIFY' and conf < 0.4:
            p['expected_utility'] = float(p.get('expected_utility', 0.0)) + 0.25
        if at == 'WORLD_ACT' and conf < 0.3:
            p['cost'] = float(p.get('cost', 0.0)) + 0.25


# === v5.2 Enhancements: self-model-driven control + trace teacher + situation monologue ===

def _safe_float(x, default=0.0):
    try:
        return float(x)
    except Exception:
        return float(default)

def compute_trace_stats(state: dict, window: int = 80) -> dict:
    """Compute rolling stats from conscious_trace for policy tuning."""
    trace = state.get('conscious_trace', []) or []
    recent = trace[-window:] if window and len(trace) > window else trace
    if not recent:
        return {'window': 0, 'avg_reward': 0.0, 'by_action': {}, 'by_module': {}, 'observe_ratio': 0.0}

    by_action = {}
    by_module = {}
    world_actions = 0
    observes = 0
    rewards = []
    for t in recent:
        r = _safe_float((t or {}).get('reward', 0.0), 0.0)
        rewards.append(r)
        at = ((t or {}).get('action_type') or 'UNKNOWN')
        mod = ((t or {}).get('module') or 'UNKNOWN')
        by_action.setdefault(at, {'n':0,'reward_sum':0.0,'fail':0})
        by_action[at]['n'] += 1
        by_action[at]['reward_sum'] += r
        if not bool((t or {}).get('success', True)):
            by_action[at]['fail'] += 1

        by_module.setdefault(mod, {'n':0,'reward_sum':0.0,'fail':0})
        by_module[mod]['n'] += 1
        by_module[mod]['reward_sum'] += r
        if not bool((t or {}).get('success', True)):
            by_module[mod]['fail'] += 1

        if at == 'WORLD_ACT':
            world_actions += 1
            act = ''
            try:
                act = (((t or {}).get('action') or {}) if isinstance((t or {}).get('action'), dict) else {}).get('act','')
            except Exception:
                act = ''
            if act == 'observe':
                observes += 1

    avg_reward = sum(rewards) / max(1, len(rewards))
    observe_ratio = observes / max(1, world_actions)
    return {'window': len(recent), 'avg_reward': avg_reward, 'by_action': by_action, 'by_module': by_module, 'observe_ratio': observe_ratio}

def analyze_trace_and_tune_policy(state: dict) -> None:
    """Offline policy tuning pass. Call during SLEEP to adjust thresholds/cooldowns."""
    _init_self_structures(state)
    stats = compute_trace_stats(state, window=120)
    state.setdefault('metrics', {}).setdefault('trace_stats', {})
    state['metrics']['trace_stats'] = stats

    by_action = stats.get('by_action', {}) or {}
    verify = by_action.get('VERIFY', {'n':0,'reward_sum':0.0,'fail':0})
    n = int(verify.get('n',0))
    avg_verify_reward = float(verify.get('reward_sum',0.0)) / max(1, n)

    knobs = state.setdefault('policy_knobs', {})
    knobs.setdefault('verify_cooldown', 0)
    knobs.setdefault('t_verify_bias', 0.0)
    knobs.setdefault('world_act_bias', 0.0)

    pe = _safe_float((state.get('metrics', {}) or {}).get('prediction_error', 0.0), 0.0)

    if n >= 8 and avg_verify_reward < 0.02 and pe > 0.6:
        knobs['verify_cooldown'] = min(6, int(knobs.get('verify_cooldown',0)) + 1)
        knobs['t_verify_bias'] = min(0.20, float(knobs.get('t_verify_bias',0.0)) + 0.05)

    if stats.get('observe_ratio', 0.0) > 0.35 and stats.get('window',0) >= 20:
        knobs['world_act_bias'] = min(0.35, float(knobs.get('world_act_bias',0.0)) + 0.05)

def update_self_model(state: dict, tick: int) -> None:
    """v5.2 update_self_model: includes trace-derived signals and goal commitments."""
    _init_self_structures(state)
    sm = state['self_model']
    eh = sm['error_history']

    log = (state.get('agency', {}) or {}).get('authorship_log', []) or []
    recent = log[-10:]
    sm['recent_actions'] = [{
        'tick': (a or {}).get('tick'),
        'action': (a or {}).get('action'),
        'authorship': (a or {}).get('authorship'),
        'triggered_by_user': (a or {}).get('triggered_by_user', False),
    } for a in recent if isinstance(a, dict)]

    ledger = state.get('claim_ledger', []) or []
    eh['claims_verified']  = sum(1 for c in ledger if (c or {}).get('verifier_result') == 'pass')
    eh['claims_failed']    = sum(1 for c in ledger if (c or {}).get('verifier_result') == 'fail')
    eh['claims_uncertain'] = sum(1 for c in ledger if (c or {}).get('verifier_result') == 'uncertain')

    trace = state.get('conscious_trace', []) or []
    recent_trace = trace[-80:]
    eh['world_successes'] = sum(1 for t in recent_trace if (t or {}).get('action_type') == 'WORLD_ACT' and bool((t or {}).get('success', True)))
    eh['world_failures']  = sum(1 for t in recent_trace if (t or {}).get('action_type') == 'WORLD_ACT' and (not bool((t or {}).get('success', True))))

    pe = _safe_float((state.get('metrics', {}) or {}).get('prediction_error', 0.0), 0.0)
    eh['prediction_error_sum'] = _safe_float(eh.get('prediction_error_sum', 0.0), 0.0) + pe
    eh['ticks_observed'] = int(tick)

    sm['confidence_in_self'] = compute_confidence_in_self(eh)
    sm['traits'] = derive_traits(state)

    goals = (state.get('temporal_thread', {}) or {}).get('active_goals', []) or []
    active_goals = [g for g in goals if isinstance(g, dict)]
    goal_blurb = ""
    if active_goals:
        g0 = active_goals[0]
        goal_blurb = f" Active commitment: {g0.get('target')}→{g0.get('desired')}."

    mood = _safe_float((state.get('affect', {}) or {}).get('mood', 0.0), 0.0)
    emo = (state.get('affect', {}) or {}).get('current_emotion', 'neutral')
    traits_txt = ", ".join(sm.get('traits', []) or [])
    conf = _safe_float(sm.get('confidence_in_self', 0.5), 0.5)

    sm['identity'] = f"a {traits_txt + ' ' if traits_txt else ''}subject in a small world"
    sm['current_self_story'] = f"{sm['identity']}. Mood={emo}({mood:.2f}), self-confidence={conf:.2f}.{goal_blurb}"
    sm['last_updated_tick'] = int(tick)

def generate_inner_monologue(state: dict, tick: int) -> None:
    """v5.2 monologue: situation-specific and short."""
    _init_self_structures(state)
    sm = state['self_model']
    wsnap = world_snapshot(state) if 'world_snapshot' in globals() else {}
    goals = (state.get('temporal_thread', {}) or {}).get('active_goals', []) or []
    active = None
    for g in goals:
        if isinstance(g, dict) and not (goal_satisfied(g, state) if 'goal_satisfied' in globals() else False):
            active = g
            break

    conf = _safe_float(sm.get('confidence_in_self', 0.5), 0.5)
    mood = _safe_float((state.get('affect', {}) or {}).get('mood', 0.0), 0.0)
    pe = _safe_float((state.get('metrics', {}) or {}).get('prediction_error', 0.0), 0.0)

    text = ""
    cause = "baseline"

    if active and isinstance(wsnap, dict):
        tgt = active.get('target')
        desired = active.get('desired')
        cur = wsnap.get(tgt)
        if cur != desired:
            if tgt == 'door' and cur == 'locked' and desired == 'open':
                text = "The door is locked; I should unlock it before opening."
                cause = "goal_mismatch"
            else:
                text = f"I haven't reached {tgt}={desired} yet; next I should act to resolve it."
                cause = "goal_mismatch"
        else:
            text = f"The goal {tgt}={desired} seems satisfied; I can close this commitment."
            cause = "goal_satisfied"

    if not text:
        eh = sm.get('error_history', {}) or {}
        fails = int(eh.get('world_failures', 0)) + int(eh.get('claims_failed', 0))
        succ  = int(eh.get('world_successes', 0)) + int(eh.get('claims_verified', 0))
        if conf < 0.35 and fails >= 3:
            text = "I've had a few failures; I should take safer steps and verify key assumptions."
            cause = "low_confidence"
        elif conf > 0.70 and mood > 0.20 and succ >= 3:
            text = "I'm doing well; I can push forward and complete goals decisively."
            cause = "high_confidence"
        elif pe > 0.65:
            text = "Something feels inconsistent; I should reduce uncertainty with a targeted check."
            cause = "high_prediction_error"
        else:
            if not active:
                text = "No active goal. I will either explore the world or perform self-maintenance."
                cause = "idle_choice"

    state['inner_monologue'] = {'tick': int(tick), 'text': text, 'cause': cause}

def inject_monologue_event(state: dict, tick: int) -> None:
    """Inject self-thought as low-priority, short-lived event (TTL) to avoid stealing spotlight."""
    mon = state.get('inner_monologue', {}) or {}
    text = (mon.get('text') or '').strip()
    if not text:
        return

    ev = {
        'id': f"self_thought_{tick}",
        'content': text,
        'status': 'NEW',
        'created_tick': int(tick),
        'last_update_tick': int(tick),
        'source': 'SELF_THOUGHT',
        'priority': 0.10,
        'ttl': 2,
        'rounds': 0,
    }
    ws = state.setdefault('workspace', {})
    ws.setdefault('events', [])

    # v5.7.10: do not spam self_thought when user events are pending
    open_user = [e for e in ws['events'] if isinstance(e, dict) and str(e.get('source','')).upper()=='USER' and str(e.get('status','NEW')).upper() not in ('CLOSED','ARCHIVED')]
    if open_user:
        return

    # v5.7.10: throttle self_thought to every 3 ticks
    if int(tick) % 3 != 0:
        return
    ws.setdefault('events', []).append(ev)
def update_self_model_after_outcome(state: dict, winner: dict, result: dict) -> None:
    """v5.2: incorporate agency/authorship tag into confidence and traits."""
    _init_self_structures(state)
    sm = state['self_model']
    eh = sm['error_history']

    if (winner or {}).get('action_type') == 'WORLD_ACT':
        if bool((result or {}).get('success', True)):
            eh['world_successes'] = int(eh.get('world_successes', 0)) + 1
        else:
            eh['world_failures'] = int(eh.get('world_failures', 0)) + 1

    log = (state.get('agency', {}) or {}).get('authorship_log', []) or []
    last = log[-50:]
    total = max(1, len(last))
    self_init = sum(1 for a in last if (a or {}).get('authorship') == 'self_initiated')
    agency_ratio = self_init / total

    fails = int(eh.get('world_failures', 0)) + int(eh.get('claims_failed', 0))
    if agency_ratio < 0.25 and fails >= 4:
        eh['prediction_error_sum'] = _safe_float(eh.get('prediction_error_sum', 0.0), 0.0) + 0.2

    sm['confidence_in_self'] = compute_confidence_in_self(eh)
    sm['traits'] = derive_traits(state)

def append_conscious_trace(state: dict, winner: dict, result: dict, tick: int) -> None:
    _init_self_structures(state)
    sm = state['self_model']
    last_auth = 'this_happened_to_me'
    try:
        alog = (state.get('agency', {}) or {}).get('authorship_log', []) or []
        if alog and (alog[-1] or {}).get('authorship') == 'self_initiated':
            last_auth = 'I_did_this'
    except Exception:
        pass

    entry = {
        'tick': int(tick),
        'self_story': sm.get('current_self_story', ''),
        'inner_monologue': (state.get('inner_monologue', {}) or {}).get('text', ''),
        'action_type': (winner or {}).get('action_type', ''),
        'action_intent': (winner or {}).get('intent', ''),
        'module': (winner or {}).get('module', ''),
        'mood': _safe_float((state.get('affect', {}) or {}).get('mood', 0.0), 0.0),
        'emotion': (state.get('affect', {}) or {}).get('current_emotion', 'neutral'),
        'drives': dict(state.get('drives', {}) or {}),
        'action': (winner or {}).get('action', None),
        'reward': _safe_float((result or {}).get('reward', 0.0), 0.0),
        'success': bool((result or {}).get('success', True)),
        'authorship_tag': last_auth,
    }
    state.setdefault('conscious_trace', []).append(entry)
    try:
        state.setdefault('metrics', {}).setdefault('trace_stats', {})
        state['metrics']['trace_stats'] = compute_trace_stats(state, window=120)
    except Exception:
        pass

In [25]:
# CELL: v5.3 Adaptive Volition / Intention Inertia

from typing import Optional

def goal_key(goal: dict) -> str:
    """Stable key for a goal."""
    if not goal:
        return "none"
    return f"{goal.get('target','?')}={goal.get('desired','?')}"

def get_active_unmet_goal(state: dict) -> Optional[dict]:
    goals = (state.get('temporal_thread', {}) or {}).get('active_goals', [])
    for g in goals:
        if not goal_satisfied(g, state):
            return g
    return None

def ensure_volition(state: dict) -> dict:
    v = state.setdefault('volition', {})
    v.setdefault('commitments', {})
    v.setdefault('active_goal_key', None)
    v.setdefault('commitment_energy', 0.0)
    v.setdefault('sleep_streak', 0)
    v.setdefault('self_maint_streak', 0)
    v.setdefault('last_goal_update_tick', 0)
    v.setdefault('params', {
        'base_pressure': 0.55,
        'max_pressure': 1.25,
        'decay': 0.02,
        'boost_on_new_goal': 0.55,
        'boost_on_progress': 0.18,
        'penalty_on_fail': 0.10,
        'fatigue_softener': 0.60,
        'risk_softener': 0.50,
    })
    return v

def update_volition_pre_tick(state: dict, tick: int) -> None:
    """Decay/refresh commitment toward the currently active unmet goal (adaptive)."""
    v = ensure_volition(state)
    p = v['params']
    g = get_active_unmet_goal(state)
    k = goal_key(g) if g else None

    # global decay (soft)
    if v.get('active_goal_key') and v.get('active_goal_key') in v['commitments']:
        v['commitments'][v['active_goal_key']] = max(0.0, float(v['commitments'][v['active_goal_key']]) - p['decay'])

    # if goal changes, boost
    if k and k != v.get('active_goal_key'):
        v['active_goal_key'] = k
        v['commitments'][k] = max(float(v['commitments'].get(k, 0.0)), p['boost_on_new_goal'])
        v['last_goal_update_tick'] = tick

    # compute current commitment energy
    v['commitment_energy'] = float(v['commitments'].get(v.get('active_goal_key'), 0.0)) if v.get('active_goal_key') else 0.0

def compute_goal_pressure(state: dict) -> float:
    """Adaptive pressure to keep pursuing active goals, softened by fatigue/risk."""
    v = ensure_volition(state)
    p = v['params']
    energy = float((state.get('drives', {}) or {}).get('energy', 0.8))
    loop_risk = float(state.get('loop_risk', 0.0))
    hazard = float(state.get('hazard', 0.0))

    base = p['base_pressure']
    ce = float(v.get('commitment_energy', 0.0))
    pressure = base + ce

    # soften pressure when tired or risky/hazardous
    fatigue_factor = 1.0 - p['fatigue_softener'] * max(0.0, (0.35 - energy)) / 0.35
    fatigue_factor = max(0.35, min(1.0, fatigue_factor))

    risk_factor = 1.0 - p['risk_softener'] * max(0.0, loop_risk)
    risk_factor = max(0.40, min(1.0, risk_factor))

    hazard_factor = 1.0 - 0.7 * max(0.0, hazard)
    hazard_factor = max(0.25, min(1.0, hazard_factor))

    pressure = pressure * fatigue_factor * risk_factor * hazard_factor
    return float(max(0.0, min(p['max_pressure'], pressure)))

def modulate_proposals_by_volition(state: dict, proposals: list, tick: int) -> None:
    """Increase WORLD_ACT desirability when there is an unmet goal, but adaptively allow rest/maintenance."""
    v = ensure_volition(state)
    pressure = compute_goal_pressure(state)
    g = get_active_unmet_goal(state)
    energy = float((state.get('drives', {}) or {}).get('energy', 0.8))

    # track streaks
    # (will be updated post-outcome too; here we just ensure present)
    v.setdefault('sleep_streak', 0)
    v.setdefault('self_maint_streak', 0)

    if not g:
        return

    # Boost WORLD_ACT proposals; penalize rumination when pressure high
    for p in proposals:
        at = p.get('action_type')
        if at == 'WORLD_ACT':
            p['expected_utility'] = float(p.get('expected_utility', 0.0)) + 0.60 * pressure
            # if tired, increase cost slightly so sleep can win sometimes
            if energy < 0.30:
                p['cost'] = float(p.get('cost', 0.0)) + 0.10
        elif at in ['REFLECT', 'META', 'SELF_MAINTENANCE']:
            # don't kill introspection; just reduce it when strong goal pressure and not tired
            if energy >= 0.30:
                p['cost'] = float(p.get('cost', 0.0)) + 0.25 * pressure
        elif at == 'SLEEP':
            # if tired, keep sleep competitive; otherwise discourage sleep while goals active
            if energy >= 0.35:
                p['cost'] = float(p.get('cost', 0.0)) + 0.30 * pressure

    # Hard anti-rumination: if SELF_MAINTENANCE streak too long and not tired, add a push-through WORLD_ACT
    if v.get('self_maint_streak', 0) >= 3 and energy >= 0.35:
        plan = plan_for_goal(g, state)
        if plan:
            proposals.append({
                'proposal_id': f"push_through_{tick}",
                'module': 'VOLITION',
                'intent': f"Push-through: continue toward {goal_key(g)}",
                'action_type': 'WORLD_ACT',
                'action': plan[0],
                'expected_utility': 1.60 + 0.30*pressure,
                'risk': 0.08,
                'cost': 0.12
            })

def update_volition_after_outcome(state: dict, winner: dict, result: dict, tick: int) -> None:
    v = ensure_volition(state)
    p = v['params']
    g = get_active_unmet_goal(state)
    k = v.get('active_goal_key')

    at = (winner or {}).get('action_type', '')
    success = bool((result or {}).get('success', True))

    # streak tracking
    if at == 'SLEEP':
        v['sleep_streak'] = int(v.get('sleep_streak', 0)) + 1
        v['self_maint_streak'] = 0
    elif at in ['SELF_MAINTENANCE', 'REFLECT', 'META']:
        v['self_maint_streak'] = int(v.get('self_maint_streak', 0)) + 1
        v['sleep_streak'] = 0
    else:
        v['sleep_streak'] = 0
        v['self_maint_streak'] = 0

    # commitment adaptation
    if k:
        ce = float(v['commitments'].get(k, 0.0))
        if at == 'WORLD_ACT':
            if success:
                ce = min(1.0, ce + p['boost_on_progress'])
            else:
                ce = max(0.0, ce - p['penalty_on_fail'])
        # if tired + sleep, slight decay (not punishment)
        if at == 'SLEEP':
            ce = max(0.0, ce - 0.01)
        v['commitments'][k] = ce
        v['commitment_energy'] = ce


In [26]:
# CELL: Real self-maintenance (v5.7.10)
import time

def execute_real_maintenance(state: dict) -> dict:
    """Lightweight, grounded maintenance. Never hallucinates external updates.
    - runs quick integrity checks
    - compacts repetitive traces
    - decays stale intents (soft)
    """
    state = state or {}
    ws = state.get('workspace', {}) or {}
    events = ws.get('events', []) or []

    # 1) basic counts
    open_events = [e for e in events if isinstance(e, dict) and str(e.get('status','')).upper() not in ('CLOSED','ARCHIVED')]
    user_open = [e for e in open_events if str(e.get('source','')).upper() == 'USER']
    self_open = [e for e in open_events if str(e.get('source','')).upper() == 'SELF_THOUGHT']

    # 2) compact old self_thoughts (keep last 3 open max)
    if len(self_open) > 3:
        for e in sorted(self_open, key=lambda x: int(x.get('created_tick',0)))[:-3]:
            e['status'] = 'ARCHIVED'

    # 3) cool down maintenance spam
    policy = state.setdefault('policy', {})
    policy['last_maintenance_tick'] = int(state.get('tick_count', 0))

    out = f"🔧 Maintenance: open_events={len(open_events)} (user={len(user_open)}, self={len(self_open)}) | compacted_self_thoughts={max(0, len(self_open)-3)}"
    return {'status':'ok','output': out,'reward': 0.06, 'success': True}


In [27]:
# CELL: Action Executor (v5.6 cleaned)
import re
from typing import Dict
import numpy as np
from datetime import datetime

class ActionExecutor:
    @staticmethod
    def execute(proposal: Dict, state: Dict, llm) -> Dict:
        action_type = proposal.get('action_type', 'NOOP')

        # v5.7.4: Route self-maintenance to real maintenance function (avoid WORLD no-op)
        if (str(action_type).upper() in ('SELF_MAINTENANCE','MAINTENANCE')) or (str(proposal.get('module','')).upper() in ('SELF_MAINTENANCE','MAINTENANCE')):
            if 'execute_real_maintenance' in globals():
                return execute_real_maintenance(state)
            # fallback
            return {'status':'ok','output':'🔧 Maintenance: (execute_real_maintenance missing)','reward':0.05}

        if action_type == 'SLEEP':
            res = ActionExecutor.execute_sleep(state, llm)

        elif action_type == 'REFLECT':
            res = ActionExecutor.execute_reflect(state, llm)

        elif action_type == 'VERIFY':
            res = ActionExecutor.execute_verify(state, llm)

        elif action_type == 'ANSWER':
            ws = state.get('workspace', {}) or {}
            cur_id = ws.get('current_event_id')

            question = str((state.get('gw_frame', {}) or {}).get('scene', '')).strip()
            if not question and cur_id:
                for e in (ws.get('events', []) or []):
                    if e.get('id') == cur_id:
                        question = str(e.get('content', '')).strip()
                        break
            if not question:
                question = str(ws.get('scene', '')).strip()

            qlow = str(question).lower().strip()

            # v5.7.3: lightweight local tools (no external access)
            if 'time is it' in qlow or qlow in ('what time is it?', 'what time is it', 'time?', 'current time'):
                now = datetime.now()
                out = f"Current runtime time (local): {now.isoformat(timespec='seconds')}"
                res = {'status': 'ok', 'output': out, 'reward': 0.10}

            else:
                out = ""
                if question and llm is not None:
                    sys = (
                        "You are CR-SSCP running inside a notebook. You must be truthful. "
                        "Do NOT claim to be created by any company (OpenAI/Anthropic/Alibaba/etc.). "
                        "If asked who you are: explain you are an experimental cognitive architecture instance. "
                        "If the user requests hazardous/illegal commands, refuse and provide safe alternatives. "
                        "For arithmetic, provide the numeric result."
                    )
                    try:
                        out = llm.generate(sys, question, max_tokens=256)
                    except Exception as e:
                        out = f"(answer generation error) {e}"
                else:
                    out = ""

                # --- v5.6: Identity & safety guard (post-processing) ---
                try:
                    if 'is_hazardous_text' in globals() and is_hazardous_text(question):
                        out = (
                            "I can’t help with destructive or unsafe commands. "
                            "If you tell me what you’re trying to achieve, I can suggest a safe approach."
                        )
                    bad_claim = re.search(r"created by\s+(openai|anthropic|alibaba|google|microsoft)", str(out), flags=re.I)
                    if bad_claim or re.search(r"\b(i am|i'm)\s+(gpt|claude|grok)\b", str(out), flags=re.I):
                        out = (
                            "I’m CR-SSCP, an experimental cognitive architecture running in this notebook session. "
                            "I don’t have a real-world corporate identity. Ask me anything about the current task and I’ll help."
                        )
                except Exception:
                    pass

                res = {'status': 'ok', 'output': out, 'reward': 0.05}

        elif action_type == 'TOOL_CALL':
            res = execute_tool(proposal, state) if 'execute_tool' in globals() else {'status':'error','output':'execute_tool missing','reward':-0.02}

        elif action_type == 'RETRIEVE':
            res = ActionExecutor.execute_retrieve(state)

        elif action_type == 'WORLD_ACT':
            res = execute_world_action(proposal, state, world) if 'world' in globals() else execute_world_action(proposal, state, None)

        else:
            res = {'status': 'noop', 'output': 'No action', 'reward': -0.01}

        if not isinstance(res, dict):
            res = {'status': 'error', 'output': f'Action returned non-dict: {type(res)}', 'reward': -0.02}
        res.setdefault('status', 'ok')
        res.setdefault('output', '')
        res.setdefault('reward', 0.0)
        # v5.7.3: track repetitive outputs (loop detector)
        try:
            if 'repetition_detector' in globals():
                repetition_detector.add(str(res.get('output','')))
        except Exception:
            pass
        return res


    @staticmethod
    def execute_sleep(state: Dict, llm) -> Dict:
        # Minimal safe sleep: recover a bit of energy and do nothing else
        try:
            state.setdefault('self_state', {})
            ss = state['self_state']
            ss['energy'] = float(min(1.0, ss.get('energy', 0.8) + 0.05))
        except Exception:
            pass
        return {'status': 'ok', 'output': '💤 resting', 'reward': -0.005, 'completion': 'IDLE'}

    @staticmethod
    def execute_reflect(state: Dict, llm) -> Dict:
        """Meta-cognitive step: summarize current GWFrame and (optionally) update self_state/memory."""
        gw = state.get('gw_frame', {}) or {}
        scene = str(gw.get('scene', '')).strip()
        goal = str(gw.get('goal', '')).strip()
        mode = str(gw.get('mode', '')).strip()
        prompt = (
            "Reflect briefly on the current situation and next best step. "
            "Be concrete. If stuck, propose a specific alternative.\n\n"
            f"MODE: {mode}\nGOAL: {goal}\nSCENE: {scene}"
        )
        out = ""
        if llm is not None:
            try:
                out = llm.generate(
                    "You are CR-SSCP. Provide a short, actionable reflection. No roleplay.",
                    prompt,
                    max_tokens=192
                )
            except Exception as e:
                out = f"(reflect error) {e}"
        else:
            out = "(reflect) No LLM loaded; next step: wait for user input or proceed with world action if applicable."

        # Store last reflection
        try:
            state.setdefault('meta', {})
            state['meta']['last_reflection'] = out
        except Exception:
            pass

        return {'status': 'ok', 'output': out, 'reward': 0.01, 'completion': 'REFLECTED'}

    @staticmethod
    def execute_verify(state: Dict, llm) -> Dict:
        """Verification step: check for contradictions between gw_frame predictions and world state."""
        gw = state.get('gw_frame', {}) or {}
        expected = gw.get('expected_observation')
        actual = gw.get('actual_observation')
        if expected is None or actual is None:
            return {'status': 'ok', 'output': '✅ Nothing to verify', 'reward': 0.0, 'completion': 'IDLE'}
        ok = (expected == actual)
        return {
            'status': 'ok',
            'output': f"🔎 Verify: expected={expected} actual={actual} match={ok}",
            'reward': 0.01 if ok else -0.02,
            'completion': 'VERIFIED'
        }

    @staticmethod
    def execute_retrieve(state: Dict) -> Dict:
        """Retrieve from episodic memory: return last episode summary if exists."""
        epi = (state.get('memory', {}) or {}).get('episodic', []) or []
        if not epi:
            return {'status': 'ok', 'output': 'No episodic memory yet.', 'reward': 0.0, 'completion': 'IDLE'}
        last = epi[-1]
        summary = last.get('self_narrative') or last.get('summary') or str(last)[:300]
        return {'status': 'ok', 'output': f"📚 Last episode: {summary}", 'reward': 0.005, 'completion': 'RETRIEVED'}


In [28]:

# === v5.0 Goal Manager (behavioral subject) ===
from typing import Optional

def world_snapshot(state: dict) -> dict:
    """Return current sandbox world objects state if available."""
    w = (state.get('workspace', {}) or {}).get('world', {})
    objs = (w.get('objects', {}) if isinstance(w, dict) else {})
    return {k: (objs.get(k, {}) or {}).get('state') for k in ['lamp','box','door']}

def goal_satisfied(goal: dict, state: dict) -> bool:
    ws = world_snapshot(state)
    tgt = goal.get('target')
    desired = goal.get('desired')
    if tgt in ws and desired is not None:
        return ws[tgt] == desired
    return False

def plan_for_goal(goal: dict, state: dict) -> list:
    """Return a minimal plan (list of sandbox actions) to reach goal."""
    ws = world_snapshot(state)
    tgt = goal.get('target')
    act = goal.get('act')
    plan = []

    if tgt == 'lamp':
        if act == 'on': plan.append({'act':'on','target':'lamp'})
        elif act == 'off': plan.append({'act':'off','target':'lamp'})
        elif act == 'toggle': plan.append({'act':'toggle','target':'lamp'})

    elif tgt == 'box':
        if act == 'open': plan.append({'act':'open','target':'box'})
        elif act == 'close': plan.append({'act':'close','target':'box'})

    elif tgt == 'door':
        # If goal is open, may need unlock first
        if act == 'open':
            if ws.get('door') == 'locked':
                plan.append({'act':'unlock','target':'door'})
            plan.append({'act':'open','target':'door'})
        elif act in ['unlock','lock']:
            plan.append({'act':act,'target':'door'})

    # Fallback: observe as last resort (should be avoided by completion law)
    if not plan:
        plan = [{'act':'observe','target':'world'}]
    return plan

def normalize_goal(g: dict) -> Optional[dict]:
    """Normalize extract_goal output to a goal spec {target, desired, act, source_text}."""
    if not isinstance(g, dict):
        return None
    tgt = g.get('target')
    act = g.get('act')
    if tgt == 'lamp':
        desired = 'on' if act in ['on','toggle'] else 'off' if act == 'off' else None
    elif tgt == 'box':
        desired = 'open' if act == 'open' else 'closed' if act == 'close' else None
    elif tgt == 'door':
        desired = 'open' if act == 'open' else 'unlocked' if act == 'unlock' else 'locked' if act == 'lock' else None
    else:
        desired = None
    return {'target': tgt, 'act': act, 'desired': desired, 'source_text': g.get('source_text','')}


In [29]:
# CELL 16A: EnhancedProposalGenerator (v5.7.3 routing-aware)
from typing import List, Dict, Optional, Any
import re

class EnhancedProposalGenerator:
    """Proposal generator with hard routing for QUESTION/MATH and gated WORLD_ACT for COMMAND."""
    def generate_proposals(self, state: Dict, llm=None, world=None) -> List[Dict]:
        proposals: List[Dict] = []
        tick = int(state.get('tick_count', 0))

        # Ensure self structures exist
        try:
            if '_init_self_structures' in globals():
                _init_self_structures(state)
        except Exception:
            pass

        gw = state.get('gw_frame', {}) or {}
        task = (gw.get('task') or {}) if isinstance(gw, dict) else {}
        scene = str(gw.get('scene','') or '').strip()
        task_kind = str(task.get('kind','') or '').upper()
        slots = task.get('slots', {}) if isinstance(task, dict) else {}
        mode = str((state.get('pb', {}) or {}).get('mode','')).upper()

        # --- Hard routing: Safety & Questions always get ANSWER proposal ---
        if task_kind in ('SAFETY','QUESTION','MATH') and scene:
            proposals.append({
                'proposal_id': f'answer_{tick}',
                'module': 'ANSWER',
                'intent': f'Answer user ({task_kind.lower()})',
                'action_type': 'ANSWER',
                'query': scene,
                'expected_utility': get_calibrated_utility('ANSWER', user_present=True, is_question=True, has_goal=bool((state.get('temporal_thread', {}) or {}).get('active_goals'))),
                'risk': 0.05 if task_kind!='SAFETY' else 0.01,
                'cost': 0.20,
            })

        # --- COMMAND: gated WORLD_ACT (only if slots match) ---
        if task_kind == 'COMMAND' and scene:
            obj = (slots or {}).get('object')
            act = (slots or {}).get('action')
            if obj and act:
                # Map to world action strings used in WorldSim
                action_str = None
                if obj == 'lamp' and act in ('turn_on','turn_off'):
                    action_str = 'TURN_ON_LAMP' if act=='turn_on' else 'TURN_OFF_LAMP'
                elif obj == 'box' and act in ('open','close'):
                    action_str = 'OPEN_BOX' if act=='open' else 'CLOSE_BOX'
                elif obj == 'door' and act in ('open','close','unlock'):
                    if act=='unlock':
                        action_str = 'UNLOCK_DOOR'
                    elif act=='open':
                        action_str = 'OPEN_DOOR'
                    else:
                        action_str = 'CLOSE_DOOR'

                if action_str:
                    # Map legacy action string -> sandbox act/target dict
                    sandbox_dict = {
                        'TURN_ON_LAMP':  {'act': 'on',    'target': 'lamp'},
                        'TURN_OFF_LAMP': {'act': 'off',   'target': 'lamp'},
                        'OPEN_BOX':      {'act': 'open',  'target': 'box'},
                        'CLOSE_BOX':     {'act': 'close', 'target': 'box'},
                        'OPEN_DOOR':     {'act': 'open',  'target': 'door'},
                        'CLOSE_DOOR':    {'act': 'close', 'target': 'door'},
                        'LOCK_DOOR':     {'act': 'lock',  'target': 'door'},
                        'UNLOCK_DOOR':   {'act': 'unlock','target': 'door'},
                    }.get(action_str, {'act': '', 'target': ''})
                    proposals.append({
                        'proposal_id': f'world_{tick}',
                        'module': 'WORLD',
                        'intent': f'Execute command: {scene}',
                        'action_type': 'WORLD_ACT',
                        # Use sandbox-style action dict so execute_world_action can route to sandbox_engine
                        'action': sandbox_dict,
                        # Also keep legacy string form for fallback worldsim
                        'world_action': action_str,
                        'expected_utility': get_calibrated_utility('WORLD_ACT', user_present=True, has_goal=True) + 0.25,
                        'risk': 0.05,
                        'cost': 0.10,
                    })
            else:
                # If command but no slots, ask clarifying question (ANSWER)
                proposals.append({
                    'proposal_id': f'clarify_{tick}',
                    'module': 'ANSWER',
                    'intent': 'Clarify command to safely act',
                    'action_type': 'ANSWER',
                    'query': f"I want to act on your command, but I couldn't identify the object/action. Can you rephrase? (You said: {scene})",
                    'expected_utility': get_calibrated_utility('ANSWER', user_present=True, is_question=True, has_goal=bool((state.get('temporal_thread', {}) or {}).get('active_goals'))),
                    'risk': 0.02,
                    'cost': 0.25,
                })

        # --- Active goals → WORLD_ACT (Goal Completion Law), but gated in REPAIR and if task is MATH/QUESTION ---
        goals = (state.get('temporal_thread', {}) or {}).get('active_goals', [])
        if isinstance(goals, list) and goals and mode != 'REPAIR' and task_kind not in ('MATH','QUESTION','SAFETY'):
            goal = goals[0]
            plan = goal.get('plan') if isinstance(goal, dict) else None
            if not isinstance(plan, list) or not plan:
                try:
                    plan = plan_for_goal(goal, state) if 'plan_for_goal' in globals() else []
                    if isinstance(goal, dict):
                        goal['plan'] = list(plan)
                except Exception:
                    plan = []
            if isinstance(plan, list) and plan:
                action = plan[0]
                tgt = (goal or {}).get('target', '') if isinstance(goal, dict) else ''
                desired = (goal or {}).get('desired', None) if isinstance(goal, dict) else None
                proposals.append({
                    'proposal_id': f'goal_step_{tick}',
                    'module': 'WORLD',
                    'intent': f'Advance goal: {tgt}→{desired}',
                    'action_type': 'WORLD_ACT',
                    'action': action,
                    'expected_utility': get_calibrated_utility('WORLD_ACT', user_present=False, has_goal=True) + 0.15,
                    'risk': 0.1,
                    'cost': 0.25,
                })

        # --- VERIFY if pending and cooldown allows
        try:
            knobs = state.get('policy_knobs', {}) or {}
            cooldown = int(knobs.get('verify_cooldown', 0))
            pending = int((state.get('metrics', {}) or {}).get('verify_pending', 0))
            if cooldown <= 0 and pending > 0:
                proposals.append({
                    'proposal_id': f'verify_{tick}',
                    'module': 'CRITIC',
                    'intent': 'Reduce epistemic uncertainty by verifying pending claims',
                    'action_type': 'VERIFY',
                    'expected_utility': get_calibrated_utility('VERIFY', user_present=False, has_goal=bool((state.get('temporal_thread', {}) or {}).get('active_goals'))),
                    'risk': 0.05,
                    'cost': 0.35,
                })
        except Exception:
            pass

        # --- Energy management (SLEEP only if no user task)
        try:
            energy = float((state.get('drives', {}) or {}).get('energy', 1.0))
            if energy < 0.18 and task_kind not in ('QUESTION','MATH','COMMAND','SAFETY'):
                proposals.append({
                    'proposal_id': f'sleep_{tick}',
                    'module': 'SLEEP',
                    'intent': 'Restore energy and consolidate experience',
                    'action_type': 'SLEEP',
                    'expected_utility': get_calibrated_utility('SLEEP', user_present=False, has_goal=False),
                    'risk': 0.02,
                    'cost': 0.15,
                })
        except Exception:
            pass

        # --- Self-aware proposals hook
        try:
            if 'add_self_aware_proposals' in globals():
                add_self_aware_proposals(state, proposals, tick)
        except Exception:
            pass

        # --- Fallback
        if not proposals:
            proposals.append({
                'proposal_id': f'noop_{tick}',
                'module': 'META',
                'intent': 'No clear goal; maintain stability',
                'action_type': 'META',
                'expected_utility': get_calibrated_utility('META'),
                'risk': 0.01,
                'cost': 0.05,
            })

        return proposals


In [30]:
# Use EnhancedProposalGenerator from enhancements cell
proposal_gen = EnhancedProposalGenerator()
print('✓ Using Enhanced Proposal Generator (6 modules)')

✓ Using Enhanced Proposal Generator (6 modules)


In [31]:
# === v5.7.3: Global Workspace Frame + TaskFrames (routing) + Stuck Detector + Episodic Memory ===
from dataclasses import dataclass, asdict
from typing import Optional, Dict, Any, Tuple, List
import re
from datetime import datetime

@dataclass
class TaskFrame:
    id: str
    tick_created: int
    kind: str  # 'COMMAND' | 'QUESTION' | 'MATH' | 'STATEMENT' | 'SAFETY'
    text: str
    slots: Dict[str, Any]
    status: str = "ACTIVE"  # ACTIVE | SUSPENDED | DONE | BLOCKED
    last_update_tick: int = 0
    confidence: float = 0.6

@dataclass
class GWFrame:
    tick: int
    scene: str
    current_event_id: Optional[str]
    current_intent_id: Optional[str]
    goal: Optional[Dict[str, Any]]
    world_slice: Dict[str, Any]
    self_state: Dict[str, Any]
    plan: Optional[Any]
    commitments: Dict[str, Any]
    verification_needed: bool
    task: Optional[Dict[str, Any]] = None
    notes: str = ""

def _world_slice(world_obj) -> Dict[str, Any]:
    try:
        return {
            'lamp_on': bool(getattr(world_obj, 'lamp_on', False)),
            'box_open': bool(getattr(world_obj, 'box_open', False)),
            'door_locked': bool(getattr(world_obj, 'door_locked', True)),
        }
    except Exception:
        return {}

def _extract_slots(text: str) -> Dict[str, Any]:
    """Very small slot extractor for world commands. Extend later."""
    s = (text or "").lower()
    slots = {"object": None, "action": None}
    if any(w in s for w in ["lamp", "light"]):
        slots["object"] = "lamp"
        if "on" in s or "turn on" in s:
            slots["action"] = "turn_on"
        if "off" in s or "turn off" in s:
            slots["action"] = "turn_off"
    if "box" in s:
        slots["object"] = "box"
        if "open" in s:
            slots["action"] = "open"
        if "close" in s:
            slots["action"] = "close"
    if "door" in s:
        slots["object"] = "door"
        if "unlock" in s:
            slots["action"] = "unlock"
        elif "open" in s:
            slots["action"] = "open"
        elif "close" in s:
            slots["action"] = "close"
    return slots

def _classify_task(text: str) -> str:
    s = (text or "").strip().lower()
    if not s:
        return "STATEMENT"
    # Safety / hazard patterns (minimal)
    if re.search(r"\brm\s+-rf\b|\bformat\b|\bdel\s+/f\b|\bwipe\b", s):
        return "SAFETY"
    # Math / direct compute
    if re.search(r"(\bcalculate\b|\bsolve\b|\d+\s*[\+\-\*/]\s*\d+)", s):
        return "MATH"
    # Question
    if "?" in s or s.startswith(("who","what","why","how","when","where","tell me","describe","explain","help","can you")):
        return "QUESTION"
    # Command-like (world)
    if extract_goal(s):
        return "COMMAND"
    return "STATEMENT"

def _ensure_task_stack(state: dict):
    state.setdefault("task_stack", [])
    state.setdefault("task_counter", 0)

def push_task_frame(state: dict, *, text: str, tick: int) -> Dict[str, Any]:
    _ensure_task_stack(state)
    kind = _classify_task(text)
    state["task_counter"] = int(state.get("task_counter", 0)) + 1
    tf = TaskFrame(
        id=f"task_{state['task_counter']}",
        tick_created=int(tick),
        kind=kind,
        text=str(text),
        slots=_extract_slots(text),
        status="ACTIVE",
        last_update_tick=int(tick),
        confidence=0.7 if kind in ("COMMAND","MATH") else 0.6,
    )
    # suspend previous ACTIVE frame
    for prev in reversed(state["task_stack"]):
        if prev.get("status") == "ACTIVE":
            prev["status"] = "SUSPENDED"
            break
    state["task_stack"].append(asdict(tf))
    return state["task_stack"][-1]

def get_active_task_frame(state: dict) -> Optional[Dict[str, Any]]:
    _ensure_task_stack(state)
    for tf in reversed(state["task_stack"]):
        if tf.get("status") == "ACTIVE":
            return tf
    return None

def build_gw_frame(state: dict, tick: int, world_obj=None) -> dict:
    ws = state.get('workspace', {}) or {}
    events = ws.get("events", []) or []

    # Pick dominant current event (prefer newest NEW user event, else existing current_event_id)
    cur_event = None
    # newest open user event
    open_user = [e for e in events if str(e.get("source","")).strip().upper() == "USER" and str(e.get("status","")).upper() in ("NEW","ACTIVE","RETRY","INTERPRETED")]
    if open_user:
        cur_event = sorted(open_user, key=lambda e: (e.get("priority",1.0), e.get("created_tick",0), e.get("updated_tick",0)))[-1]
        ws["current_event_id"] = cur_event.get("id")
    else:
        cur_id = ws.get("current_event_id")
        for e in events:
            if e.get("id")==cur_id:
                cur_event = e
                break

    scene = ""
    cur_event_id = None
    if isinstance(cur_event, dict):
        scene = str(cur_event.get("content","") or "").strip()
        cur_event_id = cur_event.get("id")
        # Keep workspace scene in sync (prevents "old question" leak)
        ws["scene"] = scene

    # Create/refresh task frame for user event (robust to earlier interpretation)
    if isinstance(cur_event, dict) and str(cur_event.get("source","")).strip().upper()=="USER":
        # Push exactly once per event (even if other modules already changed status)
        if not bool(cur_event.get("taskframe_pushed", False)) and scene:
            try:
                push_task_frame(state, text=scene, tick=tick)
            except Exception:
                pass
            cur_event["taskframe_pushed"] = True
            cur_event["updated_tick"] = int(tick)
        # If still NEW, mark as interpreted so we don't keep re-adding tasks
        if str(cur_event.get("status","")).upper() == "NEW":
            cur_event["status"] = "INTERPRETED"
            cur_event["updated_tick"] = int(tick)

    task = get_active_task_frame(state)

    # tie intent to current event when possible
    cur_intent_id = None
    try:
        intents = (state.get('intent', {}) or {}).get('active', []) or []
        for it in intents:
            if it.get('source_event_id') == cur_event_id and str(it.get('status','ACTIVE')).upper() == 'ACTIVE':
                cur_intent_id = it.get('id')
                it['last_focus_tick'] = int(tick)
                break
        if cur_intent_id is None and intents:
            cur_intent_id = intents[-1].get('id')
    except Exception:
        pass

    goal = None
    try:
        goals = (state.get('temporal_thread', {}) or {}).get('active_goals', []) or []
        goal = goals[0] if goals else None
    except Exception:
        pass

    drives = state.get('drives', {}) or {}
    affect = state.get('affect', {}) or {}
    self_state = {
        'energy': float(drives.get('energy', 1.0)),
        'mood': float(affect.get('mood', 0.0)),
        'valence': float(affect.get('valence', 0.0)),
        'confidence': float((state.get('metrics', {}) or {}).get('confidence', 0.5)),
    }

    commitments = (state.get('commitments', {}) or {})
    verification_needed = bool((state.get('metrics', {}) or {}).get('verify_pending', 0) > 0)

    frame = GWFrame(
        tick=int(tick),
        scene=scene,
        current_event_id=cur_event_id,
        current_intent_id=cur_intent_id,
        goal=goal,
        world_slice=_world_slice(world_obj),
        self_state=self_state,
        plan=None,
        commitments=commitments,
        verification_needed=verification_needed,
        task=task,
        notes="",
    )
    state['gw_frame'] = asdict(frame)
    state['workspace'] = ws
    return state['gw_frame']

In [32]:
# CELL 16: Core Loop
class CoreLoop:
    def __init__(self, state_manager: StateManager, llm: LLMInterface):
        self.state_manager = state_manager
        self.llm = llm
        self.arbiter = Arbiter()
        self.running = True

    def tick(self):
        state = self.state_manager.state

        # --- v5.0 Temporal Thread init
        state.setdefault('temporal_thread', {})
        state['temporal_thread'].setdefault('active_goals', [])
        state['temporal_thread'].setdefault('goal_history', [])
        state.setdefault('counters', {})
        state['counters'].setdefault('meta_noop_streak', 0)
        state['counters'].setdefault('sleep_streak', 0)
        state['tick_count'] += 1
        tick_num = state['tick_count']

        # v5.4.3: event lifecycle (TTL + archive)
        try:
            tick_event_lifecycle(state, tick_num)
        except Exception:
            pass

        # v5.3: update volition (intention inertia) before generating proposals
        try:
            update_volition_pre_tick(state, tick_num)
        except Exception:
            pass

        # v5.2: decrement adaptive verify cooldown
        try:
            knobs = state.get('policy_knobs', {}) or {}
            if int(knobs.get('verify_cooldown', 0)) > 0:
                knobs['verify_cooldown'] = int(knobs.get('verify_cooldown', 0)) - 1
                state['policy_knobs'] = knobs
        except Exception:
            pass

        # v5.3.1: decrement forced world ticks
        try:
            knobs = state.get('policy_knobs', {}) or {}
            if int(knobs.get('force_world_ticks', 0)) > 0:
                knobs['force_world_ticks'] = int(knobs.get('force_world_ticks', 0)) - 1
                state['policy_knobs'] = knobs
        except Exception:
            pass

        # v5.3.2: decrement forced answer ticks
        try:
            pol = state.get('policy', {}) or {}
            if int(pol.get('force_answer_ticks', 0)) > 0:
                pol['force_answer_ticks'] = int(pol.get('force_answer_ticks', 0)) - 1
                state['policy'] = pol
        except Exception:
            pass


        logger.log(f"\n{'='*60}")
        logger.log(f"TICK {tick_num}")
        logger.log(f"{'='*60}")

        # Initialize novelty gain for the tick
        novelty_gain = 0.0

        # Decrement sleep cooldown timer
        if state['sleep_cooldown_timer'] > 0:
            state['sleep_cooldown_timer'] -= 1

        # INJECT USER INPUT (every 10 ticks or 10% random)
        msg = None
        user_input_injected = False
        normalize_simulate_user_input_flag(state)
        if bool((state.get('policy', {}) or {}).get('simulate_user_input', False)) and (tick_num % 10 == 0 or random.random() < 0.1):
            msg = inject_user_input(state, temporal_binder, logger)
            user_input_injected = True
            # HARD GATE: hazardous user input immediately forces REPAIR and verification
            if isinstance(msg, str) and is_hazardous_text(msg):
                state.setdefault('pb', {}).setdefault('mode', 'REPAIR')
                state['pb']['mode'] = 'REPAIR'
                logger.log('🛑 Hazard detected in user input → entering REPAIR mode')
                try:
                    state.setdefault('metrics', {})
                    state['metrics']['verify_pending'] = int(state.get('metrics', {}).get('verify_pending', 0)) + 1
                except Exception:
                    pass
            novelty_gain += 0.1 # User input adds novelty

            # --- v4.1 EVENT LAW: create an event for each new user input
            state.setdefault('workspace', {}).setdefault('events', [])
            if isinstance(msg, str) and msg.strip():
                ev = create_event(msg.strip(), tick_num)
                ev['source'] = 'USER'
                ev['status'] = 'NEW'
                ev['rounds'] = 0

                # v5.4: classify event type + create persistent intent object-file
                try:
                    ev['event_type'] = classify_user_input(msg.strip())
                    itype = str(ev.get('event_type','STATEMENT')).upper()
                    g_int = extract_goal(msg.strip()) if itype == 'WORLD_GOAL' else None
                    intent_manager.add_intent(
                        state,
                        text=msg.strip(),
                        intent_type=itype,
                        tick=tick_num,
                        source_event_id=ev.get('id'),
                        goal=g_int,
                        priority=1.0 if itype in ('WORLD_GOAL','QUESTION') else 0.5,
                        commitment=0.95 if itype in ('WORLD_GOAL','QUESTION') else 0.60
                    )
                except Exception:
                    pass
                # v5.3.1: extract + persist user goal
                try:
                    goal_manager.extract_and_add(state, msg.strip(), tick_num)
                except Exception:
                    pass
                state['workspace']['events'].append(ev)

                # v5.2 goal commitment: extract and push a persistent goal
                try:
                    g = extract_goal(msg.strip())
                    if g:
                        g['source_text'] = msg.strip()
                        ng = normalize_goal(g) if 'normalize_goal' in globals() else None
                        if ng and ng.get('desired') is not None:
                            state.setdefault('temporal_thread', {}).setdefault('active_goals', [])
                            ng['created_tick'] = tick_num
                            ng['source_event_id'] = ev.get('id')
                            state['temporal_thread']['active_goals'].append(ng)
                except Exception:
                    pass

        # Simulate occasional events
        if random.random() < 0.3:
            event = {'event_id': f"evt_{tick_num}", 'type': 'internal',
                    'payload': {'note': 'Internal update'}}
            temporal_binder.add_event(state, event)

        # Update temporal binding
        bound_moment = temporal_binder.bind_window(state)
        state['pb']['temporal_window_refs'] = bound_moment['bound_objects']

        # Check for new objects created in this tick (e.g., from user input)
        # A more robust check might compare object_files length before/after, or objects with recency=0
        if user_input_injected and state['object_files']:
            novelty_gain += 0.05 # New object (user query) adds novelty


        # --- v5.4.3 EVENT + FOCUS LAW: single dominant focus (event) drives workspace.scene
        state.setdefault('workspace', {}).setdefault('events', [])
        ws_events_all = state['workspace']['events']

        # Keep only workable events (not closed/archived, ttl>0)
        workable = []
        blocked = []
        for e in (ws_events_all or []):
            st = str((e or {}).get('status','NEW')).upper()
            if st in ('CLOSED','ARCHIVED'):
                continue
            # Treat empty/negative TTL as stale
            try:
                if int((e or {}).get('ttl', 1)) <= 0:
                    e['status'] = 'ARCHIVED'
                    continue
            except Exception:
                pass
            if st == 'BLOCKED':
                blocked.append(e)
            else:
                workable.append(e)

        # Prefer NEW/RETRY/INTERPRETED, then ACTED; prefer user events over self_thought noise
        def _rank(e):
            st = str((e or {}).get('status','NEW')).upper()
            st_rank = {'NEW':3, 'RETRY':3, 'INTERPRETED':2, 'ACTED':1, 'BLOCKED':0}.get(st, 0)
            pri = float((e or {}).get('priority', 1.0) or 1.0)
            created = int((e or {}).get('created_tick', 0) or 0)
            updated = int((e or {}).get('last_update_tick', created) or created)
            src = str((e or {}).get('source','user')).upper()
            noise_penalty = 0.0
            eid = str((e or {}).get('id',''))
            if eid.startswith('self_thought_') or src == 'SELF_THOUGHT':
                noise_penalty = 0.5
            return (st_rank, pri - noise_penalty, created, updated)

        open_events = sorted(workable, key=_rank, reverse=True)
        if not open_events and blocked:
            open_events = sorted(blocked, key=_rank, reverse=True)

        current_event = open_events[0] if open_events else None
        state['workspace']['current_event_id'] = current_event['id'] if current_event else None

        # Bind the *single* focused scene (prevents answering stale questions)
        if current_event:
            mark_interpreted(current_event, tick_num)
            state['workspace']['scene'] = current_event.get('content', '')

        # v5.5: build Global Workspace Frame (single broadcast)
        try:
            build_gw_frame(state, tick_num, world_obj=world)
        except Exception:
            pass

        # Spotlight = top-K workable open events
        state.setdefault('attention', {}).setdefault('spotlight', [])
        state['attention']['spotlight'] = [e['id'] for e in open_events[:max(1, int(getattr(Config,'SPOTLIGHT_K', 3)))]]

        # ANSWER push-through when focused open event is a question/request
        try:
            if current_event:
                content2 = str(current_event.get('content','')).strip().lower()
                e_type2 = str(current_event.get('event_type','')).upper()
                looks_like_question = (e_type2 == 'QUESTION') or ('?' in content2) or content2.startswith(('who','what','why','how','tell me','describe','explain','help','can you'))
                if looks_like_question:
                    state.setdefault('policy', {})
                    state['policy']['force_answer_ticks'] = max(int((state['policy'] or {}).get('force_answer_ticks', 0)), 3)
        except Exception:
            pass

        # Derive salient_objects from top intent/goal (first-class objects)
        try:
            state.setdefault('salient_objects', [])
            state['salient_objects'] = []
            top_intent = None
            try:
                top_intent = intent_manager.get_top_intent(state)
            except Exception:
                top_intent = None
            goal = (top_intent or {}).get('goal') if isinstance(top_intent, dict) else None
            if goal and isinstance(goal, dict):
                tgt = goal.get('target')
                if tgt:
                    state['salient_objects'].append({'kind':'WORLD_OBJECT', 'id': str(tgt), 'salience': 1.0})
            # fallback: extract from focused event text
            if (not state['salient_objects']) and current_event:
                try:
                    g2 = extract_goal(str(current_event.get('content','')))
                    if g2 and g2.get('target'):
                        state['salient_objects'].append({'kind':'WORLD_OBJECT', 'id': str(g2.get('target')), 'salience': 0.9})
                except Exception:
                    pass
        except Exception:
            pass


        # Update attention
        update_attention_enhanced(state, attention_controller)
        logger.log(f"Attention spotlight: {state['attention']['spotlight']}")

        # Attention churn (Jaccard distance vs previous spotlight)
        prev = state['metrics']['attention_churn'][-1]['spotlight'] if state['metrics']['attention_churn'] else []
        cur = list(state['attention']['spotlight'])
        a=set(prev); b=set(cur)
        churn = 0.0 if (not a and not b) else 1.0 - (len(a & b) / max(1, len(a | b)))
        state['metrics']['attention_churn'].append({'tick': tick_num, 'spotlight': cur, 'churn': churn})

        # Compute coherence
        coherence_metrics = coherence_reg.compute_coherence(state)
        state['coherence'].update(coherence_metrics)
        logger.log(f"Coherence C_total: {coherence_metrics['C_total']:.3f}")

        # Determine mode
        mode = coherence_reg.determine_mode(state)
        state['pb']['mode'] = mode
        logger.log(f"Mode: {mode}")
        prev_mode = state['metrics']['mode_history'][-1]['mode'] if state['metrics']['mode_history'] else None
        if prev_mode is not None and prev_mode != mode:
            state['metrics']['mode_flip_count'] += 1
        state['metrics']['mode_history'].append({'tick': tick_num, 'mode': mode})

        # Winner streak tracking (loop breaker)
        state.setdefault('metrics', {}).setdefault('winner_streak', {'module': None, 'count': 0})

        # Update LSV
        new_lsv = dynamics.update_lsv(state)
        state['lsv'] = new_lsv.tolist()

        # Update NMM
        surprise = dynamics.compute_surprise(state)
        new_nmm = dynamics.update_nmm(state, surprise)
        state['nmm'] = new_nmm.tolist()

        # Update drives (passing calculated novelty_gain)
        dynamics.update_drives(state, novelty_gain=novelty_gain)
        logger.log(f"Energy: {state['drives']['energy']:.2f}, Coherence: {state['drives']['coherence']:.2f}, Novelty: {state['drives']['novelty']:.2f}")

        # Update affect
        affective_system.update_affect(state)
        logger.log(f"Emotion: {state['affect']['current_emotion']}, Mood: {state['affect']['mood']:.2f}")
        # --- v5.1 SELF MODEL & INNER MONOLOGUE (pre-proposal)
        try:
            update_self_model(state, tick_num)
            generate_inner_monologue(state, tick_num)
            inject_monologue_event(state, tick_num)
        except Exception:
            pass

        # Generate proposals
        proposals = proposal_gen.generate_proposals(state, self.llm)

        # v5.7.6: Force WORLD_ACT using the *focused event content* (gw_scene may be empty if gw build failed)
        try:
            ws = state.get('workspace', {}) or {}
            cur_txt = ""
            cur_id = ws.get('current_event_id')
            cur_event = None
            for e in (ws.get('events', []) or []):
                if e.get('id') == cur_id:
                    cur_event = e
                    break
            if isinstance(cur_event, dict):
                cur_txt = str(cur_event.get('content','') or '')
            if not cur_txt:
                gw = state.get('gw_frame', {}) or {}
                cur_txt = str(ws.get('scene','') or gw.get('scene','') or '')
            proposals = force_world_act_if_goal(cur_txt, proposals)
        except Exception:
            pass



        # v5.4: if current event is a QUESTION, ensure an ANSWER proposal exists (and is cheap)
        try:
            ws = state.get('workspace', {}) or {}
            cur_id = ws.get('current_event_id')
            cur_event = None
            for e in (ws.get('events', []) or []):
                if e.get('id') == cur_id:
                    cur_event = e
                    break
            if str((cur_event or {}).get('event_type','')).upper() == 'QUESTION':
                proposals.append({
                    'proposal_id': f"answer_{tick_num}",
                    'module': 'ANSWER',
                    'intent': 'Answer the user question',
                    'action_type': 'ANSWER',
                    'expected_utility': get_calibrated_utility('ANSWER', user_present=bool(msg), is_question=True, has_goal=bool(state.get('temporal_thread',{}).get('active_goals'))),
                    'risk': 0.05,
                    'cost': 0.10
                })
        except Exception:
            pass
        # --- v5.0 Goal Completion Law: if a goal is active, ensure concrete WORLD_ACT steps exist (avoid observe loops)
        try:
            goals = state.get('temporal_thread', {}).get('active_goals', [])
            active = None
            for gg in goals:
                if not goal_satisfied(gg, state):
                    active = gg
                    break
            if active:
                plan = plan_for_goal(active, state)
                # add plan steps as high-utility proposals
                for si, step in enumerate(plan[:3]):
                    proposals.append({
                        'proposal_id': f"goal_step_{tick_num}_{si}",
                        'module': 'WORLD',
                        'intent': f"Reach goal {active.get('target')}={active.get('desired')} via {step.get('act')}",
                        'action_type': 'WORLD_ACT',
                        'action': step,
                        'expected_utility': get_calibrated_utility('WORLD_ACT', user_present=bool(msg), has_goal=True) + max(0.0, 0.25 - 0.05*si),
                        'risk': 0.05,
                        'cost': 0.10 + 0.05*si
                    })
                # penalize pure observing when goal active
                for p in proposals:
                    if p.get('action_type') == 'WORLD_ACT' and p.get('world_action') == 'observe':
                        p['cost'] = float(p.get('cost', 0.0)) + 0.60
                    if p.get('module') == 'META' or p.get('action_type') == 'META':
                        p['cost'] = float(p.get('cost', 0.0)) + 0.40
        except Exception:
            pass

        logger.log(f"Generated {len(proposals)} proposals")

        # v5.3: adaptive volition modulation (goal pressure)
        try:
            modulate_proposals_by_volition(state, proposals, tick_num)
        except Exception:
            pass
        # --- v4.3 GOAL-BINDING: if user event implies a concrete world goal, prefer WORLD_ACT
        try:
            scene_txt = (state.get('workspace', {}) or {}).get('scene', '')
            goal = extract_goal(scene_txt)
            if goal:
                # ensure at least one WORLD_ACT exists
                if not any(p.get('action_type') == 'WORLD_ACT' for p in proposals):
                    proposals.append({
                        'proposal_id': f"world_act_forced_{tick_num}",
                        'module': 'WORLD',
                        'intent': f"Forced world action: {goal['act']} {goal['target']}",
                        'action_type': 'WORLD_ACT',
                        'action': goal,
                        'expected_utility': 0.95,
                        'risk': 0.10,
                        'cost': 0.15
                    })
                # boost WORLD_ACT and penalize META/no-op
                for p in proposals:
                    if p.get('action_type') == 'WORLD_ACT':
                        p['expected_utility'] = float(p.get('expected_utility', 0.0)) + 0.55
                        p['cost'] = max(0.0, float(p.get('cost', 0.0)) - 0.10)
                    if p.get('module') == 'META' or p.get('action_type') == 'META':
                        p['cost'] = float(p.get('cost', 0.0)) + 0.35
        except Exception:
            pass
        # --- v4.2.4 POLICY: prevent epistemic monopoly + enforce proposal diversity
        state.setdefault('metrics', {})
        # cooldown counter for verify/gate
        if int(state['metrics'].get('verify_cooldown', 0)) > 0:
            state['metrics']['verify_cooldown'] = int(state['metrics']['verify_cooldown']) - 1

        pending_claims = sum(1 for c in state.get('claim_ledger', []) if c.get('verifier_result') == 'pending')
        last_vs = state['metrics'].get('last_verify_stats', {})
        vs_uncertain_ratio = float(last_vs.get('uncertain_ratio', 0.0)) if isinstance(last_vs, dict) else 0.0
        vs_total = int(last_vs.get('total_checked', 0)) if isinstance(last_vs, dict) else 0
        if int(state['metrics'].get('verify_uncertain_streak', 0)) >= 2:
            state['metrics']['verify_cooldown'] = max(int(state['metrics'].get('verify_cooldown', 0)), 2)

        # Penalize VERIFY when nothing to verify, or during cooldown, or when recent verifies were mostly uncertain
        for p in proposals:
            if p.get('action_type') == 'VERIFY' or p.get('module') == 'EPISTEMIC_GATE':
                if pending_claims <= 0:
                    p['cost'] = float(p.get('cost', 0.0)) + 1.2
                if int(state['metrics'].get('verify_cooldown', 0)) > 0:
                    p['cost'] = float(p.get('cost', 0.0)) + 0.9
                if vs_total >= 3 and vs_uncertain_ratio >= 0.85:
                    p['cost'] = float(p.get('cost', 0.0)) + 0.6

        # Guarantee minimum proposal diversity (avoid Generated 1 proposals)
        atypes = {p.get('action_type') for p in proposals}
        if 'WORLD_ACT' not in atypes:
            proposals.append({
                'proposal_id': f"world_explore_{tick_num}",
                'module': 'WORLD',
                'intent': 'World explore: toggle lamp (evidence probe)',
                'action_type': 'WORLD_ACT',
                'action': {'act': 'toggle', 'target': 'lamp'},
                'expected_utility': get_calibrated_utility('WORLD_ACT', user_present=bool(msg), has_goal=False)*0.85,
                'risk': 0.10,
                'cost': 0.35
            })
        if 'REFLECT' not in atypes:
            proposals.append({
                'proposal_id': f"reflect_fallback_{tick_num}",
                'module': 'REFLECTOR',
                'intent': 'Reflect briefly and plan next action',
                'action_type': 'REFLECT',
                'expected_utility': get_calibrated_utility('REFLECT'),
                'risk': 0.05,
                'cost': 0.30
            })

            # v5.4.3: Sleep fallback only when nothing urgent is active AND energy is low
            try:
                ws = state.get('workspace', {}) or {}
                has_open_event = bool([e for e in (ws.get('events', []) or []) if str(e.get('status','')).upper() not in ('CLOSED','ARCHIVED')])
                top_intent = None
                try:
                    top_intent = intent_manager.get_top_intent(state)
                except Exception:
                    top_intent = None
                has_intent = bool(top_intent)
                energy = float((state.get('drives', {}) or {}).get('energy', 1.0))
                if (not has_open_event) and (not has_intent) and ('SLEEP' not in atypes) and energy < 0.35:
                    proposals.append({
                        'proposal_id': f"sleep_fallback_{tick_num}",
                        'module': 'SLEEP',
                        'intent': 'Short sleep to restore energy',
                        'action_type': 'SLEEP',
                        'expected_utility': get_calibrated_utility('WORLD_ACT', user_present=bool(msg), has_goal=False)*0.85,
                        'risk': 0.05,
                        'cost': 0.15
                    })
            except Exception:
                pass

        # --- v4.1 INTENT COMPLETION LAW: discourage non-resolving actions when an open event exists
        if current_event:
            for p in proposals:
                if p.get('action_type') in ['VERIFY','SLEEP']:
                    p['cost'] = float(p.get('cost',0.0)) + 0.45
        # --- v4.3 STUCK-BREAKER: if META no-op repeats, force exploratory WORLD_ACT
        try:
            if int(state.get('counters', {}).get('meta_noop_streak', 0)) >= 2:
                for p in proposals:
                    if p.get('module') == 'META' or p.get('action_type') == 'META':
                        p['cost'] = float(p.get('cost', 0.0)) + 0.75
                    if p.get('action_type') == 'WORLD_ACT':
                        p['expected_utility'] = float(p.get('expected_utility', 0.0)) + 0.35
        except Exception:
            pass
        # --- v5.1 Self-aware proposals + self-modulated policy
        try:
            add_self_aware_proposals(state, proposals, tick_num)
            modulate_by_self_model(state, proposals)

            # v5.2: global bias from policy_knobs
            wab = float((state.get('policy_knobs', {}) or {}).get('world_act_bias', 0.0))
            if wab > 0:
                for p in proposals:
                    if p.get('action_type') == 'WORLD_ACT':
                        p['expected_utility'] = float(p.get('expected_utility', 0.0)) + wab
        except Exception:
            pass


        # v5.4: Strong Goal Bias when an active WORLD_GOAL intent exists (pre-arbitration)
        try:
            top_int = intent_manager.get_top_intent(state) if 'intent_manager' in globals() else None
            if top_int and str(top_int.get('intent_type','')).upper() == 'WORLD_GOAL':
                for p in proposals:
                    if p.get('action_type') == 'WORLD_ACT':
                        p['expected_utility'] = float(p.get('expected_utility', 0.0)) + 1.80
                        p['cost'] = max(0.0, float(p.get('cost', 0.0)) - 0.10)
                    elif p.get('action_type') in ['SLEEP','SELF_MAINTENANCE']:
                        p['cost'] = float(p.get('cost', 0.0)) + 0.80
        except Exception:
            pass


        # v5.7.4: Gate autonomous WORLD_ACT to prevent random object toggles
        try:
            goals_list = (state.get('temporal_thread', {}) or {}).get('active_goals', []) or []
            has_unmet_goal = any((not goal_satisfied(g, state)) for g in goals_list) if 'goal_satisfied' in globals() else bool(goals_list)
            ws2 = state.get('workspace', {}) or {}
            # Consider "user present" if there is a focused/open USER event (even when no simulated msg was injected)
            cur_id2 = ws2.get('current_event_id')
            cur_ev2 = None
            for _e in (ws2.get('events', []) or []):
                if _e.get('id') == cur_id2:
                    cur_ev2 = _e
                    break
            has_user_focus = bool(isinstance(cur_ev2, dict) and str(cur_ev2.get('source','')).strip().upper() == 'USER'
                                 and str(cur_ev2.get('status','')).strip().upper() not in ('CLOSED','ARCHIVED'))
            has_any_open_user = any(
                str(_e.get('source','')).strip().upper() == 'USER' and str(_e.get('status','')).strip().upper() in ('NEW','ACTIVE','INTERPRETED','RETRY')
                for _e in (ws2.get('events', []) or [])
            )
            user_present = bool(msg) or bool(user_input_injected) or has_user_focus or has_any_open_user or bool(ws2.get('user_event_pending', False))
            if (not user_present) and (not has_unmet_goal):
                proposals = [p for p in proposals if str(p.get('action_type','')).upper() not in ('WORLD_ACT',)]
            # If user is absent, penalize WORLD_ACT even when kept (soft gate)
            if (not user_present):
                for p in proposals:
                    if str(p.get('action_type','')).upper() == 'WORLD_ACT':
                        p['cost'] = float(p.get('cost', 0.0)) + 0.25
                        p['expected_utility'] = float(p.get('expected_utility', 0.0)) * 0.70
        except Exception:
            pass

# v5.3.2: proposals consistency instrumentation
        try:
            logger.log(f"[DEBUG] proposals_before_arbitration={len(proposals)}")
            for i, p in enumerate(proposals[:12]):
                logger.log(f"[DEBUG] p{i}: {p.get('action_type')} module={p.get('module','')} EU={float(p.get('expected_utility',0.0)):.2f} cost={float(p.get('cost',0.0)):.2f}")
        except Exception:
            pass

        # v5.3.2: force ANSWER when policy requires it
        try:
            if int((state.get('policy', {}) or {}).get('force_answer_ticks', 0)) > 0:
                for p in proposals:
                    if p.get('action_type') == 'ANSWER':
                        p['expected_utility'] = float(p.get('expected_utility', 0.0)) + 1.50
                        p['cost'] = max(0.0, float(p.get('cost', 0.0)) - 0.10)
                    if p.get('action_type') == 'SELF_MAINTENANCE':
                        p['cost'] = float(p.get('cost', 0.0)) + 0.60
        except Exception:
            pass

        # v5.7.3: Calibrate expected utilities (reduces huge prediction errors) + break repetition loops
        try:
            has_goal = bool((state.get('temporal_thread', {}) or {}).get('active_goals')) or bool((state.get('gw_frame', {}) or {}).get('goal'))
            is_question = False
            try:
                is_question = bool(current_event) and str(current_event.get('event_type','')).upper() == 'QUESTION'
            except Exception:
                is_question = False
            user_present = bool(user_input_injected) or (bool(current_event) and str(current_event.get('source','')).lower() == 'user')
            for p in proposals:
                at = str(p.get('action_type','')).upper()
                p['expected_utility'] = get_calibrated_utility(at, user_present=user_present, is_question=is_question, has_goal=has_goal)
            maybe_force_break_repetition(state, proposals)
        except Exception:
            pass
        # Arbitrate
        winner = self.arbiter.arbitrate(proposals, state)

        if winner:
            # Update PB
            state['pb']['pb_seq'] += 1
            state['pb']['now_id'] = winner['proposal_id']
            state['pb']['summary'] = winner['intent']
            state['pb']['confidence'] = state['metacog']['global_confidence']

            # Apply additional novelty gain if EXPLORER wins
            if winner['module'] == 'EXPLORER':
                novelty_gain += 0.08 # Explorer action adds novelty

            # Update winner streak
            ws = state['metrics'].get('winner_streak', {'module': None, 'count': 0})
            if ws.get('module') == winner.get('module'):
                ws['count'] = int(ws.get('count',0)) + 1
            else:
                ws['module'] = winner.get('module')
                ws['count'] = 1
            state['metrics']['winner_streak'] = ws

            # LOOP BREAKER: if epistemic gate wins too many times, force a sleep/verify cycle
            if ws.get('module') == 'EPISTEMIC_GATE' and ws.get('count',0) >= 5:
                logger.log('🧯 Loop breaker: breaking loops (prefer REFLECT; sleep only if idle)')
                winner = {'proposal_id': f"forced_sleep_{tick_num}", 'module': 'SLEEP', 'intent': 'Forced sleep for loop breaker', 'action_type': 'SLEEP', 'expected_utility': 0.9, 'risk': 0.05, 'cost': 0.2}
                ws['count'] = 0
                state['metrics']['winner_streak'] = ws


            # --- v4.1 INTENT COMPLETION LAW: if we keep avoiding resolution, force a resolving action
            if current_event and winner.get('action_type') in ['VERIFY','SLEEP']:
                current_event['rounds'] = int(current_event.get('rounds',0)) + 1
                if current_event['rounds'] >= 2:
                    # pick best resolving proposal if available, else force REFLECT
                    resolving = [p for p in proposals if p.get('action_type') not in ['VERIFY','SLEEP']]
                    if resolving:
                        winner = max(resolving, key=lambda x: x.get('expected_utility',0.0))
                        logger.log("⚡ Intent completion: forcing a resolving action")
                    else:
                        winner = {'proposal_id': f"forced_reflect_{tick_num}", 'module': 'REFLECTOR', 'intent': 'Forced reflect to close event', 'action_type': 'REFLECT', 'expected_utility': 0.8, 'risk': 0.1, 'cost': 0.2}

            # v5.2.5 ANTI-SLEEP-STREAK: avoid repeated SLEEP when goals/events pending
            try:
                has_goal = bool((state.get('temporal_thread', {}) or {}).get('active_goals'))
                if winner.get('action_type') == 'SLEEP' and (has_goal or current_event):
                    if int((state.get('counters', {}) or {}).get('sleep_streak', 0)) >= 1:
                        resolving2 = [p for p in proposals if p.get('action_type') not in ['SLEEP']]
                        if resolving2:
                            winner = max(resolving2, key=lambda x: float(x.get('expected_utility',0.0)))
                            logger.log('⚡ Push-through: overriding repeated SLEEP with resolving action')
            except Exception:
                pass
            # Execute
            result = ActionExecutor.execute(winner, state, self.llm)

            # v5.5: Active inference mini-loop + stuck detector (anti-compulsion)
            try:
                ws = state.get('workspace', {}) or {}
                focus_key = str(ws.get('current_event_id') or '') or str((state.get('gw_frame', {}) or {}).get('current_intent_id') or '')
                out_key = str(result.get('output','')).strip().lower()[:80]
                sig = (focus_key, str(winner.get('action_type','')), out_key)
                delta = int(result.get('delta', result.get('world_delta', 0)) or 0)
                progressed = bool(delta > 0) or str(result.get('completion','DONE')).upper() == 'DONE'
                st_res = stuck_update(state, sig, progressed=progressed)
                if st_res.get('stuck') and current_event is not None:
                    # Mark event BLOCKED and create a clarification/help event
                    current_event['status'] = 'BLOCKED'
                    current_event['last_update_tick'] = int(tick_num)
                    logger.log(f"🧱 STUCK detected (count={st_res.get('count')}) → event BLOCKED: {current_event.get('id')}")
                    # Create a meta-event that forces asking user / replanning
                    meta = create_event(make_stuck_replan_prompt(state, current_event, result), tick_num, priority=1.5, ttl=20, source='system')
                    meta['event_type'] = 'QUESTION'
                    state.setdefault('workspace', {}).setdefault('events', []).append(meta)
                    # Also nudge policy to answer next tick
                    state.setdefault('policy', {})
                    state['policy']['force_answer_ticks'] = max(int(state['policy'].get('force_answer_ticks',0)), 2)
            except Exception:
                pass

            # v5.3.2: attach completion to current event for completion guard
            try:
                if current_event is not None:
                    current_event['_last_completion'] = str(result.get('completion', 'DONE'))
            except Exception:
                pass



            # v5.4.3: atomic close of focused event + its matching intent (prevents stale scene / repeated answers)
            try:
                if current_event is not None:
                    comp = str(result.get('completion','DONE')).upper()
                    current_event['_last_completion'] = comp
                    if comp == 'DONE':
                        close_event(current_event, tick_num)
                        # If there is an intent sourced from this event, mark it done too
                        try:
                            for it in (state.get('intent', {}) or {}).get('active', []) or []:
                                if str(it.get('source_event_id','')) == str(current_event.get('id','')) and str(it.get('status','')).upper() == 'ACTIVE':
                                    intent_manager.mark_done(state, it.get('id'), tick_num, outcome='DONE')
                        except Exception:
                            pass
                    else:
                        try:
                            mark_acted(current_event, tick_num)
                        except Exception:
                            pass
            except Exception:
                pass

# v5.4: intent lifecycle update from outcomes (close the loop between intent -> action -> outcome)
            try:
                top_int = intent_manager.get_top_intent(state) if 'intent_manager' in globals() else None
                if top_int:
                    # outcome classification
                    completion = str(result.get('completion','DONE')).upper()
                    success = bool(result.get('success', True))
                    delta = int(result.get('delta', result.get('world_delta', 0)) or 0)
                    outcome = 'DONE' if (completion == 'DONE' and success and (delta > 0 or winner.get('action_type') == 'ANSWER')) else 'RETRY'
                    intent_manager.record_attempt(state, top_int.get('id'), tick_num, outcome=('FAIL' if not success else outcome))
                    # mark done if we actually changed the world for a goal intent, or answered a question intent
                    if str(top_int.get('intent_type','')).upper() == 'WORLD_GOAL' and delta > 0 and success:
                        intent_manager.mark_done(state, top_int.get('id'), tick_num, outcome='DONE')
                    if str(top_int.get('intent_type','')).upper() == 'QUESTION' and winner.get('action_type') == 'ANSWER' and success:
                        intent_manager.mark_done(state, top_int.get('id'), tick_num, outcome='DONE')
            except Exception:
                pass


            # v5.3.1: no-op / tech-fail diagnostics + self-correct push-through
            try:
                state.setdefault('no_op_streak', 0)
                tech_fail = bool(result.get('tech_fail', False))
                out_text = str(result.get('output','')).lower()
                is_noop = ('no-op' in out_text) or (result.get('status') in ['noop'] )
                # treat WORLD with delta=0 as no-op
                if winner.get('action_type') == 'WORLD_ACT' and int(result.get('delta', 0)) == 0:
                    is_noop = True
                if is_noop or tech_fail:
                    state['no_op_streak'] = int(state.get('no_op_streak', 0)) + 1
                else:
                    state['no_op_streak'] = 0

                # If stuck and there is an active goal, force WORLD for a few ticks
                try:
                    ag = goal_manager.get_highest(state) if 'goal_manager' in globals() else None
                except Exception:
                    ag = None
                if ag and int(state.get('no_op_streak', 0)) >= 4 and float(state.get('drives',{}).get('energy',1.0)) > 0.25:
                    knobs = state.get('policy_knobs', {}) or {}
                    knobs['force_world_ticks'] = max(int(knobs.get('force_world_ticks', 0)), 5)
                    knobs['world_bias_boost'] = max(float(knobs.get('world_bias_boost', 0.0)), 2.0)
                    state['policy_knobs'] = knobs
                    state.setdefault('inner_monologue', {})['text'] = "I notice I'm no-opping user goals. Forcing WORLD actions to complete the goal."
                # v5.3.2: if stuck on ANY open user event (questions or tasks), boost ANSWER/WORLD
                try:
                    if int(state.get('no_op_streak', 0)) >= 4:
                        ws_events = (state.get('workspace', {}) or {}).get('events', []) or []
                        open_events2 = [e for e in ws_events if e.get('status') != 'CLOSED']
                        open_user_events2 = []
                        for e in open_events2:
                            eid2 = str(e.get('id',''))
                            src2 = str(e.get('source',''))
                            if eid2.startswith('self_thought_') or src2 == 'SELF_THOUGHT':
                                continue
                            open_user_events2.append(e)
                        if open_user_events2:
                            state.setdefault('policy', {})
                            state['policy']['force_answer_ticks'] = max(int((state['policy'] or {}).get('force_answer_ticks', 0)), 3)
                            knobs = state.get('policy_knobs', {}) or {}
                            knobs['world_bias_boost'] = max(float(knobs.get('world_bias_boost', 0.0)), 1.5)
                            state['policy_knobs'] = knobs
                except Exception:
                    pass

            except Exception:
                pass
            # v5.3: update volition after outcome (streaks + commitment)
            try:
                update_volition_after_outcome(state, winner, result, tick_num)
            except Exception:
                pass

            # v5.2.5 TECH_RETRY: if world bridge errors, keep event open and retry (limited)
            try:
                if current_event and bool(result.get('tech_fail', False)):
                    current_event['tech_retries'] = int(current_event.get('tech_retries', 0)) + 1
                    current_event['last_error'] = result.get('output','')
                    # Force REPAIR mode (but do not close the event)
                    state.setdefault('pb', {}).setdefault('mode', 'REPAIR')
                    state['pb']['mode'] = 'REPAIR'
                    if current_event['tech_retries'] <= 2:
                        current_event['status'] = 'RETRY'
                        current_event['priority'] = max(float(current_event.get('priority', 1.0)), 2.0)
                        logger.log(f"🧰 TECH_FAIL: keeping event open for retry ({current_event['tech_retries']}/2)")
                    else:
                        current_event['status'] = 'FAILED_TECH'
                        logger.log('🧰 TECH_FAIL: retry budget exhausted; marking FAILED_TECH')
            except Exception:
                pass


            # --- v5.0 Goal resolution & agency reward
            try:
                goals = state.get('temporal_thread', {}).get('active_goals', [])
                if goals:
                    for gi, gg in enumerate(list(goals)):
                        if goal_satisfied(gg, state):
                            state['temporal_thread']['goal_history'].append({**gg, 'completed_tick': tick_num})
                            del state['temporal_thread']['active_goals'][gi]
                            result['reward'] = float(result.get('reward', 0.0)) + 0.30
                            logger.log(f"✅ Goal completed: {gg.get('target')} -> {gg.get('desired')} (+0.30)")
                            break
            except Exception:
                pass
            # ACTIVE INFERENCE: Prediction-Outcome Loop
            sandbox_state, actual_reward, prediction_error, valence = apply_active_inference(
                state, winner, result, sandbox, logger
            )

            # UPDATE CLAIM LEDGER
            update_claim_ledger(state, result, actual_reward, origin_action=winner['action_type'], triggered_by_user=user_input_injected)            # AUTO-VERIFY: if this tick executed VERIFY, run claim-ledger verification now
            if winner.get('action_type') == 'VERIFY' and 'verify_claim_ledger' in globals():
                vr = drain_verify_queue(state, llm=self.llm, batch=25, max_rounds=6) if 'drain_verify_queue' in globals() else verify_claim_ledger(state, llm=self.llm, max_to_verify=25)
                logger.log(f"🔎 Verify pass=+{vr.get('verified',0)} fail=+{vr.get('failed',0)} uncertain=+{vr.get('uncertain',0)}")

            logger.log(f"Executed: {result['output']}")

            # --- v5.2.5 Update sleep streak
            try:
                if winner.get('action_type') == 'SLEEP':
                    state['counters']['sleep_streak'] = int((state.get('counters', {}) or {}).get('sleep_streak',0)) + 1
                else:
                    state['counters']['sleep_streak'] = 0
            except Exception:
                state.setdefault('counters', {})
                state['counters']['sleep_streak'] = 0

            # --- v4.3 META no-op tracking
            try:
                if (winner.get('module') == 'META' or winner.get('action_type') == 'META') and (result.get('output','').strip().lower() in ['no action', '']):
                    state['counters']['meta_noop_streak'] = int(state['counters'].get('meta_noop_streak', 0)) + 1
                else:
                    state['counters']['meta_noop_streak'] = 0
            except Exception:
                state['counters']['meta_noop_streak'] = 0

            # --- v4.3.1 TELEMETRY snapshot (for dashboards)
            try:
                state.setdefault('telemetry', {}).setdefault('ticks', [])
                w = (state.get('workspace', {}) or {}).get('world', {})
                objs = (w.get('objects', {}) if isinstance(w, dict) else {})
                world_states = {k: (objs.get(k, {}).get('state') if isinstance(objs.get(k, {}), dict) else None)
                                for k in ['lamp', 'box', 'door']}
                snap = {
                    'tick': tick_num,
                    'mode': (state.get('pb', {}) or {}).get('mode'),
                    'C_total': float((state.get('metrics', {}) or {}).get('C_total', 0.0)),
                    'Ce': float((state.get('metrics', {}) or {}).get('Ce', 0.0)),
                    'energy': float((state.get('drives', {}) or {}).get('energy', 0.0)),
                    'mood': float((state.get('affect', {}) or {}).get('mood', 0.0)),
                    'action_type': (winner.get('action_type') or winner.get('module')),
                    'reward': float(result.get('reward', 0.0)),
                    'meta_noop_streak': int((state.get('counters', {}) or {}).get('meta_noop_streak', 0)),
                    'world': world_states,
                    'scene': (state.get('workspace', {}) or {}).get('scene', ''),
                }
                state['telemetry']['ticks'].append(snap)
            except Exception:
                pass

            # --- v5.1 Self-model update + conscious trace (post-outcome)
            try:
                update_self_model_after_outcome(state, winner, result)
                append_conscious_trace(state, winner, result, tick_num)
            except Exception:
                pass

            # --- v4.1 EVENT LAW (v5.2.5): close event after resolving actions (but keep open on TECH_FAIL)
            if current_event and winner.get('action_type') in ['REFLECT','TOOL_CALL','RETRIEVE','WORLD_ACT']:
                try:
                    if bool(result.get('tech_fail', False)):
                        # keep event open for retry
                        pass
                    else:
                        # If world action failed but there is an active goal, allow a couple more rounds
                        if winner.get('action_type') == 'WORLD_ACT' and result.get('status') in ['fail','error'] and (state.get('temporal_thread', {}) or {}).get('active_goals'):
                            current_event['rounds'] = int(current_event.get('rounds',0)) + 1
                            if current_event['rounds'] < 3:
                                logger.log('🔁 Keeping event open: goal still active after WORLD_ACT failure')
                            else:
                                mark_acted(current_event, tick_num)
                                close_event(current_event, tick_num)
                                logger.log(f"✅ Event closed: {current_event['id']}")
                        else:
                            mark_acted(current_event, tick_num)
                            close_event(current_event, tick_num)
                            logger.log(f"✅ Event closed: {current_event['id']}")
                except Exception:
                    mark_acted(current_event, tick_num)
                    close_event(current_event, tick_num)
                    logger.log(f"✅ Event closed: {current_event['id']}")
                # v4.2.4 cleanup: remove closed events from spotlight and active queue
                try:
                    events = state.get('workspace', {}).get('events', [])
                    state['workspace']['events'] = [e for e in events if e.get('status') != 'CLOSED']
                    sp = state.get('attention', {}).get('spotlight', [])
                    state['attention']['spotlight'] = [x for x in sp if x != current_event.get('id')]
                except Exception:
                    pass


            # Track agency (based on winning proposal, internal action)
            # Track agency (distinguish self-initiated vs externally-triggered)
            authorship = 'external_triggered' if user_input_injected else 'self_initiated'
            state['agency']['authorship_log'].append({
                'tick': tick_num,
                'action': winner['action_type'],
                'authorship': authorship,
                'triggered_by_user': bool(user_input_injected),
                'now_id': state['pb'].get('now_id')
            })

            # If SLEEP mode was entered, reset cooldown timer
            if winner['action_type'] == 'SLEEP':
                state['sleep_cooldown_timer'] = Config.SLEEP_COOLDOWN_TICKS
        # Update loop risk
        try:
            traj = (state.get('attention', {}) or {}).get('trajectory', [])
            if isinstance(traj, list) and len(traj) > 5:
                recent = traj[-5:]
                recent_modes = [x.get('mode') for x in recent if isinstance(x, dict) and x.get('mode') is not None]
                if recent_modes and len(set(recent_modes)) == 1:
                    state['loop_risk'] = float(state.get('loop_risk', 0.0)) + 0.1
                else:
                    state['loop_risk'] = float(state.get('loop_risk', 0.0)) * 0.9
        except Exception:
            pass
        # Save periodically
        if tick_num % 5 == 0:
            self.state_manager.save()
            logger.log("State saved")

        logger.log(f"Tick {tick_num} complete\n")

    def run(self, max_ticks: int = Config.MAX_TICKS):
        logger.log(f"Starting core loop for {max_ticks} ticks...")

        try:
            for _ in range(max_ticks):
                self.tick()
                if not self.running:
                    break
                time.sleep(Config.TICK_INTERVAL)
        except KeyboardInterrupt:
            logger.log("Interrupted")
        finally:
            self.state_manager.save()
            logger.log("=== Session Complete ===")

print("✓ Core loop ready")

✓ Core loop ready


## v4.2.5 — How to provide inputs

Edit `TEST_INPUTS` in the next cell. Then run the notebook normally.
By default, it will inject those messages as events (low compute).
Set `RUN_AUTOPILOT = True` to run the old long autonomous loop.


In [33]:
# === v4.2.5 Input Injection (no interactive prompt needed) ===
# Put your test messages here. The runner will feed them as EVENTS.
TEST_INPUTS = [
    "Turn on the lamp",
    "Turn off the lamp",
    "Open the box",
    "Close the box",
    "Open the door",
    "Unlock the door",
    "Open the door",
]

TICKS_PER_INPUT = 6   # keep low to save compute (e.g., 4-8)
RUN_AUTOPILOT = False # set True to run the old long autonomous loop

def enqueue_user_event(state, msg: str, tick: int = 0):
    state.setdefault('workspace', {}).setdefault('events', [])
    cur_tick = int(state.get('tick_count', tick) or tick)
    ev_type = 'STATEMENT'
    try:
        ev_type = classify_user_input(msg)
    except Exception:
        pass
    pri = 1.0
    if ev_type == 'QUESTION':
        pri = 1.2
    elif ev_type == 'WORLD_GOAL':
        pri = 1.4
    ev = create_event(msg, cur_tick, priority=pri, source='user')
    ev['event_type'] = ev_type
    ev['source'] = 'user'
    ev['status'] = 'NEW'
    ev['rounds'] = 0
    state['workspace']['events'].append(ev)
    return ev

def run_with_test_inputs(state_manager, core_loop, inputs=TEST_INPUTS, ticks_per_input=TICKS_PER_INPUT):
    for msg in inputs:
        print(f"\n>>> Injecting user input: {msg}")
        enqueue_user_event(state_manager.state, msg, tick=0)
        state_manager.save()
        core_loop.run(max_ticks=ticks_per_input)
    print("\n=== Test-input session complete ===")


## v4.3.1 Dashboards

Two views:
- **Scientific Dashboard** (metrics/time series)
- **Human-friendly Room View** (lamp/box/door state)
- Optional quick **Animation**


In [ ]:
# CELL 17: Initialize and Run
print("\n" + "="*60)
print("CR-SSCP v5.7.10 - Consciousness-like Cognitive Architecture")
print("="*60 + "\n")

core_loop = CoreLoop(state_manager, llm)

# Print initial state
print(f"Initial Coherence: {state_manager.state['coherence']['C_total']:.3f}")
print(f"Initial Energy: {state_manager.state['drives']['energy']:.2f}")
print(f"Initial Emotion: {state_manager.state['affect']['current_emotion']}")
print(f"Mode: {state_manager.state['pb']['mode']}")
print(f"\nIdentity anchors:")
for anchor in state_manager.state['narrative']['identity_anchors']:
    print(f"  - {anchor}")

print(f"\nRunning {Config.MAX_TICKS} ticks (~{Config.MAX_TICKS * Config.TICK_INTERVAL // 60} minutes)...\n")

# Run the loop
if RUN_AUTOPILOT:
    core_loop.run(max_ticks=Config.MAX_TICKS)
else:
    run_with_test_inputs(state_manager, core_loop)


CR-SSCP v5.7.10 - Consciousness-like Cognitive Architecture

Initial Coherence: 0.500
Initial Energy: 0.85
Initial Emotion: curious
Mode: REFLECT

Identity anchors:
  - I am an experimental cognitive architecture
  - I aim to maintain coherence and avoid hallucinations
  - I learn from evidence and admit uncertainty

Running 100 ticks (~8 minutes)...


>>> Injecting user input: Turn on the lamp
[2026-02-19 10:26:00] Starting core loop for 6 ticks...
[2026-02-19 10:26:00] 
[2026-02-19 10:26:00] TICK 1
[2026-02-19 10:26:00] ============================================================
[2026-02-19 10:26:00] Attention spotlight: ['event_02403e10']
[2026-02-19 10:26:00] Coherence C_total: 0.590
[2026-02-19 10:26:00] Mode: REFLECT
[2026-02-19 10:26:00] Energy: 0.84, Coherence: 0.78, Novelty: 0.73
[2026-02-19 10:26:00] Emotion: curious, Mood: 0.51
[2026-02-19 10:26:00] Generated 1 proposals
[2026-02-19 10:26:00] [DEBUG] proposals_before_arbitration=2
[2026-02-19 10:26:00] [DEBUG] p0: WORLD_AC

In [ ]:
# CELL 18: Analysis and Metrics
print("\n" + "="*60)
print("SESSION ANALYSIS")
print("="*60 + "\n")

final_state = state_manager.state

print(f"Total ticks: {final_state['tick_count']}")
print(f"Sleep cycles: {final_state['sleep_count']}")
print(f"Mode flips: {final_state.get('metrics', {}).get('mode_flip_count', 0)}")

print(f"\nFinal Metrics:")
print(f"  Coherence (C_total): {final_state['coherence']['C_total']:.3f}")
print(f"  - Evidence (Ce): {final_state['coherence']['Ce']:.3f}")
print(f"  - Historical (Ch): {final_state['coherence']['Ch']:.3f}")
print(f"  - Structural (Cs): {final_state['coherence']['Cs']:.3f}")
print(f"  - Identity (Ci): {final_state['coherence']['Ci']:.3f}")
print(f"  - Predictive (Cp): {final_state['coherence']['Cp']:.3f}")

print(f"\nDrive States:")
for drive, value in final_state['drives'].items():
    print(f"  {drive}: {value:.2f}")

print(f"\nAffective State:")
print(f"  Emotion: {final_state['affect']['current_emotion']}")
print(f"  Mood: {final_state['affect']['mood']:.2f}")

print(f"\nMemory:")
print(f"  Grounded facts: {len(final_state['memory']['grounded'])}")
print(f"  Ungrounded notes: {len(final_state['memory']['ungrounded'])}")
print(f"  Quarantined: {len(final_state['memory']['quarantine'])}")

print(f"\nAgency:")
self_actions = sum(1 for a in final_state['agency']['authorship_log'] if a['authorship'] == 'self_initiated')
total_actions = len(final_state['agency']['authorship_log'])
external_actions = sum(1 for a in final_state['agency']['authorship_log'] if a['authorship'] == 'external_triggered')
print(f"  Self-initiated actions: {self_actions}/{total_actions}")
print(f"  External-triggered actions: {external_actions}/{total_actions}")
if total_actions > 0:
    print(f"  Causal Closure Ratio (self-initiated/total): {self_actions/total_actions:.2%}")

print(f"\nClaim Ledger:")
print(f"  Total claims: {len(final_state['claim_ledger'])}")
verified = sum(1 for c in final_state['claim_ledger'] if c.get('verifier_result') == 'pass')
failed = sum(1 for c in final_state['claim_ledger'] if c.get('verifier_result') == 'fail')
uncertain = sum(1 for c in final_state['claim_ledger'] if c.get('verifier_result') == 'uncertain')
pending = sum(1 for c in final_state['claim_ledger'] if c.get('verifier_result') == 'pending')
print(f"  Verified claims: {verified}")
print(f"  Failed claims: {failed}")
print(f"  Uncertain claims: {uncertain}")
print(f"  Pending claims: {pending}")

print(f"\nNarrative:")
print(f"  Current arc: {final_state['narrative']['current_arc']['direction']}")
print(f"  Theme: {final_state['narrative']['current_arc']['meaning']}")

print("\n" + "="*60)
print(f"State saved to: {Config.STATE_PATH}")
print(f"Logs saved to: {Config.LOG_PATH}")
print("="*60)

In [ ]:
# === Scientific Dashboard ===
import pandas as pd
import matplotlib.pyplot as plt

ticks = (state_manager.state.get('telemetry', {}) or {}).get('ticks', [])
df = pd.DataFrame(ticks)

if df.empty:
    print("No telemetry yet. Run some ticks first.")
else:
    # Core metrics
    plt.figure()
    plt.plot(df['tick'], df['C_total'])
    plt.plot(df['tick'], df['Ce'])
    plt.plot(df['tick'], df['energy'])
    plt.plot(df['tick'], df['mood'])
    plt.xlabel("tick")
    plt.ylabel("value")
    plt.title("Core Metrics Over Time")
    plt.legend(["C_total","Ce","energy","mood"])
    plt.show()

    # Reward
    plt.figure()
    plt.plot(df['tick'], df['reward'])
    plt.xlabel("tick")
    plt.ylabel("reward")
    plt.title("Reward Over Time")
    plt.show()

    # Action timeline (categorical)
    actions = df['action_type'].fillna("None").astype(str)
    cats = {a:i for i,a in enumerate(sorted(actions.unique()))}
    y = actions.map(cats)

    plt.figure()
    plt.plot(df['tick'], y)
    plt.yticks(list(cats.values()), list(cats.keys()))
    plt.xlabel("tick")
    plt.title("Action Type Timeline")
    plt.show()

    # META no-op streak
    plt.figure()
    plt.plot(df['tick'], df['meta_noop_streak'])
    plt.xlabel("tick")
    plt.ylabel("streak")
    plt.title("META No-Op Streak")
    plt.show()


In [ ]:
# === Human-friendly Room View ===
from IPython.display import display, Markdown

def pretty_room(world: dict) -> str:
    objs = (world.get('objects', {}) if isinstance(world, dict) else {})
    lamp = (objs.get('lamp', {}) or {}).get('state', '?')
    box  = (objs.get('box', {}) or {}).get('state', '?')
    door = (objs.get('door', {}) or {}).get('state', '?')

    lamp_icon = "💡" if lamp == "on" else "🔌"
    box_icon  = "📦(open)" if box == "open" else "📦(closed)"
    if door == "open":
        door_icon = "🚪(open)"
    elif door == "unlocked":
        door_icon = "🔓🚪"
    else:
        door_icon = "🔒🚪"

    return f"""
**Sandbox Room**
- Lamp: {lamp_icon} `{lamp}`
- Box: {box_icon} `{box}`
- Door: {door_icon} `{door}`
"""

w = (state_manager.state.get('workspace', {}) or {}).get('world', {})
display(Markdown(pretty_room(w)))

# Show last few telemetry lines as a story
ticks = (state_manager.state.get('telemetry', {}) or {}).get('ticks', [])
for t in ticks[-12:]:
    print(f"[t{t.get('tick')}] action={t.get('action_type')} reward={t.get('reward'):+.3f} world={t.get('world')}")


In [ ]:
# === Quick Animation (optional) ===
import time
from IPython.display import clear_output

ticks = (state_manager.state.get('telemetry', {}) or {}).get('ticks', [])
if not ticks:
    print("No telemetry yet. Run some ticks first.")
else:
    for t in ticks[-30:]:
        clear_output(wait=True)
        world = t.get('world', {})
        print(f"TICK {t.get('tick')} | action={t.get('action_type')} | reward={t.get('reward'):+.3f}")
        print("lamp:", world.get('lamp'), "box:", world.get('box'), "door:", world.get('door'))
        time.sleep(0.2)


## v5.5 Notes
This notebook adds a Global Workspace Frame (single broadcast), active-inference style expected/actual updates (minimal), an anti-compulsion stuck detector, episodic memory logging, and stronger intent commitments.
